## Data load

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

In [4]:
same_df = pd.read_csv('/content/drive/MyDrive/Time Series Project/Stock Price/data/one_hot_same.csv')
int_df = pd.read_csv('/content/drive/MyDrive/Time Series Project/Stock Price/data/one_hot_int.csv')

#### To make Boolean

In [5]:
same_df['Covid19'] = same_df['Covid19'].apply(lambda x: 1 if x else 0)
int_df['Covid19'] = int_df['Covid19'].apply(lambda x: 1 if x else 0)

In [6]:
boolean_columns = [
    "Covid19",
    "US_President_Republican",
    "US_President_Democratic",
    "KR_President_Democratic",
    "KR_President_Conservative",
    "KR_President_Neutral"
]

same_df[boolean_columns] = same_df[boolean_columns].astype(bool)
int_df[boolean_columns] = int_df[boolean_columns].astype(bool)

In [7]:
same_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5865 entries, 0 to 5864
Data columns (total 37 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Date                       5865 non-null   object 
 1   Open                       5865 non-null   float64
 2   High                       5865 non-null   float64
 3   Low                        5865 non-null   float64
 4   Close                      5865 non-null   float64
 5   Volume                     5865 non-null   float64
 6   Change                     5865 non-null   float64
 7   Transaction                5865 non-null   float64
 8   SE_Operating_Profit        5865 non-null   float64
 9   SE_Net_Profit              5865 non-null   float64
 10  SE_Total_Equity            5865 non-null   float64
 11  SE_Total_Liabilities       5865 non-null   float64
 12  SE_Revenue                 5865 non-null   float64
 13  SE_Foreign                 5865 non-null   float

#### Set index

In [8]:
same_df.set_index('Date', inplace = True)
int_df.set_index('Date', inplace = True)

#### Normalization(only for LSTM), xy split

In [9]:
features = ['Open', 'High', 'Low', 'Volume', 'Change', 'Transaction','SE_Operating_Profit', 'SE_Net_Profit', 'SE_Total_Equity',
       'SE_Total_Liabilities', 'SE_Revenue', 'SE_Foreign', 'US_Inflation', 'KR_Inflation', 'CN_Inflation', 'US_GDP_Annual', 'KR_GDP_Annual',
       'CN_GDP_Annual', 'US_Economic_Growth_Rate', 'KR_Economic_Growth_Rate', 'US_Base', 'US_10', 'KR_Base', 'KR_10', 'WTI', 'Exchange', 'SOX_Close',
       'Phone', 'KR_Unemployment', 'Covid19', 'US_President_Republican', 'US_President_Democratic', 'KR_President_Democratic', 'KR_President_Conservative', 'KR_President_Neutral']
target = ['Close']

In [10]:
x_same = same_df[features].values
y_same = same_df[target].values

x_scaler, y_scaler = MinMaxScaler(), MinMaxScaler()

x_same_scaled = x_scaler.fit_transform(x_same)
y_same_scaled = y_scaler.fit_transform(y_same.reshape(-1, 1))

In [11]:
print(x_same_scaled.shape, y_same_scaled.shape)

(5865, 35) (5865, 1)


In [12]:
x_int = int_df[features].values
y_int = int_df[target].values

x_scaler, y_scaler = MinMaxScaler(), MinMaxScaler()

x_int_scaled = x_scaler.fit_transform(x_int)
y_int_scaled = y_scaler.fit_transform(y_int.reshape(-1, 1))

#### Train, Test split

In [13]:
N = len(same_df)
print(N)

5865


In [14]:
train_len = int(N*0.6)
val_len = int(N*0.2)
test_len = int(N*0.2)
print(train_len, val_len, test_len)

3519 1173 1173


In [15]:
x_train_same_df = x_same_scaled[:3519]
y_train_same_df = y_same_scaled[:3519]

x_val_same_df = x_same_scaled[3519:4692]
y_val_same_df = y_same_scaled[3519:4692]

x_test_same_df = x_same_scaled[4692:]
y_test_same_df = y_same_scaled[4692:]

In [16]:
print(len(x_train_same_df), len(x_val_same_df), len(x_test_same_df))
print(len(y_train_same_df), len(y_val_same_df), len(y_test_same_df))

3519 1173 1173
3519 1173 1173


In [17]:
x_train_int_df = x_int_scaled[:3519]
y_train_int_df = y_int_scaled[:3519]

x_val_int_df = x_int_scaled[3519:4692]
y_val_int_df = y_int_scaled[3519:4692]

x_test_int_df = x_int_scaled[4692:]
y_test_int_df = y_int_scaled[4692:]

## Model Setting

In [18]:
!pip install wandb -qU

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 90.9 MB/s eta 0:00:00


In [19]:
!pip install tensorflow

In [20]:
import wandb
wandb.login()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
wandb.login(relogin=True)

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [21]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
import wandb
from wandb.integration.keras import WandbCallback
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Bidirectional, Dense, Dropout, Input, Flatten, Concatenate

In [22]:
# GPU가 사용 가능한지 확인
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        # 첫 번째 GPU 메모리를 다 사용할 수 있도록 설정 (필요에 따라 조정 가능)
        tf.config.experimental.set_memory_growth(gpus[0], True)
        print("GPU is set to be used.")
    except RuntimeError as e:
        print(f"Error setting GPU memory growth: {e}")
else:
    print("No GPU found. Using CPU instead.")

GPU is set to be used.


In [23]:
def create_sequences(X, y, time_steps):
    sequences, labels = [], []
    for i in range(len(X) - time_steps):
        sequences.append(X[i:i+time_steps])
        labels.append(y[i+time_steps])
    return np.array(sequences), np.array(labels)

def rmse_metric(y_true, y_pred):
    return tf.sqrt(tf.reduce_mean(tf.square(y_true - y_pred)))


In [24]:
class Attention(tf.keras.layers.Layer):
    def __init__(self):
        super(Attention, self).__init__()

    def build(self, input_shape):
        self.W = self.add_weight(name="att_weight", shape=(input_shape[-1], input_shape[-1]), initializer="normal")
        self.b = self.add_weight(name="att_bias", shape=(input_shape[-1],), initializer="zeros")
        super(Attention, self).build(input_shape)

    def call(self, x):
        score = tf.nn.tanh(tf.tensordot(x, self.W, axes=1) + self.b)
        attention_weights = tf.nn.softmax(score, axis=1)
        context_vector = attention_weights * x
        context_vector = tf.reduce_sum(context_vector, axis=1)
        return context_vector

In [25]:
class MultiHeadAttentionLayer(tf.keras.layers.Layer):
    def __init__(self, num_heads, key_dim):
        super(MultiHeadAttentionLayer, self).__init__()
        self.attention = tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=key_dim)

    def call(self, query, value):
        return self.attention(query=query, value=value, key=value)

In [26]:
early_stopping = EarlyStopping(
    monitor='val_loss',  # 모니터할 지표 (예: 'val_loss' 또는 'val_rmse')
    patience=5,          # 성능 향상이 없는 epoch 수 (5번 동안 개선 없으면 중단)
    restore_best_weights=True,  # 가장 성능이 좋은 가중치를 복원
    verbose=1            # 로그 출력 여부
)

## LSTM

### base

w/ naive data

#### Same

In [ ]:
wandb.init(project="Stock Price LSTM - Split, Same", name="initial", config={
    "epochs": 30,
    "batch_size": 32,
    "time_steps": 30,  # LSTM이 한번에 처리하는 시계열 길이
    "lstm_units": [64, 64],
    "optimizer": "adam",
    "loss": "rmse",
})

model_checkpoint = ModelCheckpoint(
    '/content/drive/MyDrive/Time Series Project/Stock Price/Model/model_path/Initial.keras',  # 모델 저장 경로
    monitor='val_loss',  # 모니터할 지표
    save_best_only=True,  # 가장 좋은 모델만 저장
    mode='min',  # `val_loss`가 최소화될 때 저장
    verbose=1  # 저장 시 로그 출력 여부
)

time_steps = wandb.config.time_steps  # Use W&B config
X_train, y_train = create_sequences(x_train_same_df, y_train_same_df, time_steps)
X_val, y_val = create_sequences(x_val_same_df, y_val_same_df, time_steps)

model_same_init = tf.keras.Sequential([
    tf.keras.layers.LSTM(wandb.config.lstm_units[0], activation='tanh', input_shape=(time_steps, len(features)), return_sequences=True),
    tf.keras.layers.LSTM(wandb.config.lstm_units[1], activation='tanh'),
    tf.keras.layers.Dense(1)
])

# 모델 컴파일: 사용자 정의 RMSE 메트릭 사용
model_same_init.compile(optimizer=wandb.config.optimizer, loss=rmse_metric, metrics=[rmse_metric])

# Epoch-by-epoch 학습과 W&B 로깅
for epoch in range(wandb.config.epochs):
    print(f"Epoch {epoch + 1}/{wandb.config.epochs}")
    history = model_same_init.fit(
        X_train, y_train,
        epochs=1,  # 한 번에 1 epoch만 학습
        batch_size=wandb.config.batch_size,
        validation_data=(X_val, y_val),
        verbose=1,
    )
    # W&B에 로그 기록
    wandb.log({
        "epoch": epoch + 1,  # 로그에 1부터 시작하는 epoch 번호 기록
        "train_loss": history.history['loss'][0],
        "val_loss": history.history['val_loss'][0],
        "train_rmse": history.history['rmse_metric'][0],
        "val_rmse": history.history['val_rmse_metric'][0],
    })

model_same_init.summary()

wandb: Currently logged in as: yebin. Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/30
110/110 ━━━━━━━━━━━━━━━━━━━━ 6s 20ms/step - loss: 0.1133 - rmse_metric: 0.1133 - val_loss: 0.2198 - val_rmse_metric: 0.2207
Epoch 2/30
110/110 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.0328 - rmse_metric: 0.0328 - val_loss: 0.1703 - val_rmse_metric: 0.1712
Epoch 3/30
110/110 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.0249 - rmse_metric: 0.0249 - val_loss: 0.1808 - val_rmse_metric: 0.1816
Epoch 4/30
110/110 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.0239 - rmse_metric: 0.0239 - val_loss: 0.1667 - val_rmse_metric: 0.1675
Epoch 5/30
110/110 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step - loss: 0.0243 - rmse_metric: 0.0243 - val_loss: 0.1579 - val_rmse_metric: 0.1587
Epoch 6/30
110/110 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - loss: 0.0261 - rmse_metric: 0.0261 - val_loss: 0.1994 - val_rmse_metric: 0.2003
Epoch 7/30
110/110 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.0188 - rmse_metric: 0.0188 - val_loss: 0.2128 - val_rmse_metric: 0.2135
Epoch 8/30
110/110 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss:

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                          │ (None, 30, 64)              │          25,600 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 64)                  │          33,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 176,069 (687.77 KB)

 Trainable params: 58,689 (229.25 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 117,380 (458.52 KB)

#### Interpolation

In [ ]:
wandb.init(project="Stock Price LSTM - Split, Interpolation", name="initial", config={
    "epochs": 30,
    "batch_size": 32,
    "time_steps": 30,  # LSTM이 한번에 처리하는 시계열 길이
    "lstm_units": [64, 32],
    "optimizer": "adam",
    "loss": "rmse",
},reinit=True)

model_checkpoint = ModelCheckpoint(
    '/content/drive/MyDrive/Time Series Project/Stock Price/Model/model_path/Initial.keras',  # 모델 저장 경로
    monitor='val_loss',  # 모니터할 지표
    save_best_only=True,  # 가장 좋은 모델만 저장
    mode='min',  # `val_loss`가 최소화될 때 저장
    verbose=1  # 저장 시 로그 출력 여부
)

time_steps = wandb.config.time_steps  # Use W&B config
X_train, y_train = create_sequences(x_train_int_df, y_train_int_df, time_steps)
X_val, y_val = create_sequences(x_val_int_df, y_val_int_df, time_steps)

model_interp_init = tf.keras.Sequential([
    tf.keras.layers.LSTM(wandb.config.lstm_units[0], activation='tanh', input_shape=(time_steps, len(features)), return_sequences=True),
    tf.keras.layers.LSTM(wandb.config.lstm_units[1], activation='tanh'),
    tf.keras.layers.Dense(1)
])

# 모델 컴파일: 사용자 정의 RMSE 메트릭 사용
model_interp_init.compile(optimizer=wandb.config.optimizer, loss=rmse_metric, metrics=[rmse_metric])

# Epoch-by-epoch 학습과 W&B 로깅
for epoch in range(wandb.config.epochs):
    print(f"Epoch {epoch + 1}/{wandb.config.epochs}")
    history = model_interp_init.fit(
        X_train, y_train,
        epochs=1,  # 한 번에 1 epoch만 학습
        batch_size=wandb.config.batch_size,
        validation_data=(X_val, y_val),
        verbose=1,
    )
    # W&B에 로그 기록
    wandb.log({
        "epoch": epoch + 1,  # 로그에 1부터 시작하는 epoch 번호 기록
        "train_loss": history.history['loss'][0],
        "val_loss": history.history['val_loss'][0],
        "train_rmse": history.history['rmse_metric'][0],
        "val_rmse": history.history['val_rmse_metric'][0],
    })

model_interp_init.summary()

epoch,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
train_loss,█▃▃▃▃▂▂▂▂▂▂▂▂▁▂▁▂▁▁▁▁▁▁▁▂▁▁▁▁▁
train_rmse,█▄▃▃▃▂▂▂▂▂▂▂▂▁▂▁▂▁▁▁▁▁▁▁▂▁▁▁▁▁
val_loss,▇▂▃▂▁▅▆▆▃▃█▄▃▇▃▃▅▄▇▅▆▄▄▅▅▇▄▃▄▆
val_rmse,▇▂▃▂▁▅▆▆▃▃█▄▃▇▃▃▅▄▇▅▆▄▄▅▅▇▄▃▄▆
epoch,30
train_loss,0.01062
train_rmse,0.01075
val_loss,0.20827
val_rmse,0.20922


Epoch 1/30
110/110 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - loss: 0.1023 - rmse_metric: 0.1023 - val_loss: 0.2374 - val_rmse_metric: 0.2386
Epoch 2/30
110/110 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0334 - rmse_metric: 0.0334 - val_loss: 0.1883 - val_rmse_metric: 0.1893
Epoch 3/30
110/110 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0276 - rmse_metric: 0.0276 - val_loss: 0.1933 - val_rmse_metric: 0.1945
Epoch 4/30
110/110 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0213 - rmse_metric: 0.0213 - val_loss: 0.2169 - val_rmse_metric: 0.2182
Epoch 5/30
110/110 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.0222 - rmse_metric: 0.0222 - val_loss: 0.1877 - val_rmse_metric: 0.1889
Epoch 6/30
110/110 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.0226 - rmse_metric: 0.0226 - val_loss: 0.2216 - val_rmse_metric: 0.2227
Epoch 7/30
110/110 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.0214 - rmse_metric: 0.0214 - val_loss: 0.1821 - val_rmse_metric: 0.1832
Epoch 8/30
110/110 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss:

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_2 (LSTM)                        │ (None, 30, 64)              │          25,600 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_3 (LSTM)                        │ (None, 32)                  │          12,416 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 114,149 (445.90 KB)

 Trainable params: 38,049 (148.63 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 76,100 (297.27 KB)

## LSTM Hyperparameter tuning - Same

- layer 더 쌓기
- hyp 조작
- windowing
- same, int 다 하기

In [ ]:
X_train.shape

(3489, 30, 35)

##### 1

In [ ]:
wandb.init(project="Stock Price LSTM - Split, Same", name="hyper1", config={
    "epochs": 50,
    "batch_size": 32,
    "time_steps": 30,  # LSTM이 한번에 처리하는 시계열 길이
    "lstm_units": [64,64,64],
    "optimizer": "adam",
    "loss": "rmse",
},reinit=True)

epoch,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
train_loss,█▄▃▃▃▃▂▂▂▂▂▂▂▂▁▁▂▂▁▁▁▁▁▁▁▁▁▁▁▁
train_rmse,█▄▃▃▃▃▂▂▂▂▂▂▂▂▁▁▂▂▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▂▂▅▂▆▁▃▃▂▃▂▂▅▂▂▇▇▅▄▇▃▇▆▆▇▆▄▅▆
val_rmse,█▂▂▅▂▆▁▃▃▂▃▂▂▅▂▂▇▇▅▄▇▃▇▆▆▇▆▄▅▆
epoch,30
train_loss,0.00992
train_rmse,0.00997
val_loss,0.21863
val_rmse,0.22044


In [ ]:
model_checkpoint = ModelCheckpoint(
    '/content/drive/MyDrive/Time Series Project/Stock Price/Model/model_path/hyper1.keras',  # 모델 저장 경로
    monitor='val_loss',  # 모니터할 지표
    save_best_only=True,  # 가장 좋은 모델만 저장
    mode='min',  # `val_loss`가 최소화될 때 저장
    verbose=1  # 저장 시 로그 출력 여부
)

time_steps = wandb.config.time_steps  # Use W&B config
X_train, y_train = create_sequences(x_train_same_df, y_train_same_df, time_steps)
X_val, y_val = create_sequences(x_val_same_df, y_val_same_df, time_steps)

model_same_hyper1 = tf.keras.Sequential([
    tf.keras.layers.LSTM(wandb.config.lstm_units[0], activation='tanh', input_shape=(time_steps, len(features)), return_sequences=True),
    tf.keras.layers.LSTM(wandb.config.lstm_units[1], activation='tanh', return_sequences=True),
    tf.keras.layers.LSTM(wandb.config.lstm_units[2], activation='tanh'),
    tf.keras.layers.Dense(1)
])

# 모델 컴파일: 사용자 정의 RMSE 메트릭 사용
model_same_hyper1.compile(optimizer=wandb.config.optimizer, loss=rmse_metric, metrics=[rmse_metric])

# Epoch-by-epoch 학습과 W&B 로깅
for epoch in range(wandb.config.epochs):
    print(f"Epoch {epoch + 1}/{wandb.config.epochs}")
    history = model_same_hyper1.fit(
        X_train, y_train,
        epochs=1,  # 한 번에 1 epoch만 학습
        batch_size=wandb.config.batch_size,
        validation_data=(X_val, y_val),
        verbose=1,
    )
    # W&B에 로그 기록
    wandb.log({
        "epoch": epoch + 1,  # 로그에 1부터 시작하는 epoch 번호 기록
        "train_loss": history.history['loss'][0],
        "val_loss": history.history['val_loss'][0],
        "train_rmse": history.history['rmse_metric'][0],
        "val_rmse": history.history['val_rmse_metric'][0],
    })

model_same_hyper1.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50
110/110 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - loss: 0.0850 - rmse_metric: 0.0850 - val_loss: 0.2343 - val_rmse_metric: 0.2353
Epoch 2/50
110/110 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0339 - rmse_metric: 0.0339 - val_loss: 0.2254 - val_rmse_metric: 0.2264
Epoch 3/50
110/110 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0250 - rmse_metric: 0.0250 - val_loss: 0.2105 - val_rmse_metric: 0.2116
Epoch 4/50
110/110 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0239 - rmse_metric: 0.0239 - val_loss: 0.2034 - val_rmse_metric: 0.2043
Epoch 5/50
110/110 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0219 - rmse_metric: 0.0219 - val_loss: 0.2016 - val_rmse_metric: 0.2026
Epoch 6/50
110/110 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.0216 - rmse_metric: 0.0216 - val_loss: 0.2085 - val_rmse_metric: 0.2096
Epoch 7/50
110/110 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - loss: 0.0218 - rmse_metric: 0.0218 - val_loss: 0.1840 - val_rmse_metric: 0.1850
Epoch 8/50
110/110 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss:

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_7 (LSTM)                        │ (None, 30, 64)              │          25,600 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_8 (LSTM)                        │ (None, 30, 64)              │          33,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_9 (LSTM)                        │ (None, 64)                  │          33,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 275,141 (1.05 MB)

 Trainable params: 91,713 (358.25 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 183,428 (716.52 KB)

##### 2

In [ ]:
wandb.init(project="Stock Price LSTM - Split, Same", name="hyper2", config={
    "epochs": 50,
    "batch_size": 64,
    "time_steps": 30,  # LSTM이 한번에 처리하는 시계열 길이
    "lstm_units": [128,128,128],
    "optimizer": "adam",
    "loss": "rmse",
}, reinit = True)

epoch,▁▁▁▁▂▂▂▃▃▄▄▄▄▅▅▆▆▆▇█▁▁▁▁▂▂▃▃▃▃▄▄▅▅▅▆▆▆▇█
train_loss,▇▅▆▅▆▄▃▄▃▂▃▂▂▂▂▂▁▂▂▁█▇▆▄▄▄▄▃▄▂▂▃▃▂▂▁▁▁▁▁
train_rmse,▆▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁█▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,▆▃▆▄▇██▇▇▇▆▆▇▆▆▅▆▅█▇█▆▅▅▃▃▂▄▂▂▃▂▁▂▂▄▂▃▂▁
val_rmse,▆▅▄▅▇▄▇██▇▆▇▇▅▇▇▅▇▇▇▇▄▅▇▄▂▁▃▂▄▂▃▂▂▂▂▁▂▃▃
epoch,50
train_loss,0.00829
train_rmse,0.00831
val_loss,0.1632
val_rmse,0.16399


In [ ]:
model_checkpoint = ModelCheckpoint(
    '/content/drive/MyDrive/Time Series Project/Stock Price/Model/model_path/hyper2.keras',  # 모델 저장 경로
    monitor='val_loss',  # 모니터할 지표
    save_best_only=True,  # 가장 좋은 모델만 저장
    mode='min',  # `val_loss`가 최소화될 때 저장
    verbose=1  # 저장 시 로그 출력 여부
)

time_steps = wandb.config.time_steps  # Use W&B config
X_train, y_train = create_sequences(x_train_same_df, y_train_same_df, time_steps)
X_val, y_val = create_sequences(x_val_same_df, y_val_same_df, time_steps)

model_same_hyper2 = tf.keras.Sequential([
    tf.keras.layers.LSTM(wandb.config.lstm_units[0], activation='tanh', input_shape=(time_steps, len(features)), return_sequences=True),
    tf.keras.layers.LSTM(wandb.config.lstm_units[1], activation='tanh', return_sequences=True),
    tf.keras.layers.LSTM(wandb.config.lstm_units[2], activation='tanh'),
    tf.keras.layers.Dense(1)
])

# 모델 컴파일: 사용자 정의 RMSE 메트릭 사용
model_same_hyper2.compile(optimizer=wandb.config.optimizer, loss=rmse_metric, metrics=[rmse_metric])

# Epoch-by-epoch 학습과 W&B 로깅
for epoch in range(wandb.config.epochs):
    print(f"Epoch {epoch + 1}/{wandb.config.epochs}")
    history = model_same_hyper2.fit(
        X_train, y_train,
        epochs=1,  # 한 번에 1 epoch만 학습
        batch_size=wandb.config.batch_size,
        validation_data=(X_val, y_val),
        verbose=1,
    )
    # W&B에 로그 기록
    wandb.log({
        "epoch": epoch + 1,  # 로그에 1부터 시작하는 epoch 번호 기록
        "train_loss": history.history['loss'][0],
        "val_loss": history.history['val_loss'][0],
        "train_rmse": history.history['rmse_metric'][0],
        "val_rmse": history.history['val_rmse_metric'][0],
    })

model_same_hyper2.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 3s 16ms/step - loss: 0.1319 - rmse_metric: 0.1319 - val_loss: 0.2000 - val_rmse_metric: 0.2008
Epoch 2/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0285 - rmse_metric: 0.0285 - val_loss: 0.1811 - val_rmse_metric: 0.1818
Epoch 3/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0257 - rmse_metric: 0.0257 - val_loss: 0.1789 - val_rmse_metric: 0.1796
Epoch 4/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0233 - rmse_metric: 0.0233 - val_loss: 0.1667 - val_rmse_metric: 0.1674
Epoch 5/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0258 - rmse_metric: 0.0258 - val_loss: 0.1598 - val_rmse_metric: 0.1605
Epoch 6/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0234 - rmse_metric: 0.0234 - val_loss: 0.1750 - val_rmse_metric: 0.1757
Epoch 7/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0193 - rmse_metric: 0.0193 - val_loss: 0.1612 - val_rmse_metric: 0.1618
Epoch 8/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0168 - r

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_16 (LSTM)                       │ (None, 30, 128)             │          83,968 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_17 (LSTM)                       │ (None, 30, 128)             │         131,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_18 (LSTM)                       │ (None, 128)                 │         131,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,041,797 (3.97 MB)

 Trainable params: 347,265 (1.32 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 694,532 (2.65 MB)

##### 3

In [ ]:
wandb.init(project="Stock Price LSTM - Split, Same", name="hyper3", config={
    "epochs": 50,
    "batch_size": 64,
    "time_steps": 30,  # LSTM이 한번에 처리하는 시계열 길이
    "lstm_units": [64,64,64],
    "optimizer": "adam",
    "loss": "rmse",
},reinit=True)

epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
train_loss,█▃▃▃▃▂▂▂▂▂▂▂▁▁▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_rmse,█▃▃▃▃▂▂▂▂▂▂▂▁▁▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▅▅▃▂▂█▃▃▁▄▁▁▂▄▂▁▂▄▄▄▅▃▃▃▄▄▅▅▄▆▃▃▃▄▅▄▅▃▄
val_rmse,█▅▅▃▂▂█▃▃▁▄▁▁▂▄▂▁▂▄▄▃▄▃▃▄▄▄▅▅▄▆▃▃▄▃▄▅▃▄▄
epoch,50
train_loss,0.0099
train_rmse,0.0099
val_loss,0.17181
val_rmse,0.17259


In [ ]:
model_checkpoint = ModelCheckpoint(
    '/content/drive/MyDrive/Time Series Project/Stock Price/Model/model_path/hyper3.keras',  # 모델 저장 경로
    monitor='val_loss',  # 모니터할 지표
    save_best_only=True,  # 가장 좋은 모델만 저장
    mode='min',  # `val_loss`가 최소화될 때 저장
    verbose=1  # 저장 시 로그 출력 여부
)

time_steps = wandb.config.time_steps  # Use W&B config
X_train, y_train = create_sequences(x_train_same_df, y_train_same_df, time_steps)
X_val, y_val = create_sequences(x_val_same_df, y_val_same_df, time_steps)

model_same_hyper3 = tf.keras.Sequential([
    tf.keras.layers.LSTM(wandb.config.lstm_units[0], activation='tanh', input_shape=(time_steps, len(features)), return_sequences=True),
    tf.keras.layers.LSTM(wandb.config.lstm_units[1], activation='tanh', return_sequences=True),
    tf.keras.layers.LSTM(wandb.config.lstm_units[2], activation='tanh'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1)
])

# 모델 컴파일: 사용자 정의 RMSE 메트릭 사용
model_same_hyper3.compile(optimizer=wandb.config.optimizer, loss=rmse_metric, metrics=[rmse_metric])

# Epoch-by-epoch 학습과 W&B 로깅
for epoch in range(wandb.config.epochs):
    print(f"Epoch {epoch + 1}/{wandb.config.epochs}")
    history = model_same_hyper3.fit(
        X_train, y_train,
        epochs=1,  # 한 번에 1 epoch만 학습
        batch_size=wandb.config.batch_size,
        validation_data=(X_val, y_val),
        verbose=1,
    )
    # W&B에 로그 기록
    wandb.log({
        "epoch": epoch + 1,  # 로그에 1부터 시작하는 epoch 번호 기록
        "train_loss": history.history['loss'][0],
        "val_loss": history.history['val_loss'][0],
        "train_rmse": history.history['rmse_metric'][0],
        "val_rmse": history.history['val_rmse_metric'][0],
    })

model_same_hyper3.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - loss: 0.1524 - rmse_metric: 0.1524 - val_loss: 0.1889 - val_rmse_metric: 0.1899
Epoch 2/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0282 - rmse_metric: 0.0282 - val_loss: 0.1858 - val_rmse_metric: 0.1868
Epoch 3/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0242 - rmse_metric: 0.0242 - val_loss: 0.1840 - val_rmse_metric: 0.1850
Epoch 4/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0223 - rmse_metric: 0.0223 - val_loss: 0.1917 - val_rmse_metric: 0.1927
Epoch 5/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0193 - rmse_metric: 0.0193 - val_loss: 0.1978 - val_rmse_metric: 0.1988
Epoch 6/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0173 - rmse_metric: 0.0173 - val_loss: 0.1961 - val_rmse_metric: 0.1972
Epoch 7/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0166 - rmse_metric: 0.0166 - val_loss: 0.1831 - val_rmse_metric: 0.1840
Epoch 8/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0214 - r

Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_19 (LSTM)                       │ (None, 30, 64)              │          25,600 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_20 (LSTM)                       │ (None, 30, 64)              │          33,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_21 (LSTM)                       │ (None, 64)                  │          33,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 128)                 │           8,320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 300,293 (1.15 MB)

 Trainable params: 100,097 (391.00 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 200,196 (782.02 KB)

##### 4

In [ ]:
wandb.init(project="Stock Price LSTM - Split, Same", name="hyper4", config={
    "epochs": 50,
    "batch_size": 64,
    "time_steps": 30,  # LSTM이 한번에 처리하는 시계열 길이
    "lstm_units": [128,128,128],
    "optimizer": "adam",
    "loss": "rmse",
},reinit=True)

epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
train_loss,█▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_rmse,█▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,▂▁▁▂▃▁▂▅▂▅▄▃▂▆▅▄▇▅▅▅▅▇▅▄█▄▅▄▄▆▅▄▆▆▄▄▅▃▅▅
val_rmse,▂▁▁▂▃▁▂▅▂▅▄▃▂▆▅▄▇▅▅▅▅▇▅▄█▄▅▄▅▄▅▄▆▆▄▄▅▃▅▅
epoch,50
train_loss,0.0097
train_rmse,0.00969
val_loss,0.20581
val_rmse,0.2069


In [ ]:
model_checkpoint = ModelCheckpoint(
    '/content/drive/MyDrive/Time Series Project/Stock Price/Model/model_path/hyper4.keras',  # 모델 저장 경로
    monitor='val_loss',  # 모니터할 지표
    save_best_only=True,  # 가장 좋은 모델만 저장
    mode='min',  # `val_loss`가 최소화될 때 저장
    verbose=1  # 저장 시 로그 출력 여부
)

time_steps = wandb.config.time_steps  # Use W&B config
X_train, y_train = create_sequences(x_train_same_df, y_train_same_df, time_steps)
X_val, y_val = create_sequences(x_val_same_df, y_val_same_df, time_steps)

model_same_hyper4 = tf.keras.Sequential([
    tf.keras.layers.LSTM(wandb.config.lstm_units[0], activation='tanh', input_shape=(time_steps, len(features)), return_sequences=True),
    tf.keras.layers.LSTM(wandb.config.lstm_units[1], activation='tanh', return_sequences=True),
    tf.keras.layers.LSTM(wandb.config.lstm_units[2], activation='tanh'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1)
])

# 모델 컴파일: 사용자 정의 RMSE 메트릭 사용
model_same_hyper4.compile(optimizer=wandb.config.optimizer, loss=rmse_metric, metrics=[rmse_metric])

# Epoch-by-epoch 학습과 W&B 로깅
for epoch in range(wandb.config.epochs):
    print(f"Epoch {epoch + 1}/{wandb.config.epochs}")
    history = model_same_hyper4.fit(
        X_train, y_train,
        epochs=1,  # 한 번에 1 epoch만 학습
        batch_size=wandb.config.batch_size,
        validation_data=(X_val, y_val),
        verbose=1,
    )
    # W&B에 로그 기록
    wandb.log({
        "epoch": epoch + 1,  # 로그에 1부터 시작하는 epoch 번호 기록
        "train_loss": history.history['loss'][0],
        "val_loss": history.history['val_loss'][0],
        "train_rmse": history.history['rmse_metric'][0],
        "val_rmse": history.history['val_rmse_metric'][0],
    })

model_same_hyper4.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - loss: 0.1109 - rmse_metric: 0.1108 - val_loss: 0.1704 - val_rmse_metric: 0.1709
Epoch 2/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0315 - rmse_metric: 0.0315 - val_loss: 0.1613 - val_rmse_metric: 0.1618
Epoch 3/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0217 - rmse_metric: 0.0217 - val_loss: 0.1380 - val_rmse_metric: 0.1385
Epoch 4/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 0.0222 - rmse_metric: 0.0222 - val_loss: 0.1678 - val_rmse_metric: 0.1683
Epoch 5/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 0.0229 - rmse_metric: 0.0229 - val_loss: 0.1744 - val_rmse_metric: 0.1751
Epoch 6/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 0.0198 - rmse_metric: 0.0198 - val_loss: 0.1908 - val_rmse_metric: 0.1915
Epoch 7/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.0200 - rmse_metric: 0.0200 - val_loss: 0.1576 - val_rmse_metric: 0.1582
Epoch 8/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 0.0179 - r

Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_22 (LSTM)                       │ (None, 30, 128)             │          83,968 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_23 (LSTM)                       │ (None, 30, 128)             │         131,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_24 (LSTM)                       │ (None, 128)                 │         131,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ (None, 128)                 │          16,512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_10 (Dense)                     │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,091,333 (4.16 MB)

 Trainable params: 363,777 (1.39 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 727,556 (2.78 MB)

##### 5

In [ ]:
wandb.init(project="Stock Price LSTM - Split, Same", name="hyper5", config={
    "epochs": 50,
    "batch_size": 64,
    "time_steps": 30,  # LSTM이 한번에 처리하는 시계열 길이
    "lstm_units": [128,64,32],
    "optimizer": "adam",
    "loss": "rmse",
},reinit=True)

epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
train_loss,█▆▅▅▅▅▃▄▄▃▃▂▃▂▃▃▂▂▂▂▂▂▂▂▂▁▂▂▂▂▂▁▂▁▁▁▁▂▁▁
train_rmse,█▄▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁
val_loss,▇▅▁▇█▅▆▄▇▃▆▆█▅▅▇▅▅▆█▄▄▄▄▃▂▂▃▆▄▅▄▄▅▅▃▄▂▄▂
val_rmse,▇▅▁▇█▅▆▄▇▃▆▆█▅▅▇▅▅▆█▄▄▃▄▄▄▅▂▂▆▅▄▄▅▅▃▄▂▄▂
epoch,50
train_loss,0.01024
train_rmse,0.01022
val_loss,0.14463
val_rmse,0.14522


In [ ]:
model_checkpoint = ModelCheckpoint(
    '/content/drive/MyDrive/Time Series Project/Stock Price/Model/model_path/hyper5.keras',  # 모델 저장 경로
    monitor='val_loss',  # 모니터할 지표
    save_best_only=True,  # 가장 좋은 모델만 저장
    mode='min',  # `val_loss`가 최소화될 때 저장
    verbose=1  # 저장 시 로그 출력 여부
)

time_steps = wandb.config.time_steps  # Use W&B config
X_train, y_train = create_sequences(x_train_same_df, y_train_same_df, time_steps)
X_val, y_val = create_sequences(x_val_same_df, y_val_same_df, time_steps)

model_same_hyper5 = tf.keras.Sequential([
    tf.keras.layers.LSTM(wandb.config.lstm_units[0], activation='tanh', input_shape=(time_steps, len(features)), return_sequences=True),
    tf.keras.layers.LSTM(wandb.config.lstm_units[1], activation='tanh', return_sequences=True),
    tf.keras.layers.LSTM(wandb.config.lstm_units[2], activation='tanh'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(1)
])

# 모델 컴파일: 사용자 정의 RMSE 메트릭 사용
model_same_hyper5.compile(optimizer=wandb.config.optimizer, loss=rmse_metric, metrics=[rmse_metric])

# Epoch-by-epoch 학습과 W&B 로깅
for epoch in range(wandb.config.epochs):
    print(f"Epoch {epoch + 1}/{wandb.config.epochs}")
    history = model_same_hyper5.fit(
        X_train, y_train,
        epochs=1,  # 한 번에 1 epoch만 학습
        batch_size=wandb.config.batch_size,
        validation_data=(X_val, y_val),
        verbose=1,
    )
    # W&B에 로그 기록
    wandb.log({
        "epoch": epoch + 1,  # 로그에 1부터 시작하는 epoch 번호 기록
        "train_loss": history.history['loss'][0],
        "val_loss": history.history['val_loss'][0],
        "train_rmse": history.history['rmse_metric'][0],
        "val_rmse": history.history['val_rmse_metric'][0],
    })

model_same_hyper5.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - loss: 0.0899 - rmse_metric: 0.0899 - val_loss: 0.1864 - val_rmse_metric: 0.1872
Epoch 2/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.0256 - rmse_metric: 0.0256 - val_loss: 0.2257 - val_rmse_metric: 0.2266
Epoch 3/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 0.0300 - rmse_metric: 0.0300 - val_loss: 0.1897 - val_rmse_metric: 0.1906
Epoch 4/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 0.0249 - rmse_metric: 0.0249 - val_loss: 0.2005 - val_rmse_metric: 0.2014
Epoch 5/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0219 - rmse_metric: 0.0219 - val_loss: 0.1975 - val_rmse_metric: 0.1984
Epoch 6/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0201 - rmse_metric: 0.0201 - val_loss: 0.1805 - val_rmse_metric: 0.1813
Epoch 7/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 0.0175 - rmse_metric: 0.0175 - val_loss: 0.1908 - val_rmse_metric: 0.1917
Epoch 8/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0165 - r

Model: "sequential_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_25 (LSTM)                       │ (None, 30, 128)             │          83,968 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_26 (LSTM)                       │ (None, 30, 64)              │          49,408 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_27 (LSTM)                       │ (None, 32)                  │          12,416 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_2 (Flatten)                  │ (None, 32)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_11 (Dense)                     │ (None, 32)                  │           1,056 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_12 (Dense)                     │ (None, 16)                  │             528 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_13 (Dense)                     │ (None, 1)                   │              17 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 442,181 (1.69 MB)

 Trainable params: 147,393 (575.75 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 294,788 (1.12 MB)

##### 6

In [ ]:
wandb.init(project="Stock Price LSTM - Split, Same", name="hyper6", config={
    "epochs": 50,
    "batch_size": 64,
    "time_steps": 30,  # LSTM이 한번에 처리하는 시계열 길이
    "lstm_units": [128,64,64],
    "optimizer": "adam",
    "loss": "rmse",
},reinit=True)

epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
train_loss,█▄▄▃▃▃▂▂▂▂▂▂▂▂▂▂▁▁▂▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_rmse,█▄▄▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▂▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,▁█▂▄▃▂▃▃▄▅▅▅▄▄▆▅▄▆▅▄▅▆▄▃▄▄▄▄▂▅▄▃▃▃▃▁▃▁▁▂
val_rmse,▁█▂▄▃▂▄▃▃▄▅▄▄▆▄▄▆▅▄▅▆▄▃▄▄▄▄▂▅▅▃▅▃▃▃▁▃▁▁▂
epoch,50
train_loss,0.01002
train_rmse,0.01
val_loss,0.18714
val_rmse,0.18791


In [ ]:
model_checkpoint = ModelCheckpoint(
    '/content/drive/MyDrive/Time Series Project/Stock Price/Model/model_path/hyper6.keras',  # 모델 저장 경로
    monitor='val_loss',  # 모니터할 지표
    save_best_only=True,  # 가장 좋은 모델만 저장
    mode='min',  # `val_loss`가 최소화될 때 저장
    verbose=1  # 저장 시 로그 출력 여부
)

time_steps = wandb.config.time_steps  # Use W&B config
X_train, y_train = create_sequences(x_train_same_df, y_train_same_df, time_steps)
X_val, y_val = create_sequences(x_val_same_df, y_val_same_df, time_steps)

model_same_hyper6 = tf.keras.Sequential([
    tf.keras.layers.LSTM(wandb.config.lstm_units[0], activation='tanh', input_shape=(time_steps, len(features)), return_sequences=True),
    tf.keras.layers.LSTM(wandb.config.lstm_units[1], activation='tanh', return_sequences=True),
    tf.keras.layers.LSTM(wandb.config.lstm_units[2], activation='tanh'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1)
])

# 모델 컴파일: 사용자 정의 RMSE 메트릭 사용
model_same_hyper6.compile(optimizer=wandb.config.optimizer, loss=rmse_metric, metrics=[rmse_metric])

# Epoch-by-epoch 학습과 W&B 로깅
for epoch in range(wandb.config.epochs):
    print(f"Epoch {epoch + 1}/{wandb.config.epochs}")
    history = model_same_hyper6.fit(
        X_train, y_train,
        epochs=1,  # 한 번에 1 epoch만 학습
        batch_size=wandb.config.batch_size,
        validation_data=(X_val, y_val),
        verbose=1,
    )
    # W&B에 로그 기록
    wandb.log({
        "epoch": epoch + 1,  # 로그에 1부터 시작하는 epoch 번호 기록
        "train_loss": history.history['loss'][0],
        "val_loss": history.history['val_loss'][0],
        "train_rmse": history.history['rmse_metric'][0],
        "val_rmse": history.history['val_rmse_metric'][0],
    })

model_same_hyper6.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - loss: 0.1144 - rmse_metric: 0.1144 - val_loss: 0.1626 - val_rmse_metric: 0.1635
Epoch 2/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: 0.0263 - rmse_metric: 0.0263 - val_loss: 0.1936 - val_rmse_metric: 0.1945
Epoch 3/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 0.0248 - rmse_metric: 0.0248 - val_loss: 0.1919 - val_rmse_metric: 0.1928
Epoch 4/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 0.0225 - rmse_metric: 0.0225 - val_loss: 0.1921 - val_rmse_metric: 0.1931
Epoch 5/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0203 - rmse_metric: 0.0203 - val_loss: 0.1634 - val_rmse_metric: 0.1642
Epoch 6/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 0.0206 - rmse_metric: 0.0206 - val_loss: 0.1965 - val_rmse_metric: 0.1976
Epoch 7/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0175 - rmse_metric: 0.0175 - val_loss: 0.2174 - val_rmse_metric: 0.2186
Epoch 8/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0207 - r

Model: "sequential_10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_28 (LSTM)                       │ (None, 30, 128)             │          83,968 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_29 (LSTM)                       │ (None, 30, 64)              │          49,408 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_30 (LSTM)                       │ (None, 64)                  │          33,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_3 (Flatten)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_14 (Dense)                     │ (None, 64)                  │           4,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_15 (Dense)                     │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_16 (Dense)                     │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 518,021 (1.98 MB)

 Trainable params: 172,673 (674.50 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 345,348 (1.32 MB)

##### 7

In [ ]:
wandb.init(project="Stock Price LSTM - Split, Same", name="hyper7", config={
    "epochs": 50,
    "batch_size": 64,
    "time_steps": 30,  # LSTM이 한번에 처리하는 시계열 길이
    "lstm_units": [128,128,128],
    "optimizer": "adam",
    "loss": "rmse",
},reinit=True)

epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
train_loss,█▃▃▂▃▂▂▂▂▂▂▂▂▂▂▁▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_rmse,█▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▂▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,▁▄▄▄▁▇█▅▇▇▇█▇▇▇▇▆▆▆█▇▇▆▆▆▇▆▆▆▆█▇▆▇▇▇▆▆▅▅
val_rmse,▁▄▄▄▁▇█▅▇▇▇█▇█▇▆▇▇▆▇▇▇▇▆▆▇▇▆▆▇▇██▆█▇▆▇▅▅
epoch,50
train_loss,0.01061
train_rmse,0.01059
val_loss,0.19975
val_rmse,0.20088


In [ ]:
model_checkpoint = ModelCheckpoint(
    '/content/drive/MyDrive/Time Series Project/Stock Price/Model/model_path/hyper7.keras',  # 모델 저장 경로
    monitor='val_loss',  # 모니터할 지표
    save_best_only=True,  # 가장 좋은 모델만 저장
    mode='min',  # `val_loss`가 최소화될 때 저장
    verbose=1  # 저장 시 로그 출력 여부
)

time_steps = wandb.config.time_steps  # Use W&B config
X_train, y_train = create_sequences(x_train_same_df, y_train_same_df, time_steps)
X_val, y_val = create_sequences(x_val_same_df, y_val_same_df, time_steps)

model_same_hyper7 = tf.keras.Sequential([
    tf.keras.layers.LSTM(wandb.config.lstm_units[0], activation='tanh', input_shape=(time_steps, len(features)), return_sequences=True),
    tf.keras.layers.LSTM(wandb.config.lstm_units[1], activation='tanh', return_sequences=True),
    tf.keras.layers.LSTM(wandb.config.lstm_units[2], activation='tanh'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
])

# 모델 컴파일: 사용자 정의 RMSE 메트릭 사용
model_same_hyper7.compile(optimizer=wandb.config.optimizer, loss=rmse_metric, metrics=[rmse_metric])

# Epoch-by-epoch 학습과 W&B 로깅
for epoch in range(wandb.config.epochs):
    print(f"Epoch {epoch + 1}/{wandb.config.epochs}")
    history = model_same_hyper7.fit(
        X_train, y_train,
        epochs=1,  # 한 번에 1 epoch만 학습
        batch_size=wandb.config.batch_size,
        validation_data=(X_val, y_val),
        verbose=1,
    )
    # W&B에 로그 기록
    wandb.log({
        "epoch": epoch + 1,  # 로그에 1부터 시작하는 epoch 번호 기록
        "train_loss": history.history['loss'][0],
        "val_loss": history.history['val_loss'][0],
        "train_rmse": history.history['rmse_metric'][0],
        "val_rmse": history.history['val_rmse_metric'][0],
    })

model_same_hyper7.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step - loss: 0.1317 - rmse_metric: 0.1316 - val_loss: 0.2084 - val_rmse_metric: 0.2094
Epoch 2/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 0.0275 - rmse_metric: 0.0275 - val_loss: 0.2042 - val_rmse_metric: 0.2050
Epoch 3/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 0.0294 - rmse_metric: 0.0294 - val_loss: 0.1790 - val_rmse_metric: 0.1798
Epoch 4/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - loss: 0.0225 - rmse_metric: 0.0225 - val_loss: 0.1963 - val_rmse_metric: 0.1972
Epoch 5/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 0.0209 - rmse_metric: 0.0209 - val_loss: 0.1777 - val_rmse_metric: 0.1786
Epoch 6/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 0.0198 - rmse_metric: 0.0198 - val_loss: 0.1822 - val_rmse_metric: 0.1831
Epoch 7/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 0.0219 - rmse_metric: 0.0219 - val_loss: 0.1855 - val_rmse_metric: 0.1864
Epoch 8/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 0.0176 - r

Model: "sequential_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_31 (LSTM)                       │ (None, 30, 128)             │          83,968 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_32 (LSTM)                       │ (None, 30, 128)             │         131,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_33 (LSTM)                       │ (None, 128)                 │         131,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_4 (Flatten)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_17 (Dense)                     │ (None, 128)                 │          16,512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_18 (Dense)                     │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_19 (Dense)                     │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,115,909 (4.26 MB)

 Trainable params: 371,969 (1.42 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 743,940 (2.84 MB)

##### 8

In [ ]:
wandb.init(project="Stock Price LSTM - Split, Same", name="hyper8", config={
    "epochs": 50,
    "batch_size": 64,
    "time_steps": 60,  # LSTM이 한번에 처리하는 시계열 길이
    "lstm_units": [128,128,128],
    "optimizer": "adam",
    "loss": "rmse",
},reinit=True)

epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
train_loss,█▃▃▂▂▃▂▂▂▂▂▂▂▁▁▁▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_rmse,█▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▇▃▆▃▄▂▃▄▄▄▂▅▃▂▆▄▄▃▂▁▃▄▃▄▁▃▃▃▃▃▄▃▂▃▃▃▁▄▃
val_rmse,█▇▃▆▃▄▂▃▄▄▃▄▂▅▃▃▆▄▃▂▁▃▄▃▃▃▁▃▃▃▃▄▃▃▃▃▃▁▄▃
epoch,50
train_loss,0.01048
train_rmse,0.01045
val_loss,0.17409
val_rmse,0.1749


In [ ]:
model_checkpoint = ModelCheckpoint(
    '/content/drive/MyDrive/Time Series Project/Stock Price/Model/model_path/hyper8.keras',  # 모델 저장 경로
    monitor='val_loss',  # 모니터할 지표
    save_best_only=True,  # 가장 좋은 모델만 저장
    mode='min',  # `val_loss`가 최소화될 때 저장
    verbose=1  # 저장 시 로그 출력 여부
)

time_steps = wandb.config.time_steps  # Use W&B config
X_train, y_train = create_sequences(x_train_same_df, y_train_same_df, time_steps)
X_val, y_val = create_sequences(x_val_same_df, y_val_same_df, time_steps)

model_same_hyper8 = tf.keras.Sequential([
    tf.keras.layers.LSTM(wandb.config.lstm_units[0], activation='tanh', input_shape=(time_steps, len(features)), return_sequences=True),
    tf.keras.layers.LSTM(wandb.config.lstm_units[1], activation='tanh', return_sequences=True),
    tf.keras.layers.LSTM(wandb.config.lstm_units[2], activation='tanh'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1)
])

# 모델 컴파일: 사용자 정의 RMSE 메트릭 사용
model_same_hyper8.compile(optimizer=wandb.config.optimizer, loss=rmse_metric, metrics=[rmse_metric])

# Epoch-by-epoch 학습과 W&B 로깅
for epoch in range(wandb.config.epochs):
    print(f"Epoch {epoch + 1}/{wandb.config.epochs}")
    history = model_same_hyper8.fit(
        X_train, y_train,
        epochs=1,  # 한 번에 1 epoch만 학습
        batch_size=wandb.config.batch_size,
        validation_data=(X_val, y_val),
        verbose=1,
    )
    # W&B에 로그 기록
    wandb.log({
        "epoch": epoch + 1,  # 로그에 1부터 시작하는 epoch 번호 기록
        "train_loss": history.history['loss'][0],
        "val_loss": history.history['val_loss'][0],
        "train_rmse": history.history['rmse_metric'][0],
        "val_rmse": history.history['val_rmse_metric'][0],
    })

model_same_hyper8.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 3s 22ms/step - loss: 0.1457 - rmse_metric: 0.1457 - val_loss: 0.2693 - val_rmse_metric: 0.2739
Epoch 2/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.0447 - rmse_metric: 0.0447 - val_loss: 0.2245 - val_rmse_metric: 0.2289
Epoch 3/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.0255 - rmse_metric: 0.0255 - val_loss: 0.2077 - val_rmse_metric: 0.2115
Epoch 4/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 0.0249 - rmse_metric: 0.0249 - val_loss: 0.2360 - val_rmse_metric: 0.2401
Epoch 5/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 0.0222 - rmse_metric: 0.0222 - val_loss: 0.1929 - val_rmse_metric: 0.1962
Epoch 6/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 0.0249 - rmse_metric: 0.0249 - val_loss: 0.2195 - val_rmse_metric: 0.2235
Epoch 7/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 0.0210 - rmse_metric: 0.0210 - val_loss: 0.2222 - val_rmse_metric: 0.2264
Epoch 8/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.0196 - r

Model: "sequential_12"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_34 (LSTM)                       │ (None, 60, 128)             │          83,968 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_35 (LSTM)                       │ (None, 60, 128)             │         131,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_36 (LSTM)                       │ (None, 128)                 │         131,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_5 (Flatten)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_20 (Dense)                     │ (None, 128)                 │          16,512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_21 (Dense)                     │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,091,333 (4.16 MB)

 Trainable params: 363,777 (1.39 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 727,556 (2.78 MB)

##### 9

In [ ]:
wandb.init(project="Stock Price LSTM - Split, Same", name="hyper9", config={
    "epochs": 50,
    "batch_size": 64,
    "time_steps": 30,  # LSTM이 한번에 처리하는 시계열 길이
    "lstm_units": [128,128,64,64],
    "optimizer": "adam",
    "loss": "rmse",
},reinit=True)

epoch,▁▁▁▁▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
train_loss,█▄▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁
train_rmse,█▄▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▄▂▅▁▄▄▅▃▂▂▃▄▃▂▃▄▃▃▃▁▂▄▃▃▂▄▁▂▄▁▄▂▄▃▃▃▄▄▃
val_rmse,█▄▂▅▁▄▄▅▃▃▂▂▃▄▃▃▃▄▃▃▂▂▂▄▃▂▃▄▂▄▁▄▃▄▃▃▃▄▄▃
epoch,50
train_loss,0.01068
train_rmse,0.01067
val_loss,0.21056
val_rmse,0.21506


In [ ]:
model_checkpoint = ModelCheckpoint(
    '/content/drive/MyDrive/Time Series Project/Stock Price/Model/model_path/hyper9.keras',  # 모델 저장 경로
    monitor='val_loss',  # 모니터할 지표
    save_best_only=True,  # 가장 좋은 모델만 저장
    mode='min',  # `val_loss`가 최소화될 때 저장
    verbose=1  # 저장 시 로그 출력 여부
)

time_steps = wandb.config.time_steps  # Use W&B config
X_train, y_train = create_sequences(x_train_same_df, y_train_same_df, time_steps)
X_val, y_val = create_sequences(x_val_same_df, y_val_same_df, time_steps)

model_same_hyper9 = tf.keras.Sequential([
    tf.keras.layers.LSTM(wandb.config.lstm_units[0], activation='tanh', input_shape=(time_steps, len(features)), return_sequences=True),
    tf.keras.layers.LSTM(wandb.config.lstm_units[1], activation='tanh', return_sequences=True),
    tf.keras.layers.LSTM(wandb.config.lstm_units[2], activation='tanh', return_sequences=True),
    tf.keras.layers.LSTM(wandb.config.lstm_units[3], activation='tanh'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1)
])

# 모델 컴파일: 사용자 정의 RMSE 메트릭 사용
model_same_hyper9.compile(optimizer=wandb.config.optimizer, loss=rmse_metric, metrics=[rmse_metric])

# Epoch-by-epoch 학습과 W&B 로깅
for epoch in range(wandb.config.epochs):
    print(f"Epoch {epoch + 1}/{wandb.config.epochs}")
    history = model_same_hyper9.fit(
        X_train, y_train,
        epochs=1,  # 한 번에 1 epoch만 학습
        batch_size=wandb.config.batch_size,
        validation_data=(X_val, y_val),
        verbose=1,
    )
    # W&B에 로그 기록
    wandb.log({
        "epoch": epoch + 1,  # 로그에 1부터 시작하는 epoch 번호 기록
        "train_loss": history.history['loss'][0],
        "val_loss": history.history['val_loss'][0],
        "train_rmse": history.history['rmse_metric'][0],
        "val_rmse": history.history['val_rmse_metric'][0],
    })

model_same_hyper9.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 5s 26ms/step - loss: 0.1312 - rmse_metric: 0.1312 - val_loss: 0.1733 - val_rmse_metric: 0.1741
Epoch 2/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: 0.0281 - rmse_metric: 0.0281 - val_loss: 0.1632 - val_rmse_metric: 0.1639
Epoch 3/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 0.0266 - rmse_metric: 0.0266 - val_loss: 0.1695 - val_rmse_metric: 0.1703
Epoch 4/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 2s 34ms/step - loss: 0.0219 - rmse_metric: 0.0219 - val_loss: 0.1683 - val_rmse_metric: 0.1690
Epoch 5/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - loss: 0.0237 - rmse_metric: 0.0237 - val_loss: 0.1626 - val_rmse_metric: 0.1633
Epoch 6/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: 0.0220 - rmse_metric: 0.0220 - val_loss: 0.1687 - val_rmse_metric: 0.1694
Epoch 7/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.0190 - rmse_metric: 0.0190 - val_loss: 0.1652 - val_rmse_metric: 0.1660
Epoch 8/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 0.0218 - r

Model: "sequential_13"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_40 (LSTM)                       │ (None, 30, 128)             │          83,968 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_41 (LSTM)                       │ (None, 30, 128)             │         131,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_42 (LSTM)                       │ (None, 30, 64)              │          49,408 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_43 (LSTM)                       │ (None, 64)                  │          33,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_6 (Flatten)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_22 (Dense)                     │ (None, 128)                 │           8,320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_23 (Dense)                     │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 919,301 (3.51 MB)

 Trainable params: 306,433 (1.17 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 612,868 (2.34 MB)

##### 10

In [ ]:
wandb.init(project="Stock Price LSTM - Split, Same", name="hyper10", config={
    "epochs": 50,
    "batch_size": 64,
    "time_steps": 30,  # LSTM이 한번에 처리하는 시계열 길이
    "lstm_units": [128,128,64,64],
    "optimizer": "adam",
    "loss": "rmse",
},reinit=True)

In [ ]:
model_checkpoint = ModelCheckpoint(
    '/content/drive/MyDrive/Time Series Project/Stock Price/Model/model_path/hyper10.keras',  # 모델 저장 경로
    monitor='val_loss',  # 모니터할 지표
    save_best_only=True,  # 가장 좋은 모델만 저장
    mode='min',  # `val_loss`가 최소화될 때 저장
    verbose=1  # 저장 시 로그 출력 여부
)

time_steps = wandb.config.time_steps  # Use W&B config
X_train, y_train = create_sequences(x_train_same_df, y_train_same_df, time_steps)
X_val, y_val = create_sequences(x_val_same_df, y_val_same_df, time_steps)

model_same_hyper10 = tf.keras.Sequential([
    tf.keras.layers.LSTM(wandb.config.lstm_units[0], activation='tanh', input_shape=(time_steps, len(features)), return_sequences=True),
    tf.keras.layers.LSTM(wandb.config.lstm_units[1], activation='tanh', return_sequences=True),
    tf.keras.layers.LSTM(wandb.config.lstm_units[2], activation='tanh', return_sequences=True),
    tf.keras.layers.LSTM(wandb.config.lstm_units[3], activation='tanh'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
])

# 모델 컴파일: 사용자 정의 RMSE 메트릭 사용
model_same_hyper10.compile(optimizer=wandb.config.optimizer, loss=rmse_metric, metrics=[rmse_metric])

# Epoch-by-epoch 학습과 W&B 로깅
for epoch in range(wandb.config.epochs):
    print(f"Epoch {epoch + 1}/{wandb.config.epochs}")
    history = model_same_hyper10.fit(
        X_train, y_train,
        epochs=1,  # 한 번에 1 epoch만 학습
        batch_size=wandb.config.batch_size,
        validation_data=(X_val, y_val),
        verbose=1,
    )
    # W&B에 로그 기록
    wandb.log({
        "epoch": epoch + 1,  # 로그에 1부터 시작하는 epoch 번호 기록
        "train_loss": history.history['loss'][0],
        "val_loss": history.history['val_loss'][0],
        "train_rmse": history.history['rmse_metric'][0],
        "val_rmse": history.history['val_rmse_metric'][0],
    })

model_same_hyper10.summary()

##### 11

In [ ]:
wandb.init(project="Stock Price LSTM - Split, Same", name="hyper11", config={
    "epochs": 50,
    "batch_size": 64,
    "time_steps": 60,  # LSTM이 한번에 처리하는 시계열 길이
    "lstm_units": [128,128,128],
    "optimizer": "adam",
    "loss": "rmse",
},reinit=True)

epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
train_loss,█▃▃▂▃▂▂▂▂▂▂▂▂▂▂▂▂▁▂▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_rmse,█▃▃▂▃▂▂▂▂▂▂▂▂▂▂▂▁▂▂▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,▂▁▂▂▁▁▃▄▃▄▃▄▄▃▆▄▄▃▆▅▇▆▅▅▇█▆▆▆▇▇▆▇▆▆▅▆▆▅▅
val_rmse,▂▁▂▁▂▃▄▃▄▃▄▄▃▆▆▃▆▅▅▇▅▅▇▇▇█▆▆▆▇▇▆▇▆▆▅▆▆▅▅
epoch,50
train_loss,0.0094
train_rmse,0.00938
val_loss,0.19934
val_rmse,0.20049


In [ ]:
model_checkpoint = ModelCheckpoint(
    '/content/drive/MyDrive/Time Series Project/Stock Price/Model/model_path/hyper11.keras',  # 모델 저장 경로
    monitor='val_loss',  # 모니터할 지표
    save_best_only=True,  # 가장 좋은 모델만 저장
    mode='min',  # `val_loss`가 최소화될 때 저장
    verbose=1  # 저장 시 로그 출력 여부
)

time_steps = wandb.config.time_steps  # Use W&B config
X_train, y_train = create_sequences(x_train_same_df, y_train_same_df, time_steps)
X_val, y_val = create_sequences(x_val_same_df, y_val_same_df, time_steps)

model_same_hyper11 = tf.keras.Sequential([
    tf.keras.layers.LSTM(wandb.config.lstm_units[0], activation='tanh', input_shape=(time_steps, len(features)), return_sequences=True),
    tf.keras.layers.LSTM(wandb.config.lstm_units[1], activation='tanh', return_sequences=True),
    tf.keras.layers.LSTM(wandb.config.lstm_units[2], activation='tanh'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1)
])

# 모델 컴파일: 사용자 정의 RMSE 메트릭 사용
model_same_hyper11.compile(optimizer=wandb.config.optimizer, loss=rmse_metric, metrics=[rmse_metric])

# Epoch-by-epoch 학습과 W&B 로깅
for epoch in range(wandb.config.epochs):
    print(f"Epoch {epoch + 1}/{wandb.config.epochs}")
    history = model_same_hyper11.fit(
        X_train, y_train,
        epochs=1,  # 한 번에 1 epoch만 학습
        batch_size=wandb.config.batch_size,
        validation_data=(X_val, y_val),
        verbose=1,
    )
    # W&B에 로그 기록
    wandb.log({
        "epoch": epoch + 1,  # 로그에 1부터 시작하는 epoch 번호 기록
        "train_loss": history.history['loss'][0],
        "val_loss": history.history['val_loss'][0],
        "train_rmse": history.history['rmse_metric'][0],
        "val_rmse": history.history['val_rmse_metric'][0],
    })

model_same_hyper11.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 3s 25ms/step - loss: 0.1157 - rmse_metric: 0.1157 - val_loss: 0.2096 - val_rmse_metric: 0.2135
Epoch 2/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - loss: 0.0282 - rmse_metric: 0.0282 - val_loss: 0.1544 - val_rmse_metric: 0.1572
Epoch 3/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 0.0332 - rmse_metric: 0.0332 - val_loss: 0.1610 - val_rmse_metric: 0.1641
Epoch 4/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.0274 - rmse_metric: 0.0274 - val_loss: 0.1715 - val_rmse_metric: 0.1748
Epoch 5/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.0218 - rmse_metric: 0.0218 - val_loss: 0.1797 - val_rmse_metric: 0.1830
Epoch 6/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.0214 - rmse_metric: 0.0214 - val_loss: 0.1921 - val_rmse_metric: 0.1954
Epoch 7/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 0.0197 - rmse_metric: 0.0197 - val_loss: 0.1982 - val_rmse_metric: 0.2014
Epoch 8/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.0189 - r

Model: "sequential_14"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_44 (LSTM)                       │ (None, 60, 128)             │          83,968 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_45 (LSTM)                       │ (None, 60, 128)             │         131,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_46 (LSTM)                       │ (None, 128)                 │         131,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_7 (Flatten)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_24 (Dense)                     │ (None, 128)                 │          16,512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_25 (Dense)                     │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,091,333 (4.16 MB)

 Trainable params: 363,777 (1.39 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 727,556 (2.78 MB)

##### 12

In [ ]:
wandb.init(project="Stock Price LSTM - Split, Same", name="hyper12", config={
    "epochs": 50,
    "batch_size": 64,
    "time_steps": 30,  # LSTM이 한번에 처리하는 시계열 길이
    "lstm_units": [128,128,128],
    "optimizer": "adam",
    "loss": "rmse",
},reinit=True)

epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train_loss,█▃▃▃▂▂▂▂▂▂▂▂▂▂▁▁▂▁▁▁▂▂▂▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁
train_rmse,█▃▃▃▂▂▂▂▂▂▂▂▂▁▂▂▁▂▁▁▁▂▂▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▁▂▃▆▃▄▆▆▂▂▂▄▆▃▃▂▃▅▃▃▂▃▂▁▄▁▂▅▂▂▃▂▂▄▄▄▄▄▅
val_rmse,█▁▂▃▄▆▃▄▆▂▂▂▄▆▃▃▂▃▃▃▄▂▃▂▁▅▁▂▅▂▂▃▂▂▄▄▄▄▄▅
epoch,50
train_loss,0.01009
train_rmse,0.01008
val_loss,0.18256
val_rmse,0.18625


In [ ]:
model_checkpoint = ModelCheckpoint(
    '/content/drive/MyDrive/Time Series Project/Stock Price/Model/model_path/hyper12.keras',  # 모델 저장 경로
    monitor='val_loss',  # 모니터할 지표
    save_best_only=True,  # 가장 좋은 모델만 저장
    mode='min',  # `val_loss`가 최소화될 때 저장
    verbose=1  # 저장 시 로그 출력 여부
)

time_steps = wandb.config.time_steps  # Use W&B config
X_train, y_train = create_sequences(x_train_same_df, y_train_same_df, time_steps)
X_val, y_val = create_sequences(x_val_same_df, y_val_same_df, time_steps)

model_same_hyper12 = tf.keras.Sequential([
    Bidirectional(tf.keras.layers.LSTM(wandb.config.lstm_units[0], activation='tanh', input_shape=(time_steps, len(features)), return_sequences=True)),
    Bidirectional(tf.keras.layers.LSTM(wandb.config.lstm_units[1], activation='tanh', return_sequences=True)),
    tf.keras.layers.LSTM(wandb.config.lstm_units[2], activation='tanh'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1)
])

# 모델 컴파일: 사용자 정의 RMSE 메트릭 사용
model_same_hyper12.compile(optimizer=wandb.config.optimizer, loss=rmse_metric, metrics=[rmse_metric])

# Epoch-by-epoch 학습과 W&B 로깅
for epoch in range(wandb.config.epochs):
    print(f"Epoch {epoch + 1}/{wandb.config.epochs}")
    history = model_same_hyper12.fit(
        X_train, y_train,
        epochs=1,  # 한 번에 1 epoch만 학습
        batch_size=wandb.config.batch_size,
        validation_data=(X_val, y_val),
        verbose=1,
    )
    # W&B에 로그 기록
    wandb.log({
        "epoch": epoch + 1,  # 로그에 1부터 시작하는 epoch 번호 기록
        "train_loss": history.history['loss'][0],
        "val_loss": history.history['val_loss'][0],
        "train_rmse": history.history['rmse_metric'][0],
        "val_rmse": history.history['val_rmse_metric'][0],
    })

model_same_hyper12.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 7s 30ms/step - loss: 0.1319 - rmse_metric: 0.1319 - val_loss: 0.1664 - val_rmse_metric: 0.1670
Epoch 2/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 0.0318 - rmse_metric: 0.0318 - val_loss: 0.1764 - val_rmse_metric: 0.1770
Epoch 3/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 0.0229 - rmse_metric: 0.0229 - val_loss: 0.1787 - val_rmse_metric: 0.1794
Epoch 4/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 0.0217 - rmse_metric: 0.0217 - val_loss: 0.2024 - val_rmse_metric: 0.2032
Epoch 5/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 0.0222 - rmse_metric: 0.0222 - val_loss: 0.1910 - val_rmse_metric: 0.1919
Epoch 6/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 0.0183 - rmse_metric: 0.0183 - val_loss: 0.1865 - val_rmse_metric: 0.1873
Epoch 7/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 0.0173 - rmse_metric: 0.0173 - val_loss: 0.1854 - val_rmse_metric: 0.1863
Epoch 8/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - loss: 0.0173 - r

Model: "sequential_15"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ bidirectional (Bidirectional)        │ (None, 30, 256)             │         167,936 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_1 (Bidirectional)      │ (None, 30, 256)             │         394,240 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_49 (LSTM)                       │ (None, 128)                 │         197,120 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_8 (Flatten)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_26 (Dense)                     │ (None, 128)                 │          16,512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_27 (Dense)                     │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,327,813 (8.88 MB)

 Trainable params: 775,937 (2.96 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 1,551,876 (5.92 MB)

##### 13

In [ ]:
import tensorflow as tf

class Attention(tf.keras.layers.Layer):
    def __init__(self):
        super(Attention, self).__init__()

    def build(self, input_shape):
        self.W = self.add_weight(name="att_weight", shape=(input_shape[-1], input_shape[-1]), initializer="normal")
        self.b = self.add_weight(name="att_bias", shape=(input_shape[-1],), initializer="zeros")
        super(Attention, self).build(input_shape)

    def call(self, x):
        score = tf.nn.tanh(tf.tensordot(x, self.W, axes=1) + self.b)
        attention_weights = tf.nn.softmax(score, axis=1)
        context_vector = attention_weights * x
        context_vector = tf.reduce_sum(context_vector, axis=1)
        return context_vector

In [ ]:
wandb.init(project="Stock Price LSTM - Split, Same", name="hyper13", config={
    "epochs": 50,
    "batch_size": 64,
    "time_steps": 30,  # LSTM이 한번에 처리하는 시계열 길이
    "lstm_units": [128,128,128],
    "optimizer": "adam",
    "loss": "rmse",
},reinit=True)

epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train_loss,█▄▃▂▃▂▂▂▂▂▂▂▂▁▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_rmse,█▄▃▂▂▂▂▂▂▂▂▂▁▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,▂▃▄█▆▅▅█▅▆▄▆█▅▇█▅▇▄▅▅▆▆▃▆▅▂▂▅▅▃▂▄▁▂▁▂▂▁▃
val_rmse,▂▃▄█▆▅█▅▇▆▄▆█▅▇█▅▇▄▅▅▆▆▃▆▅▂▂▃▅▃▂▄▁▂▁▂▂▁▃
epoch,50
train_loss,0.00937
train_rmse,0.00937
val_loss,0.17237
val_rmse,0.17316


In [ ]:
model_checkpoint = ModelCheckpoint(
    '/content/drive/MyDrive/Time Series Project/Stock Price/Model/model_path/hyper13.keras',  # 모델 저장 경로
    monitor='val_loss',  # 모니터할 지표
    save_best_only=True,  # 가장 좋은 모델만 저장
    mode='min',  # `val_loss`가 최소화될 때 저장
    verbose=1  # 저장 시 로그 출력 여부
)

time_steps = wandb.config.time_steps  # Use W&B config
X_train, y_train = create_sequences(x_train_same_df, y_train_same_df, time_steps)
X_val, y_val = create_sequences(x_val_same_df, y_val_same_df, time_steps)

model_same_hyper13 = tf.keras.Sequential([
    Bidirectional(tf.keras.layers.LSTM(wandb.config.lstm_units[0], activation='tanh', input_shape=(time_steps, len(features)), return_sequences=True)),
    Bidirectional(tf.keras.layers.LSTM(wandb.config.lstm_units[1], activation='tanh', return_sequences=True)),
    Bidirectional(tf.keras.layers.LSTM(wandb.config.lstm_units[2], activation='tanh')),
    Attention(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1)
])

# 모델 컴파일: 사용자 정의 RMSE 메트릭 사용
model_same_hyper13.compile(optimizer=wandb.config.optimizer, loss=rmse_metric, metrics=[rmse_metric])

# Epoch-by-epoch 학습과 W&B 로깅
for epoch in range(wandb.config.epochs):
    print(f"Epoch {epoch + 1}/{wandb.config.epochs}")
    history = model_same_hyper13.fit(
        X_train, y_train,
        epochs=1,  # 한 번에 1 epoch만 학습
        batch_size=wandb.config.batch_size,
        validation_data=(X_val, y_val),
        verbose=1,
    )
    # W&B에 로그 기록
    wandb.log({
        "epoch": epoch + 1,  # 로그에 1부터 시작하는 epoch 번호 기록
        "train_loss": history.history['loss'][0],
        "val_loss": history.history['val_loss'][0],
        "train_rmse": history.history['rmse_metric'][0],
        "val_rmse": history.history['val_rmse_metric'][0],
    })

model_same_hyper13.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 7s 30ms/step - loss: 0.2654 - rmse_metric: 0.2654 - val_loss: 0.3013 - val_rmse_metric: 0.3020
Epoch 2/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 0.1485 - rmse_metric: 0.1485 - val_loss: 0.2923 - val_rmse_metric: 0.2937
Epoch 3/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 0.0709 - rmse_metric: 0.0709 - val_loss: 0.2225 - val_rmse_metric: 0.2237
Epoch 4/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 0.0337 - rmse_metric: 0.0337 - val_loss: 0.1508 - val_rmse_metric: 0.1516
Epoch 5/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 0.0286 - rmse_metric: 0.0286 - val_loss: 0.1557 - val_rmse_metric: 0.1564
Epoch 6/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 0.0246 - rmse_metric: 0.0246 - val_loss: 0.1513 - val_rmse_metric: 0.1520
Epoch 7/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - loss: 0.0219 - rmse_metric: 0.0219 - val_loss: 0.1511 - val_rmse_metric: 0.1518
Epoch 8/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - loss: 0.0210 - r

Model: "sequential_16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ bidirectional_2 (Bidirectional)      │ (None, 30, 256)             │         167,936 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_3 (Bidirectional)      │ (None, 30, 256)             │         394,240 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_4 (Bidirectional)      │ (None, 256)                 │         394,240 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ attention (Attention)                │ (None)                      │          65,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_9 (Flatten)                  │ (None, 1)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_28 (Dense)                     │ (None, 128)                 │             256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_29 (Dense)                     │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,067,781 (11.70 MB)

 Trainable params: 1,022,593 (3.90 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,045,188 (7.80 MB)

##### 14

In [ ]:
wandb.init(project="Stock Price LSTM - Split, Same", name="hyper14", config={
    "epochs": 50,
    "batch_size": 64,
    "time_steps": 30,  # LSTM이 한번에 처리하는 시계열 길이
    "lstm_units": [128,128,128],
    "optimizer": "adam",
    "loss": "rmse",
},reinit=True)

epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train_loss,█▅▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_rmse,█▅▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,██▅▂▂▂▂▂▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▂▁▁▁▁▁▁
val_rmse,██▅▂▂▂▂▂▂▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▂▁▁▂▁▁▁▁▂▂▁▁▁▁▂
epoch,50
train_loss,0.01065
train_rmse,0.01064
val_loss,0.14536
val_rmse,0.14605


In [ ]:
model_checkpoint = ModelCheckpoint(
    '/content/drive/MyDrive/Time Series Project/Stock Price/Model/model_path/hyper14.keras',  # 모델 저장 경로
    monitor='val_loss',  # 모니터할 지표
    save_best_only=True,  # 가장 좋은 모델만 저장
    mode='min',  # `val_loss`가 최소화될 때 저장
    verbose=1  # 저장 시 로그 출력 여부
)

time_steps = wandb.config.time_steps  # Use W&B config
X_train, y_train = create_sequences(x_train_same_df, y_train_same_df, time_steps)
X_val, y_val = create_sequences(x_val_same_df, y_val_same_df, time_steps)

model_same_hyper14 = tf.keras.Sequential([
    Bidirectional(tf.keras.layers.LSTM(wandb.config.lstm_units[0], activation='relu', input_shape=(time_steps, len(features)), return_sequences=True)),
    Bidirectional(tf.keras.layers.LSTM(wandb.config.lstm_units[1], activation='relu', return_sequences=True)),
    Bidirectional(tf.keras.layers.LSTM(wandb.config.lstm_units[2], activation='relu')),
    Attention(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1)
])

# 모델 컴파일: 사용자 정의 RMSE 메트릭 사용
model_same_hyper14.compile(optimizer=wandb.config.optimizer, loss=rmse_metric, metrics=[rmse_metric])

# Epoch-by-epoch 학습과 W&B 로깅
for epoch in range(wandb.config.epochs):
    print(f"Epoch {epoch + 1}/{wandb.config.epochs}")
    history = model_same_hyper14.fit(
        X_train, y_train,
        epochs=1,  # 한 번에 1 epoch만 학습
        batch_size=wandb.config.batch_size,
        validation_data=(X_val, y_val),
        verbose=1,
    )
    # W&B에 로그 기록
    wandb.log({
        "epoch": epoch + 1,  # 로그에 1부터 시작하는 epoch 번호 기록
        "train_loss": history.history['loss'][0],
        "val_loss": history.history['val_loss'][0],
        "train_rmse": history.history['rmse_metric'][0],
        "val_rmse": history.history['val_rmse_metric'][0],
    })

model_same_hyper14.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 22s 187ms/step - loss: 0.3860 - rmse_metric: 0.3859 - val_loss: 0.1831 - val_rmse_metric: 0.1836
Epoch 2/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 2s 37ms/step - loss: 0.0450 - rmse_metric: 0.0449 - val_loss: 0.1651 - val_rmse_metric: 0.1656
Epoch 3/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 2s 37ms/step - loss: 0.0260 - rmse_metric: 0.0260 - val_loss: 0.1527 - val_rmse_metric: 0.1532
Epoch 4/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 2s 31ms/step - loss: 0.0232 - rmse_metric: 0.0232 - val_loss: 0.1810 - val_rmse_metric: 0.1816
Epoch 5/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 2s 32ms/step - loss: 0.0204 - rmse_metric: 0.0204 - val_loss: 0.1680 - val_rmse_metric: 0.1686
Epoch 6/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 2s 32ms/step - loss: 0.0266 - rmse_metric: 0.0266 - val_loss: 0.1639 - val_rmse_metric: 0.1645
Epoch 7/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 2s 32ms/step - loss: 0.0213 - rmse_metric: 0.0213 - val_loss: 0.1544 - val_rmse_metric: 0.1549
Epoch 8/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 2s 36ms/step - loss: 0.0197 -

Model: "sequential_17"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ bidirectional_5 (Bidirectional)      │ (None, 30, 256)             │         167,936 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_6 (Bidirectional)      │ (None, 30, 256)             │         394,240 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_7 (Bidirectional)      │ (None, 256)                 │         394,240 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ attention_1 (Attention)              │ (None)                      │          65,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_10 (Flatten)                 │ (None, 1)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_30 (Dense)                     │ (None, 128)                 │             256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_31 (Dense)                     │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,067,781 (11.70 MB)

 Trainable params: 1,022,593 (3.90 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,045,188 (7.80 MB)

##### 15

In [ ]:
wandb.init(project="Stock Price LSTM - Split, Same", name="hyper15", config={
    "epochs": 50,
    "batch_size": 64,
    "time_steps": 30,  # LSTM이 한번에 처리하는 시계열 길이
    "lstm_units": [128,128,128],
    "optimizer": "adam",
    "loss": "rmse",
},reinit=True)

epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
train_loss,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_rmse,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,▅▃▁▅▃▁▃▂▄▃▂▄▂▆▄▅▅▄▅▆▅▆▅▇▇█▆▇▆▆▇▆▇▅▆▆▇▆▇▆
val_rmse,▅▃▁▅▃▂▂▄▂▁▄▂▄▃▄▅▄▄▅▅▆▄▆▇▇▆▆▆▅▇▅▆▅▅▇▆█▅▆▆
epoch,50
train_loss,0.01056
train_rmse,0.01055
val_loss,0.19117
val_rmse,0.19215


In [ ]:
model_checkpoint = ModelCheckpoint(
    '/content/drive/MyDrive/Time Series Project/Stock Price/Model/model_path/hyper15.keras',  # 모델 저장 경로
    monitor='val_loss',  # 모니터할 지표
    save_best_only=True,  # 가장 좋은 모델만 저장
    mode='min',  # `val_loss`가 최소화될 때 저장
    verbose=1  # 저장 시 로그 출력 여부
)

time_steps = wandb.config.time_steps  # Use W&B config
X_train, y_train = create_sequences(x_train_same_df, y_train_same_df, time_steps)
X_val, y_val = create_sequences(x_val_same_df, y_val_same_df, time_steps)

model_same_hyper15 = tf.keras.Sequential([
    Bidirectional(tf.keras.layers.LSTM(wandb.config.lstm_units[0], activation='tanh', input_shape=(time_steps, len(features)), return_sequences=True)),
    Bidirectional(tf.keras.layers.LSTM(wandb.config.lstm_units[1], activation='tanh', return_sequences=True)),
    Bidirectional(tf.keras.layers.LSTM(wandb.config.lstm_units[2], activation='tanh')),
    Attention(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1)
])

# 모델 컴파일: 사용자 정의 RMSE 메트릭 사용
model_same_hyper15.compile(optimizer=wandb.config.optimizer, loss=rmse_metric, metrics=[rmse_metric])

# Epoch-by-epoch 학습과 W&B 로깅
for epoch in range(wandb.config.epochs):
    print(f"Epoch {epoch + 1}/{wandb.config.epochs}")
    history = model_same_hyper15.fit(
        X_train, y_train,
        epochs=1,  # 한 번에 1 epoch만 학습
        batch_size=wandb.config.batch_size,
        validation_data=(X_val, y_val),
        verbose=1,
    )
    # W&B에 로그 기록
    wandb.log({
        "epoch": epoch + 1,  # 로그에 1부터 시작하는 epoch 번호 기록
        "train_loss": history.history['loss'][0],
        "val_loss": history.history['val_loss'][0],
        "train_rmse": history.history['rmse_metric'][0],
        "val_rmse": history.history['val_rmse_metric'][0],
    })

model_same_hyper15.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 9s 36ms/step - loss: 0.2853 - rmse_metric: 0.2852 - val_loss: 0.3240 - val_rmse_metric: 0.3247
Epoch 2/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - loss: 0.1509 - rmse_metric: 0.1509 - val_loss: 0.3304 - val_rmse_metric: 0.3317
Epoch 3/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 0.1048 - rmse_metric: 0.1048 - val_loss: 0.2978 - val_rmse_metric: 0.2990
Epoch 4/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 0.0879 - rmse_metric: 0.0878 - val_loss: 0.3216 - val_rmse_metric: 0.3228
Epoch 5/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 0.0781 - rmse_metric: 0.0781 - val_loss: 0.2857 - val_rmse_metric: 0.2870
Epoch 6/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 0.0728 - rmse_metric: 0.0728 - val_loss: 0.2621 - val_rmse_metric: 0.2633
Epoch 7/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 0.0628 - rmse_metric: 0.0628 - val_loss: 0.2630 - val_rmse_metric: 0.2642
Epoch 8/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - loss: 0.0544 - r

Model: "sequential_18"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ bidirectional_8 (Bidirectional)      │ (None, 30, 256)             │         167,936 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_9 (Bidirectional)      │ (None, 30, 256)             │         394,240 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_10 (Bidirectional)     │ (None, 256)                 │         394,240 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ attention_2 (Attention)              │ (None)                      │          65,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_11 (Flatten)                 │ (None, 1)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 1)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_32 (Dense)                     │ (None, 128)                 │             256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_33 (Dense)                     │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,067,781 (11.70 MB)

 Trainable params: 1,022,593 (3.90 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,045,188 (7.80 MB)

##### 16

In [ ]:
wandb.init(project="Stock Price LSTM - Split, Same", name="hyper16", config={
    "epochs": 50,
    "batch_size": 64,
    "time_steps": 60,  # LSTM이 한번에 처리하는 시계열 길이
    "lstm_units": [128,128,128],
    "optimizer": "adam",
    "loss": "rmse",
},reinit=True)

epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇███
train_loss,█▅▄▃▂▂▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_rmse,█▅▄▃▂▂▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,██▆▆▄▅▅▃▄▂▂▃▄▂▂▂▂▂▄▁▂▂▁▂▃▁▂▂▁▂▂▁▂▁▂▂▂▂▁▂
val_rmse,██▆█▆▄▅▅▃▄▂▂▃▄▂▃▂▂▂▄▂▂▁▂▃▁▂▂▁▂▂▂▂▁▂▂▂▂▁▂
epoch,50
train_loss,0.04835
train_rmse,0.0485
val_loss,0.21061
val_rmse,0.21108


In [ ]:
model_checkpoint = ModelCheckpoint(
    '/content/drive/MyDrive/Time Series Project/Stock Price/Model/model_path/hyper16.keras',  # 모델 저장 경로
    monitor='val_loss',  # 모니터할 지표
    save_best_only=True,  # 가장 좋은 모델만 저장
    mode='min',  # `val_loss`가 최소화될 때 저장
    verbose=1  # 저장 시 로그 출력 여부
)

time_steps = wandb.config.time_steps  # Use W&B config
X_train, y_train = create_sequences(x_train_same_df, y_train_same_df, time_steps)
X_val, y_val = create_sequences(x_val_same_df, y_val_same_df, time_steps)

model_same_hyper16 = tf.keras.Sequential([
    Bidirectional(tf.keras.layers.LSTM(wandb.config.lstm_units[0], activation='tanh', input_shape=(time_steps, len(features)), return_sequences=True)),
    Bidirectional(tf.keras.layers.LSTM(wandb.config.lstm_units[1], activation='tanh', return_sequences=True)),
    Bidirectional(tf.keras.layers.LSTM(wandb.config.lstm_units[2], activation='tanh')),
    Attention(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1)
])

# 모델 컴파일: 사용자 정의 RMSE 메트릭 사용
model_same_hyper16.compile(optimizer=wandb.config.optimizer, loss=rmse_metric, metrics=[rmse_metric])

# Epoch-by-epoch 학습과 W&B 로깅
for epoch in range(wandb.config.epochs):
    print(f"Epoch {epoch + 1}/{wandb.config.epochs}")
    history = model_same_hyper16.fit(
        X_train, y_train,
        epochs=1,  # 한 번에 1 epoch만 학습
        batch_size=wandb.config.batch_size,
        validation_data=(X_val, y_val),
        verbose=1,
    )
    # W&B에 로그 기록
    wandb.log({
        "epoch": epoch + 1,  # 로그에 1부터 시작하는 epoch 번호 기록
        "train_loss": history.history['loss'][0],
        "val_loss": history.history['val_loss'][0],
        "train_rmse": history.history['rmse_metric'][0],
        "val_rmse": history.history['val_rmse_metric'][0],
    })

model_same_hyper16.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 8s 40ms/step - loss: 0.3690 - rmse_metric: 0.3689 - val_loss: 0.2843 - val_rmse_metric: 0.2907
Epoch 2/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 2s 29ms/step - loss: 0.0515 - rmse_metric: 0.0515 - val_loss: 0.2093 - val_rmse_metric: 0.2131
Epoch 3/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 2s 29ms/step - loss: 0.0293 - rmse_metric: 0.0293 - val_loss: 0.1657 - val_rmse_metric: 0.1679
Epoch 4/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 2s 33ms/step - loss: 0.0225 - rmse_metric: 0.0225 - val_loss: 0.1578 - val_rmse_metric: 0.1597
Epoch 5/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 2s 34ms/step - loss: 0.0204 - rmse_metric: 0.0204 - val_loss: 0.1762 - val_rmse_metric: 0.1784
Epoch 6/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 2s 29ms/step - loss: 0.0217 - rmse_metric: 0.0217 - val_loss: 0.1571 - val_rmse_metric: 0.1591
Epoch 7/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 2s 29ms/step - loss: 0.0206 - rmse_metric: 0.0206 - val_loss: 0.1313 - val_rmse_metric: 0.1337
Epoch 8/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 2s 29ms/step - loss: 0.0161 - r

Model: "sequential_19"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ bidirectional_11 (Bidirectional)     │ (None, 60, 256)             │         167,936 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_12 (Bidirectional)     │ (None, 60, 256)             │         394,240 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_13 (Bidirectional)     │ (None, 256)                 │         394,240 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ attention_3 (Attention)              │ (None)                      │          65,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_12 (Flatten)                 │ (None, 1)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_34 (Dense)                     │ (None, 128)                 │             256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_35 (Dense)                     │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,067,781 (11.70 MB)

 Trainable params: 1,022,593 (3.90 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,045,188 (7.80 MB)

##### 17

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
wandb.init(project="Stock Price LSTM - Split, Same", name="hyper17", config={
    "epochs": 50,
    "batch_size": 64,
    "time_steps": 60,  # LSTM이 한번에 처리하는 시계열 길이
    "lstm_units": [128,128,128,128],
    "optimizer": "adam",
    "loss": "rmse",
},reinit=True)

In [ ]:
early_stopping = EarlyStopping(
    monitor='val_loss',  # 모니터할 지표 (예: 'val_loss' 또는 'val_rmse')
    patience=5,          # 성능 향상이 없는 epoch 수 (5번 동안 개선 없으면 중단)
    restore_best_weights=True,  # 가장 성능이 좋은 가중치를 복원
    verbose=1            # 로그 출력 여부
)

In [ ]:
model_checkpoint = ModelCheckpoint(
    '/content/drive/MyDrive/Time Series Project/Stock Price/Model/model_path/hyper17.keras',  # 모델 저장 경로
    monitor='val_loss',  # 모니터할 지표
    save_best_only=True,  # 가장 좋은 모델만 저장
    mode='min',  # `val_loss`가 최소화될 때 저장
    verbose=1  # 저장 시 로그 출력 여부
)

time_steps = wandb.config.time_steps  # Use W&B config
X_train, y_train = create_sequences(x_train_same_df, y_train_same_df, time_steps)
X_val, y_val = create_sequences(x_val_same_df, y_val_same_df, time_steps)

model_same_hyper17 = tf.keras.Sequential([
    Bidirectional(tf.keras.layers.LSTM(wandb.config.lstm_units[0], activation='tanh', input_shape=(time_steps, len(features)), return_sequences=True)),
    Bidirectional(tf.keras.layers.LSTM(wandb.config.lstm_units[1], activation='tanh', return_sequences=True)),
    Bidirectional(tf.keras.layers.LSTM(wandb.config.lstm_units[2], activation='tanh', return_sequences=True)),
    Bidirectional(tf.keras.layers.LSTM(wandb.config.lstm_units[3], activation='tanh')),
    Attention(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1)
])

# 모델 컴파일: 사용자 정의 RMSE 메트릭 사용
model_same_hyper17.compile(optimizer=wandb.config.optimizer, loss=rmse_metric, metrics=[rmse_metric])

# Epoch-by-epoch 학습과 W&B 로깅
for epoch in range(wandb.config.epochs):
    print(f"Epoch {epoch + 1}/{wandb.config.epochs}")
    history = model_same_hyper17.fit(
        X_train, y_train,
        epochs=1,  # 한 번에 1 epoch만 학습
        batch_size=wandb.config.batch_size,
        validation_data=(X_val, y_val),
        callbacks=[early_stopping],
        verbose=1,
    )
    # W&B에 로그 기록
    wandb.log({
        "epoch": epoch + 1,  # 로그에 1부터 시작하는 epoch 번호 기록
        "train_loss": history.history['loss'][0],
        "val_loss": history.history['val_loss'][0],
        "train_rmse": history.history['rmse_metric'][0],
        "val_rmse": history.history['val_rmse_metric'][0],
    })

model_same_hyper17.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 9s 52ms/step - loss: 0.3097 - rmse_metric: 0.3096 - val_loss: 0.3096 - val_rmse_metric: 0.3127
Restoring model weights from the end of the best epoch: 1.
Epoch 2/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 3s 46ms/step - loss: 0.1510 - rmse_metric: 0.1510 - val_loss: 0.3093 - val_rmse_metric: 0.3123
Restoring model weights from the end of the best epoch: 1.
Epoch 3/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 2s 40ms/step - loss: 0.1511 - rmse_metric: 0.1510 - val_loss: 0.2989 - val_rmse_metric: 0.3019
Restoring model weights from the end of the best epoch: 1.
Epoch 4/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 2s 40ms/step - loss: 0.1521 - rmse_metric: 0.1521 - val_loss: 0.3249 - val_rmse_metric: 0.3279
Restoring model weights from the end of the best epoch: 1.
Epoch 5/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 3s 50ms/step - loss: 0.1519 - rmse_metric: 0.1519 - val_loss: 0.3126 - val_rmse_metric: 0.3156
Restoring model weights from the end of the best epoch: 1.
Epoch 6/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 3s

Model: "sequential_20"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ bidirectional_14 (Bidirectional)     │ (None, 60, 256)             │         167,936 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_15 (Bidirectional)     │ (None, 60, 256)             │         394,240 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_16 (Bidirectional)     │ (None, 60, 256)             │         394,240 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_17 (Bidirectional)     │ (None, 256)                 │         394,240 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ attention_4 (Attention)              │ (None)                      │          65,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_13 (Flatten)                 │ (None, 1)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_36 (Dense)                     │ (None, 128)                 │             256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_37 (Dense)                     │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 4,250,501 (16.21 MB)

 Trainable params: 1,416,833 (5.40 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,833,668 (10.81 MB)

##### 18

In [ ]:
wandb.init(project="Stock Price LSTM - Split, Same", name="hyper18", config={
    "epochs": 50,
    "batch_size": 64,
    "time_steps": 30,  # LSTM이 한번에 처리하는 시계열 길이
    "lstm_units": [128,128,128],
    "optimizer": "adam",
    "loss": "rmse",
},reinit=True)

epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇██
train_loss,█▆▆▆▆▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_rmse,█▆▆▆▆▂▂▂▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,▇▇▇██▆▃▃▂▄▃▂▂▂▃▂▃▂▂▁▂▂▃▂▂▂▁▂▂▂▁▂▂▂▂▂▂▂▂▂
val_rmse,▇▇▇██▆▃▄▂▄▃▂▂▃▃▃▂▂▁▂▂▃▂▂▃▂▂▂▂▂▁▂▂▂▂▂▂▂▂▂
epoch,50
train_loss,0.01281
train_rmse,0.01272
val_loss,0.16783
val_rmse,0.17126


In [ ]:
model_checkpoint = ModelCheckpoint(
    '/content/drive/MyDrive/Time Series Project/Stock Price/Model/model_path/hyper18.keras',  # 모델 저장 경로
    monitor='val_loss',  # 모니터할 지표
    save_best_only=True,  # 가장 좋은 모델만 저장
    mode='min',  # `val_loss`가 최소화될 때 저장
    verbose=1  # 저장 시 로그 출력 여부
)

time_steps = wandb.config.time_steps  # Use W&B config
X_train, y_train = create_sequences(x_train_same_df, y_train_same_df, time_steps)
X_val, y_val = create_sequences(x_val_same_df, y_val_same_df, time_steps)

model_same_hyper18 = tf.keras.Sequential([
    Bidirectional(tf.keras.layers.LSTM(wandb.config.lstm_units[0], activation='tanh', input_shape=(time_steps, len(features)), return_sequences=True)),
    Bidirectional(tf.keras.layers.LSTM(wandb.config.lstm_units[1], activation='tanh', return_sequences=True)),
    Bidirectional(tf.keras.layers.LSTM(wandb.config.lstm_units[2], activation='tanh')),
    Attention(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1)
])

# 모델 컴파일: 사용자 정의 RMSE 메트릭 사용
model_same_hyper18.compile(optimizer=wandb.config.optimizer, loss=rmse_metric, metrics=[rmse_metric])

# Epoch-by-epoch 학습과 W&B 로깅
for epoch in range(wandb.config.epochs):
    print(f"Epoch {epoch + 1}/{wandb.config.epochs}")
    history = model_same_hyper18.fit(
        X_train, y_train,
        epochs=1,  # 한 번에 1 epoch만 학습
        batch_size=wandb.config.batch_size,
        validation_data=(X_val, y_val),
        callbacks=[early_stopping],
        verbose=1,
    )
    # W&B에 로그 기록
    wandb.log({
        "epoch": epoch + 1,  # 로그에 1부터 시작하는 epoch 번호 기록
        "train_loss": history.history['loss'][0],
        "val_loss": history.history['val_loss'][0],
        "train_rmse": history.history['rmse_metric'][0],
        "val_rmse": history.history['val_rmse_metric'][0],
    })

model_same_hyper18.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 6s 31ms/step - loss: 0.2989 - rmse_metric: 0.2989 - val_loss: 0.3301 - val_rmse_metric: 0.3308
Restoring model weights from the end of the best epoch: 1.
Epoch 2/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 0.1558 - rmse_metric: 0.1558 - val_loss: 0.3061 - val_rmse_metric: 0.3068
Restoring model weights from the end of the best epoch: 1.
Epoch 3/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - loss: 0.1549 - rmse_metric: 0.1549 - val_loss: 0.3177 - val_rmse_metric: 0.3184
Restoring model weights from the end of the best epoch: 1.
Epoch 4/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - loss: 0.1546 - rmse_metric: 0.1546 - val_loss: 0.3203 - val_rmse_metric: 0.3209
Restoring model weights from the end of the best epoch: 1.
Epoch 5/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 2s 27ms/step - loss: 0.1383 - rmse_metric: 0.1383 - val_loss: 0.2853 - val_rmse_metric: 0.2867
Restoring model weights from the end of the best epoch: 1.
Epoch 6/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s

Model: "sequential_22"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ bidirectional_21 (Bidirectional)     │ (None, 30, 256)             │         167,936 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_22 (Bidirectional)     │ (None, 30, 256)             │         394,240 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_23 (Bidirectional)     │ (None, 256)                 │         394,240 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ attention_6 (Attention)              │ (None)                      │          65,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_15 (Flatten)                 │ (None, 1)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_40 (Dense)                     │ (None, 128)                 │             256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_41 (Dense)                     │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,067,781 (11.70 MB)

 Trainable params: 1,022,593 (3.90 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,045,188 (7.80 MB)

##### 19

In [ ]:
wandb.init(project="Stock Price LSTM - Split, Same", name="hyper19", config={
    "epochs": 50,
    "batch_size": 64,
    "time_steps": 60,  # LSTM이 한번에 처리하는 시계열 길이
    "lstm_units": [128,128,128],
    "optimizer": "adam",
    "loss": "rmse",
},reinit=True)

epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
train_loss,█▆▆▆▅▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_rmse,█▆▆▆▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▇▇█▆▃▄▃▃▃▃▃▃▃▃▂▂▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁
val_rmse,█▇▇█▆▃▄▃▃▃▃▃▃▃▃▂▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁
epoch,50
train_loss,0.01108
train_rmse,0.01111
val_loss,0.17171
val_rmse,0.17256


In [ ]:
model_checkpoint = ModelCheckpoint(
    '/content/drive/MyDrive/Time Series Project/Stock Price/Model/model_path/hyper19.keras',  # 모델 저장 경로
    monitor='val_loss',  # 모니터할 지표
    save_best_only=True,  # 가장 좋은 모델만 저장
    mode='min',  # `val_loss`가 최소화될 때 저장
    verbose=1  # 저장 시 로그 출력 여부
)

time_steps = wandb.config.time_steps  # Use W&B config
X_train, y_train = create_sequences(x_train_same_df, y_train_same_df, time_steps)
X_val, y_val = create_sequences(x_val_same_df, y_val_same_df, time_steps)

model_same_hyper19 = tf.keras.Sequential([
    Bidirectional(tf.keras.layers.LSTM(wandb.config.lstm_units[0], activation='tanh', input_shape=(time_steps, len(features)), return_sequences=True)),
    Bidirectional(tf.keras.layers.LSTM(wandb.config.lstm_units[1], activation='tanh', return_sequences=True)),
    Bidirectional(tf.keras.layers.LSTM(wandb.config.lstm_units[2], activation='tanh')),
    Attention(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1)
])

# 모델 컴파일: 사용자 정의 RMSE 메트릭 사용
model_same_hyper19.compile(optimizer=wandb.config.optimizer, loss=rmse_metric, metrics=[rmse_metric])

# Epoch-by-epoch 학습과 W&B 로깅
for epoch in range(wandb.config.epochs):
    print(f"Epoch {epoch + 1}/{wandb.config.epochs}")
    history = model_same_hyper19.fit(
        X_train, y_train,
        epochs=1,  # 한 번에 1 epoch만 학습
        batch_size=wandb.config.batch_size,
        validation_data=(X_val, y_val),
        callbacks=[early_stopping],
        verbose=1,
    )
    # W&B에 로그 기록
    wandb.log({
        "epoch": epoch + 1,  # 로그에 1부터 시작하는 epoch 번호 기록
        "train_loss": history.history['loss'][0],
        "val_loss": history.history['val_loss'][0],
        "train_rmse": history.history['rmse_metric'][0],
        "val_rmse": history.history['val_rmse_metric'][0],
    })

model_same_hyper19.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 8s 42ms/step - loss: 0.2787 - rmse_metric: 0.2787 - val_loss: 0.3068 - val_rmse_metric: 0.3098
Restoring model weights from the end of the best epoch: 1.
Epoch 2/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 2s 29ms/step - loss: 0.1529 - rmse_metric: 0.1528 - val_loss: 0.2931 - val_rmse_metric: 0.2961
Restoring model weights from the end of the best epoch: 1.
Epoch 3/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 2s 29ms/step - loss: 0.1499 - rmse_metric: 0.1498 - val_loss: 0.3402 - val_rmse_metric: 0.3432
Restoring model weights from the end of the best epoch: 1.
Epoch 4/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 2s 29ms/step - loss: 0.1492 - rmse_metric: 0.1492 - val_loss: 0.3407 - val_rmse_metric: 0.3467
Restoring model weights from the end of the best epoch: 1.
Epoch 5/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 2s 31ms/step - loss: 0.0897 - rmse_metric: 0.0897 - val_loss: 0.1929 - val_rmse_metric: 0.1971
Restoring model weights from the end of the best epoch: 1.
Epoch 6/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 2s

Model: "sequential_23"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ bidirectional_24 (Bidirectional)     │ (None, 60, 256)             │         167,936 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_25 (Bidirectional)     │ (None, 60, 256)             │         394,240 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_26 (Bidirectional)     │ (None, 256)                 │         394,240 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ attention_7 (Attention)              │ (None)                      │          65,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_16 (Flatten)                 │ (None, 1)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_42 (Dense)                     │ (None, 128)                 │             256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_43 (Dense)                     │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,067,781 (11.70 MB)

 Trainable params: 1,022,593 (3.90 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,045,188 (7.80 MB)

##### Model Save & Load

In [ ]:
# 학습된 모델 저장
model_same_hyper13.save('/content/drive/MyDrive/Time Series Project/Stock Price/Model/model_path/hyper13.keras')
print("모델이 성공적으로 저장되었습니다!")

모델이 성공적으로 저장되었습니다!


In [ ]:
# 학습된 모델 저장
model_same_hyper19.save('/content/drive/MyDrive/Time Series Project/Stock Price/Model/model_path/hyper19.keras')
print("모델이 성공적으로 저장되었습니다!")

모델이 성공적으로 저장되었습니다!


load

In [47]:
class Attention(tf.keras.layers.Layer):
    def __init__(self):
        super(Attention, self).__init__()

    def build(self, input_shape):
        self.W = self.add_weight(name="att_weight", shape=(input_shape[-1], input_shape[-1]), initializer="normal")
        self.b = self.add_weight(name="att_bias", shape=(input_shape[-1],), initializer="zeros")
        super(Attention, self).build(input_shape)

    def call(self, x):
        score = tf.nn.tanh(tf.tensordot(x, self.W, axes=1) + self.b)
        attention_weights = tf.nn.softmax(score, axis=1)
        context_vector = attention_weights * x
        context_vector = tf.reduce_sum(context_vector, axis=1)
        return context_vector

In [48]:
from tensorflow.keras.saving import register_keras_serializable

@register_keras_serializable()
def rmse_metric(y_true, y_pred):
    return tf.sqrt(tf.reduce_mean(tf.square(y_true - y_pred)))

In [49]:
wandb_config={
    "epochs": 50,
    "batch_size": 64,
    "time_steps": 30,  # LSTM이 한번에 처리하는 시계열 길이
    "lstm_units": [128,128,128],
    "optimizer": "adam",
    "loss": "rmse",
}

In [50]:
model = tf.keras.Sequential([
    Bidirectional(LSTM(wandb_config['lstm_units'][0], activation='tanh', input_shape=(wandb_config['time_steps'], features), return_sequences=True)),
    Bidirectional(LSTM(wandb_config['lstm_units'][1], activation='tanh', return_sequences=True)),
    Bidirectional(LSTM(wandb_config['lstm_units'][2], activation='tanh')),
    Attention(),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(1)
])


In [52]:
# 모델 정의
model = tf.keras.Sequential([
    Bidirectional(LSTM(wandb_config['lstm_units'][0], activation='tanh', input_shape=(wandb_config['time_steps'], features), return_sequences=True)),
    Bidirectional(LSTM(wandb_config['lstm_units'][1], activation='tanh', return_sequences=True)),
    Bidirectional(LSTM(wandb_config['lstm_units'][2], activation='tanh')),
    Attention(),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(1)
])

# 더미 입력으로 모델 빌드
dummy_input = tf.zeros((1, wandb_config['time_steps'], features))
model(dummy_input)  # 입력 데이터를 사용해 모델 빌드

# 가중치 로드
model.load_weights('/content/drive/MyDrive/Time Series Project/Stock Price/Model/model_path/hyper13.keras')

# 모델 구조 확인
model.summary()


ValueError: Can't convert Python sequence with mixed types to Tensor.

## CNN-LSTM Hyperparameter tuning - Same

##### 1

In [ ]:
wandb.init(project="Stock Price CNN-LSTM - Split, Same", name="hyper1", config={
    "epochs": 30,
    "batch_size": 32,
    "time_steps": 30,  # LSTM이 한번에 처리하는 시계열 길이
    "lstm_units": [64, 64],
    "optimizer": "adam",
    "loss": "rmse",
},reinit=True)

In [ ]:
model_checkpoint = ModelCheckpoint(
    '/content/drive/MyDrive/Time Series Project/Stock Price/Model/model_path/int_hyper1.keras',
    monitor='val_loss',
    save_best_only=True,
    mode='min',
    verbose=1
)

time_steps = wandb.config.time_steps  # Use W&B config
X_train, y_train = create_sequences(x_train_int_df, y_train_int_df, time_steps)
X_val, y_val = create_sequences(x_val_int_df, y_val_int_df, time_steps)

model_interp_hyper1 = tf.keras.Sequential([
    tf.keras.layers.Conv1D(64, kernel_size=6, activation='relu', input_shape=(time_steps, len(features))),
    tf.keras.layers.MaxPooling1D(2),

    tf.keras.layers.Conv1D(64, kernel_size=3, activation='relu'),
    tf.keras.layers.MaxPooling1D(2),

    tf.keras.layers.LSTM(wandb.config.lstm_units[0], activation='tanh', return_sequences=True),
    tf.keras.layers.LSTM(wandb.config.lstm_units[1], activation='tanh'),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1)
])

# 모델 컴파일: 사용자 정의 RMSE 메트릭 사용
model_interp_hyper1.compile(optimizer=wandb.config.optimizer, loss=rmse_metric, metrics=[rmse_metric])

# Epoch-by-epoch 학습과 W&B 로깅
for epoch in range(wandb.config.epochs):
    print(f"Epoch {epoch + 1}/{wandb.config.epochs}")
    history = model_interp_hyper1.fit(
        X_train, y_train,
        epochs=1,  # 한 번에 1 epoch만 학습
        batch_size=wandb.config.batch_size,
        validation_data=(X_val, y_val),
        callbacks=[early_stopping],
        verbose=1,
    )
    # W&B에 로그 기록
    wandb.log({
        "epoch": epoch + 1,  # 로그에 1부터 시작하는 epoch 번호 기록
        "train_loss": history.history['loss'][0],
        "val_loss": history.history['val_loss'][0],
        "train_rmse": history.history['rmse_metric'][0],
        "val_rmse": history.history['val_rmse_metric'][0],
    })

model_interp_hyper1.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/30
110/110 ━━━━━━━━━━━━━━━━━━━━ 7s 15ms/step - loss: 0.1057 - rmse_metric: 0.1057 - val_loss: 0.2003 - val_rmse_metric: 0.2011
Restoring model weights from the end of the best epoch: 1.
Epoch 2/30
110/110 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.0288 - rmse_metric: 0.0288 - val_loss: 0.1710 - val_rmse_metric: 0.1718
Restoring model weights from the end of the best epoch: 1.
Epoch 3/30
110/110 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.0231 - rmse_metric: 0.0231 - val_loss: 0.1634 - val_rmse_metric: 0.1642
Restoring model weights from the end of the best epoch: 1.
Epoch 4/30
110/110 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.0253 - rmse_metric: 0.0253 - val_loss: 0.1691 - val_rmse_metric: 0.1700
Restoring model weights from the end of the best epoch: 1.
Epoch 5/30
110/110 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.0256 - rmse_metric: 0.0256 - val_loss: 0.1765 - val_rmse_metric: 0.1775
Restoring model weights from the end of the best epoch: 1.
Epoch 6/30
110/110 ━━━━━━━━━━━━━━━

Model: "sequential_24"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                      │ (None, 25, 64)              │          13,504 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d (MaxPooling1D)         │ (None, 12, 64)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_1 (Conv1D)                    │ (None, 10, 64)              │          12,352 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_1 (MaxPooling1D)       │ (None, 5, 64)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_75 (LSTM)                       │ (None, 5, 64)               │          33,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_76 (LSTM)                       │ (None, 64)                  │          33,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_17 (Flatten)                 │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_44 (Dense)                     │ (None, 128)                 │           8,320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_45 (Dense)                     │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 301,061 (1.15 MB)

 Trainable params: 100,353 (392.00 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 200,708 (784.02 KB)

##### 2

In [ ]:
wandb.init(project="Stock Price CNN-LSTM - Split, Same", name="hyper2", config={
    "epochs": 30,
    "batch_size": 32,
    "time_steps": 30,  # LSTM이 한번에 처리하는 시계열 길이
    "lstm_units": [64, 64, 64],
    "optimizer": "adam",
    "loss": "rmse",
},reinit=True)

epoch,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
train_loss,█▃▂▃▃▂▂▂▂▂▂▂▁▂▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁
train_rmse,█▃▃▃▃▂▂▂▂▂▂▂▁▂▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,▇▄▃▃▄▂▂▁█▂▅▄▇▅▅▅▆▆▅▅▅▆▆▇▄▆▅▅▇▅
val_rmse,▇▄▃▃▄▂▂▁█▂▅▄▇▅▅▅▆▆▅▅▅▆▇▇▄▆▅▅▇▅
epoch,30
train_loss,0.01368
train_rmse,0.01368
val_loss,0.18524
val_rmse,0.18635


In [ ]:
model_checkpoint = ModelCheckpoint(
    '/content/drive/MyDrive/Time Series Project/Stock Price/Model/model_path/int_hyper2.keras',  # 모델 저장 경로
    monitor='val_loss',  # 모니터할 지표
    save_best_only=True,  # 가장 좋은 모델만 저장
    mode='min',  # `val_loss`가 최소화될 때 저장
    verbose=1  # 저장 시 로그 출력 여부
)

time_steps = wandb.config.time_steps  # Use W&B config
X_train, y_train = create_sequences(x_train_int_df, y_train_int_df, time_steps)
X_val, y_val = create_sequences(x_val_int_df, y_val_int_df, time_steps)

model_interp_hyper2 = tf.keras.Sequential([
    tf.keras.layers.Conv1D(64, kernel_size=7, activation='relu', input_shape=(time_steps, len(features))),
    tf.keras.layers.MaxPooling1D(2),

    tf.keras.layers.Conv1D(64, kernel_size=7, activation='relu'),
    tf.keras.layers.MaxPooling1D(2),

    tf.keras.layers.LSTM(wandb.config.lstm_units[0], activation='tanh', return_sequences=True),
    tf.keras.layers.LSTM(wandb.config.lstm_units[1], activation='tanh', return_sequences=True),
    tf.keras.layers.LSTM(wandb.config.lstm_units[1], activation='tanh'),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1)
])

# 모델 컴파일: 사용자 정의 RMSE 메트릭 사용
model_interp_hyper2.compile(optimizer=wandb.config.optimizer, loss=rmse_metric, metrics=[rmse_metric])

# Epoch-by-epoch 학습과 W&B 로깅
for epoch in range(wandb.config.epochs):
    print(f"Epoch {epoch + 1}/{wandb.config.epochs}")
    history = model_interp_hyper2.fit(
        X_train, y_train,
        epochs=1,  # 한 번에 1 epoch만 학습
        batch_size=wandb.config.batch_size,
        validation_data=(X_val, y_val),
        callbacks=[early_stopping],
        verbose=1,
    )
    # W&B에 로그 기록
    wandb.log({
        "epoch": epoch + 1,  # 로그에 1부터 시작하는 epoch 번호 기록
        "train_loss": history.history['loss'][0],
        "val_loss": history.history['val_loss'][0],
        "train_rmse": history.history['rmse_metric'][0],
        "val_rmse": history.history['val_rmse_metric'][0],
    })

model_interp_hyper2.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/30
110/110 ━━━━━━━━━━━━━━━━━━━━ 8s 13ms/step - loss: 0.1553 - rmse_metric: 0.1553 - val_loss: 0.1864 - val_rmse_metric: 0.1873
Restoring model weights from the end of the best epoch: 1.
Epoch 2/30
110/110 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0296 - rmse_metric: 0.0296 - val_loss: 0.2144 - val_rmse_metric: 0.2152
Restoring model weights from the end of the best epoch: 1.
Epoch 3/30
110/110 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0302 - rmse_metric: 0.0302 - val_loss: 0.1513 - val_rmse_metric: 0.1520
Restoring model weights from the end of the best epoch: 1.
Epoch 4/30
110/110 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0233 - rmse_metric: 0.0233 - val_loss: 0.1783 - val_rmse_metric: 0.1792
Restoring model weights from the end of the best epoch: 1.
Epoch 5/30
110/110 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0245 - rmse_metric: 0.0245 - val_loss: 0.1276 - val_rmse_metric: 0.1281
Restoring model weights from the end of the best epoch: 1.
Epoch 6/30
110/110 ━━━━━━━━━━━━━━━

Model: "sequential_25"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv1d_2 (Conv1D)                    │ (None, 24, 64)              │          15,744 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_2 (MaxPooling1D)       │ (None, 12, 64)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_3 (Conv1D)                    │ (None, 6, 64)               │          28,736 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_3 (MaxPooling1D)       │ (None, 3, 64)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_77 (LSTM)                       │ (None, 3, 64)               │          33,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_78 (LSTM)                       │ (None, 3, 64)               │          33,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_79 (LSTM)                       │ (None, 64)                  │          33,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_18 (Flatten)                 │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_46 (Dense)                     │ (None, 128)                 │           8,320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_47 (Dense)                     │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 456,005 (1.74 MB)

 Trainable params: 152,001 (593.75 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 304,004 (1.16 MB)

##### 3

In [ ]:
wandb.init(project="Stock Price CNN-LSTM - Split, Same", name="hyper3", config={
    "epochs": 30,
    "batch_size": 64,
    "time_steps": 60,  # LSTM이 한번에 처리하는 시계열 길이
    "lstm_units": [64, 64, 64],
    "optimizer": "adam",
    "loss": "rmse",
},reinit=True)

wandb: Currently logged in as: yebin. Use `wandb login --relogin` to force relogin


In [ ]:
model_checkpoint = ModelCheckpoint(
    '/content/drive/MyDrive/Time Series Project/Stock Price/Model/model_path/int_hyper3.keras',
    monitor='val_loss',
    save_best_only=True,
    mode='min',
    verbose=1
)

time_steps = wandb.config.time_steps
X_train, y_train = create_sequences(x_train_int_df, y_train_int_df, time_steps)
X_val, y_val = create_sequences(x_val_int_df, y_val_int_df, time_steps)

model_interp_hyper3 = tf.keras.Sequential([
    tf.keras.layers.Conv1D(64, kernel_size=7, activation='relu', input_shape=(time_steps, len(features))),
    tf.keras.layers.MaxPooling1D(2),

    tf.keras.layers.Conv1D(64, kernel_size=7, activation='relu'),
    tf.keras.layers.MaxPooling1D(2),

    tf.keras.layers.LSTM(wandb.config.lstm_units[0], activation='tanh', return_sequences=True),
    tf.keras.layers.LSTM(wandb.config.lstm_units[1], activation='tanh', return_sequences=True),
    tf.keras.layers.LSTM(wandb.config.lstm_units[1], activation='tanh'),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
])

model_interp_hyper3.compile(optimizer=wandb.config.optimizer, loss=rmse_metric, metrics=[rmse_metric])

for epoch in range(wandb.config.epochs):
    print(f"Epoch {epoch + 1}/{wandb.config.epochs}")
    history = model_interp_hyper3.fit(
        X_train, y_train,
        epochs=1,
        batch_size=wandb.config.batch_size,
        validation_data=(X_val, y_val),
        callbacks=[early_stopping],
        verbose=1,
    )
    wandb.log({
        "epoch": epoch + 1,
        "train_loss": history.history['loss'][0],
        "val_loss": history.history['val_loss'][0],
        "train_rmse": history.history['rmse_metric'][0],
        "val_rmse": history.history['val_rmse_metric'][0],
    })

model_interp_hyper3.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/30
55/55 ━━━━━━━━━━━━━━━━━━━━ 13s 53ms/step - loss: 0.1747 - rmse_metric: 0.1746 - val_loss: 0.1884 - val_rmse_metric: 0.1917
Restoring model weights from the end of the best epoch: 1.
Epoch 2/30
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: 0.0302 - rmse_metric: 0.0302 - val_loss: 0.1875 - val_rmse_metric: 0.1907
Restoring model weights from the end of the best epoch: 1.
Epoch 3/30
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 0.0314 - rmse_metric: 0.0313 - val_loss: 0.1538 - val_rmse_metric: 0.1567
Restoring model weights from the end of the best epoch: 1.
Epoch 4/30
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 0.0250 - rmse_metric: 0.0250 - val_loss: 0.1721 - val_rmse_metric: 0.1753
Restoring model weights from the end of the best epoch: 1.
Epoch 5/30
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0237 - rmse_metric: 0.0237 - val_loss: 0.1438 - val_rmse_metric: 0.1464
Restoring model weights from the end of the best epoch: 1.
Epoch 6/30
55/55 ━━━━━━━━━━━━━━━━━━━━ 1

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                      │ (None, 54, 64)              │          15,744 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d (MaxPooling1D)         │ (None, 27, 64)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_1 (Conv1D)                    │ (None, 21, 64)              │          28,736 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_1 (MaxPooling1D)       │ (None, 10, 64)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ (None, 10, 64)              │          33,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 10, 64)              │          33,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_2 (LSTM)                        │ (None, 64)                  │          33,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 64)                  │           4,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 443,333 (1.69 MB)

 Trainable params: 147,777 (577.25 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 295,556 (1.13 MB)

##### 4

In [ ]:
wandb.finish()

epoch,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
train_loss,█▂▂▂▂▂▂▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_rmse,█▂▂▂▂▂▂▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,▅▅▂▄▁▁▆▅▅▆▃▃▆▆▆▇▇██▅█▆██▅▅▇▇▇▇
val_rmse,▅▅▂▄▁▁▆▅▅▆▃▃▆▆▆▇▇██▅█▆██▅▅▇▇▇▇
epoch,30
train_loss,0.01581
train_rmse,0.01597
val_loss,0.20711
val_rmse,0.2115


In [ ]:
wandb.init(project="Stock Price CNN-LSTM - Split, Same", name="hyper4", config={
    "epochs": 50,
    "batch_size": 64,
    "time_steps": 30,  # LSTM이 한번에 처리하는 시계열 길이
    "lstm_units": [64, 64, 64],
    "optimizer": "adam",
    "loss": "rmse",
},reinit=True)

In [ ]:
model_checkpoint = ModelCheckpoint(
    '/content/drive/MyDrive/Time Series Project/Stock Price/Model/model_path/int_hyper4.keras',
    monitor='val_loss',
    save_best_only=True,
    mode='min',
    verbose=1
)

time_steps = wandb.config.time_steps
X_train, y_train = create_sequences(x_train_int_df, y_train_int_df, time_steps)
X_val, y_val = create_sequences(x_val_int_df, y_val_int_df, time_steps)

model_interp_hyper4 = tf.keras.Sequential([
    tf.keras.layers.Conv1D(64, kernel_size=7, activation='relu', input_shape=(time_steps, len(features))),
    tf.keras.layers.MaxPooling1D(2),

    tf.keras.layers.Conv1D(64, kernel_size=7, activation='relu'),
    tf.keras.layers.MaxPooling1D(2),

    tf.keras.layers.LSTM(wandb.config.lstm_units[0], activation='tanh', return_sequences=True),
    tf.keras.layers.LSTM(wandb.config.lstm_units[1], activation='tanh', return_sequences=True),
    tf.keras.layers.LSTM(wandb.config.lstm_units[1], activation='tanh'),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(1)
])

model_interp_hyper4.compile(optimizer=wandb.config.optimizer, loss=rmse_metric, metrics=[rmse_metric])

for epoch in range(wandb.config.epochs):
    print(f"Epoch {epoch + 1}/{wandb.config.epochs}")
    history = model_interp_hyper4.fit(
        X_train, y_train,
        epochs=1,
        batch_size=wandb.config.batch_size,
        validation_data=(X_val, y_val),
        callbacks=[early_stopping],
        verbose=1,
    )
    wandb.log({
        "epoch": epoch + 1,
        "train_loss": history.history['loss'][0],
        "val_loss": history.history['val_loss'][0],
        "train_rmse": history.history['rmse_metric'][0],
        "val_rmse": history.history['val_rmse_metric'][0],
    })

model_interp_hyper4.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/30
55/55 ━━━━━━━━━━━━━━━━━━━━ 5s 20ms/step - loss: 0.2102 - rmse_metric: 0.2102 - val_loss: 0.2301 - val_rmse_metric: 0.2312
Restoring model weights from the end of the best epoch: 1.
Epoch 2/30
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 0.0527 - rmse_metric: 0.0527 - val_loss: 0.1770 - val_rmse_metric: 0.1778
Restoring model weights from the end of the best epoch: 1.
Epoch 3/30
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0398 - rmse_metric: 0.0398 - val_loss: 0.1614 - val_rmse_metric: 0.1621
Restoring model weights from the end of the best epoch: 1.
Epoch 4/30
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0384 - rmse_metric: 0.0384 - val_loss: 0.1782 - val_rmse_metric: 0.1790
Restoring model weights from the end of the best epoch: 1.
Epoch 5/30
55/55 ━━━━━━━━━━━━━━━━━━━━ 2s 43ms/step - loss: 0.0379 - rmse_metric: 0.0379 - val_loss: 0.1741 - val_rmse_metric: 0.1749
Restoring model weights from the end of the best epoch: 1.
Epoch 6/30
55/55 ━━━━━━━━━━━━━━━━━━━━ 2s

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv1d_2 (Conv1D)                    │ (None, 24, 64)              │          15,744 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_2 (MaxPooling1D)       │ (None, 12, 64)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_3 (Conv1D)                    │ (None, 6, 64)               │          28,736 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_3 (MaxPooling1D)       │ (None, 3, 64)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_3 (LSTM)                        │ (None, 3, 64)               │          33,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_4 (LSTM)                        │ (None, 3, 64)               │          33,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_5 (LSTM)                        │ (None, 64)                  │          33,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 64)                  │           4,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 443,333 (1.69 MB)

 Trainable params: 147,777 (577.25 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 295,556 (1.13 MB)

##### 5

In [ ]:
wandb.finish()

epoch,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
train_loss,█▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_rmse,█▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▃▁▃▂▃▅▃▅▄▃▅▄▅▄▄▃▄▄▄▅▆▅▅▄▄▄▄▅▄
val_rmse,█▃▁▃▂▃▅▃▅▄▃▅▄▅▄▄▃▄▄▄▅▆▅▅▄▄▄▄▅▄
epoch,30
train_loss,0.0247
train_rmse,0.02468
val_loss,0.18693
val_rmse,0.18799


In [ ]:
wandb.init(project="Stock Price CNN-LSTM - Split, Same", name="hyper5", config={
    "epochs": 50,
    "batch_size": 64,
    "time_steps": 30,
    "lstm_units": [64, 64, 64],
    "optimizer": "adam",
    "loss": "rmse",
},reinit=True)

In [ ]:
model_checkpoint = ModelCheckpoint(
    '/content/drive/MyDrive/Time Series Project/Stock Price/Model/model_path/int_hyper5.keras',
    monitor='val_loss',
    save_best_only=True,
    mode='min',
    verbose=1
)

time_steps = wandb.config.time_steps
X_train, y_train = create_sequences(x_train_int_df, y_train_int_df, time_steps)
X_val, y_val = create_sequences(x_val_int_df, y_val_int_df, time_steps)

model_interp_hyper5 = tf.keras.Sequential([
    tf.keras.layers.Conv1D(64, kernel_size=5, activation='relu', input_shape=(time_steps, len(features))),
    tf.keras.layers.MaxPooling1D(2),

    tf.keras.layers.Conv1D(64, kernel_size=5, activation='relu'),
    tf.keras.layers.MaxPooling1D(2),

    tf.keras.layers.LSTM(wandb.config.lstm_units[0], activation='tanh', return_sequences=True),
    tf.keras.layers.LSTM(wandb.config.lstm_units[1], activation='tanh', return_sequences=True),
    tf.keras.layers.LSTM(wandb.config.lstm_units[1], activation='tanh'),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(1)
])

model_interp_hyper5.compile(optimizer=wandb.config.optimizer, loss=rmse_metric, metrics=[rmse_metric])

for epoch in range(wandb.config.epochs):
    print(f"Epoch {epoch + 1}/{wandb.config.epochs}")
    history = model_interp_hyper5.fit(
        X_train, y_train,
        epochs=1,
        batch_size=wandb.config.batch_size,
        validation_data=(X_val, y_val),
        callbacks=[early_stopping, model_checkpoint],
        verbose=1,
    )

    wandb.log({
        "epoch": epoch + 1,
        "train_loss": history.history['loss'][0],
        "val_loss": history.history['val_loss'][0],
        "train_rmse": history.history['rmse_metric'][0],
        "val_rmse": history.history['val_rmse_metric'][0],
    })

model_interp_hyper5.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.2153 - rmse_metric: 0.2153
Epoch 1: val_loss improved from inf to 0.20761, saving model to /content/drive/MyDrive/Time Series Project/Stock Price/Model/model_path/int_hyper5.keras
55/55 ━━━━━━━━━━━━━━━━━━━━ 7s 55ms/step - loss: 0.2137 - rmse_metric: 0.2136 - val_loss: 0.2076 - val_rmse_metric: 0.2089
Restoring model weights from the end of the best epoch: 1.
Epoch 2/50
51/55 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0473 - rmse_metric: 0.0473
Epoch 1: val_loss did not improve from 0.20761
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0470 - rmse_metric: 0.0470 - val_loss: 0.2113 - val_rmse_metric: 0.2124
Restoring model weights from the end of the best epoch: 1.
Epoch 3/50
51/55 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0360 - rmse_metric: 0.0360
Epoch 1: val_loss did not improve from 0.20761
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0360 - rmse_metric: 0.0360 - val_loss: 0.2149 - val_rmse_metric: 0.2158
Restori

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv1d_4 (Conv1D)                    │ (None, 26, 64)              │          11,264 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_4 (MaxPooling1D)       │ (None, 13, 64)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_5 (Conv1D)                    │ (None, 9, 64)               │          20,544 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_5 (MaxPooling1D)       │ (None, 4, 64)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_6 (LSTM)                        │ (None, 4, 64)               │          33,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_7 (LSTM)                        │ (None, 4, 64)               │          33,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_8 (LSTM)                        │ (None, 64)                  │          33,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_2 (Flatten)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 128)                 │           8,320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 417,989 (1.59 MB)

 Trainable params: 139,329 (544.25 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 278,660 (1.06 MB)

##### 6

In [ ]:
wandb.finish()

epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇██
train_loss,█▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_rmse,█▃▂▂▂▂▂▂▂▂▂▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,▇██▅▄▄▂▃▅▃▂▂▅▃▃▄▄▃▃▃▃▂▂▄▁▄▄▂▃▃▃▃▃▂▃▄▃▃▂▃
val_rmse,▇██▅▄▄▂▃▅▃▂▂▅▃▄▂▄▃▃▃▃▂▂▄▁▄▄▂▃▃▃▃▃▂▃▄▃▃▂▃
epoch,50
train_loss,0.02124
train_rmse,0.02128
val_loss,0.18086
val_rmse,0.18177


In [ ]:
wandb.init(project="Stock Price CNN-LSTM - Split, Same", name="hyper6", config={
    "epochs": 50,
    "batch_size": 64,
    "time_steps": 30,
    "lstm_units": [64, 64, 64],
    "optimizer": "adam",
    "loss": "rmse",
},reinit=True)

In [ ]:
model_checkpoint = ModelCheckpoint(
    '/content/drive/MyDrive/Time Series Project/Stock Price/Model/model_path/int_hyper6.keras',
    monitor='val_loss',
    save_best_only=True,
    mode='min',
    verbose=1
)

time_steps = wandb.config.time_steps
X_train, y_train = create_sequences(x_train_int_df, y_train_int_df, time_steps)
X_val, y_val = create_sequences(x_val_int_df, y_val_int_df, time_steps)

model_interp_hyper6 = tf.keras.Sequential([
    tf.keras.layers.Conv1D(64, kernel_size=7, activation='relu', input_shape=(time_steps, len(features))),
    tf.keras.layers.MaxPooling1D(2),

    tf.keras.layers.Conv1D(64, kernel_size=7, activation='relu'),
    tf.keras.layers.MaxPooling1D(2),

    tf.keras.layers.LSTM(wandb.config.lstm_units[0], activation='tanh', return_sequences=True),
    tf.keras.layers.LSTM(wandb.config.lstm_units[1], activation='tanh', return_sequences=True),
    tf.keras.layers.LSTM(wandb.config.lstm_units[2], activation='tanh'),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(1)
])

model_interp_hyper6.compile(optimizer=wandb.config.optimizer, loss=rmse_metric, metrics=[rmse_metric])

for epoch in range(wandb.config.epochs):
    print(f"Epoch {epoch + 1}/{wandb.config.epochs}")
    history = model_interp_hyper6.fit(
        X_train, y_train,
        epochs=1,
        batch_size=wandb.config.batch_size,
        validation_data=(X_val, y_val),
        callbacks=[early_stopping, model_checkpoint],
        verbose=1,
    )

    wandb.log({
        "epoch": epoch + 1,
        "train_loss": history.history['loss'][0],
        "val_loss": history.history['val_loss'][0],
        "train_rmse": history.history['rmse_metric'][0],
        "val_rmse": history.history['val_rmse_metric'][0],
    })

model_interp_hyper6.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
51/55 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.2476 - rmse_metric: 0.2476
Epoch 1: val_loss improved from inf to 0.26817, saving model to /content/drive/MyDrive/Time Series Project/Stock Price/Model/model_path/int_hyper6.keras
55/55 ━━━━━━━━━━━━━━━━━━━━ 4s 16ms/step - loss: 0.2387 - rmse_metric: 0.2387 - val_loss: 0.2682 - val_rmse_metric: 0.2693
Restoring model weights from the end of the best epoch: 1.
Epoch 2/50
48/55 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0562 - rmse_metric: 0.0562
Epoch 1: val_loss improved from 0.26817 to 0.24676, saving model to /content/drive/MyDrive/Time Series Project/Stock Price/Model/model_path/int_hyper6.keras
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 0.0555 - rmse_metric: 0.0555 - val_loss: 0.2468 - val_rmse_metric: 0.2477
Restoring model weights from the end of the best epoch: 1.
Epoch 3/50
50/55 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0441 - rmse_metric: 0.0441
Epoch 1: val_loss improved from 0.24676 to 0.16777, saving model to

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv1d_6 (Conv1D)                    │ (None, 24, 64)              │          15,744 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_6 (MaxPooling1D)       │ (None, 12, 64)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_7 (Conv1D)                    │ (None, 6, 64)               │          28,736 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_7 (MaxPooling1D)       │ (None, 3, 64)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_9 (LSTM)                        │ (None, 3, 64)               │          33,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_10 (LSTM)                       │ (None, 3, 64)               │          33,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_11 (LSTM)                       │ (None, 64)                  │          33,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_3 (Flatten)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 128)                 │           8,320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 456,005 (1.74 MB)

 Trainable params: 152,001 (593.75 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 304,004 (1.16 MB)

##### 7

In [ ]:
wandb.finish()

wandb.init(project="Stock Price CNN-LSTM - Split, Same", name="hyper7", config={
    "epochs": 50,
    "batch_size": 64,
    "time_steps": 30,
    "lstm_units": [64, 64, 64],
    "optimizer": "adam",
    "loss": "rmse",
},reinit=True)

epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train_loss,█▃▂▂▂▂▂▂▂▂▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_rmse,█▆▅▄▄▄▃▃▄▃▃▃▃▃▂▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▂▁▁▁▁▁
val_loss,█▇▃▂▃▁▃▂▃▂▁▂▁▁▂▁▂▂▂▁▂▁▁▁▁▂▂▂▂▃▂▂▂▃▄▂▃▃▃▃
val_rmse,█▂▃▂▃▁▃▂▃▂▁▂▁▁▂▁▂▂▂▁▂▁▁▁▁▂▂▂▂▃▂▂▃▄▂▃▃▃▃▃
epoch,50
train_loss,0.01993
train_rmse,0.01994
val_loss,0.18211
val_rmse,0.18328


In [ ]:
model_checkpoint = ModelCheckpoint(
    '/content/drive/MyDrive/Time Series Project/Stock Price/Model/model_path/int_hyper7.keras',
    monitor='val_loss',
    save_best_only=True,
    mode='min',
    verbose=1
)

time_steps = wandb.config.time_steps
X_train, y_train = create_sequences(x_train_int_df, y_train_int_df, time_steps)
X_val, y_val = create_sequences(x_val_int_df, y_val_int_df, time_steps)

model_interp_hyper7 = tf.keras.Sequential([
    tf.keras.layers.Conv1D(64, kernel_size=9, activation='relu', input_shape=(time_steps, len(features))),
    tf.keras.layers.MaxPooling1D(2),

    tf.keras.layers.Conv1D(64, kernel_size=9, activation='relu'),
    tf.keras.layers.MaxPooling1D(2),

    tf.keras.layers.LSTM(wandb.config.lstm_units[0], activation='tanh', return_sequences=True),
    tf.keras.layers.LSTM(wandb.config.lstm_units[1], activation='tanh', return_sequences=True),
    tf.keras.layers.LSTM(wandb.config.lstm_units[2], activation='tanh'),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(1)
])

model_interp_hyper7.compile(optimizer=wandb.config.optimizer, loss=rmse_metric, metrics=[rmse_metric])

for epoch in range(wandb.config.epochs):
    print(f"Epoch {epoch + 1}/{wandb.config.epochs}")
    history = model_interp_hyper7.fit(
        X_train, y_train,
        epochs=1,
        batch_size=wandb.config.batch_size,
        validation_data=(X_val, y_val),
        callbacks=[early_stopping, model_checkpoint],
        verbose=1,
    )

    wandb.log({
        "epoch": epoch + 1,
        "train_loss": history.history['loss'][0],
        "val_loss": history.history['val_loss'][0],
        "train_rmse": history.history['rmse_metric'][0],
        "val_rmse": history.history['val_rmse_metric'][0],
    })

model_interp_hyper7.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.2940 - rmse_metric: 0.2940
Epoch 1: val_loss improved from inf to 0.28623, saving model to /content/drive/MyDrive/Time Series Project/Stock Price/Model/model_path/int_hyper7.keras
55/55 ━━━━━━━━━━━━━━━━━━━━ 4s 21ms/step - loss: 0.2926 - rmse_metric: 0.2925 - val_loss: 0.2862 - val_rmse_metric: 0.2870
Restoring model weights from the end of the best epoch: 1.
Epoch 2/50
54/55 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.1068 - rmse_metric: 0.1068
Epoch 1: val_loss improved from 0.28623 to 0.22068, saving model to /content/drive/MyDrive/Time Series Project/Stock Price/Model/model_path/int_hyper7.keras
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 0.1059 - rmse_metric: 0.1059 - val_loss: 0.2207 - val_rmse_metric: 0.2219
Restoring model weights from the end of the best epoch: 1.
Epoch 3/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0457 - rmse_metric: 0.0457
Epoch 1: val_loss improved from 0.22068 to 0.20471, saving model t

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv1d_8 (Conv1D)                    │ (None, 22, 64)              │          20,224 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_8 (MaxPooling1D)       │ (None, 11, 64)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_9 (Conv1D)                    │ (None, 3, 64)               │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_9 (MaxPooling1D)       │ (None, 1, 64)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_12 (LSTM)                       │ (None, 1, 64)               │          33,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_13 (LSTM)                       │ (None, 1, 64)               │          33,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_14 (LSTM)                       │ (None, 64)                  │          33,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_4 (Flatten)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (None, 128)                 │           8,320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 494,021 (1.88 MB)

 Trainable params: 164,673 (643.25 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 329,348 (1.26 MB)

##### 8

In [ ]:
wandb.finish()

wandb.init(project="Stock Price CNN-LSTM - Split, Same", name="hyper8", config={
    "epochs": 50,
    "batch_size": 64,
    "time_steps": 60,
    "lstm_units": [64, 64, 64],
    "optimizer": "adam",
    "loss": "rmse",
},reinit=True)

epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
train_loss,█▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_rmse,█▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▄▃▂▃▃▂▃▃▃▂▃▂▂▂▁▁▂▂▂▂▁▂▂▂▁▂▂▂▁▂▁▁▁▂▁▁▂▂▂
val_rmse,█▄▃▂▃▃▂▃▃▃▃▃▂▂▂▁▂▂▂▂▁▂▂▂▂▂▂▂▁▂▁▁▁▂▂▁▁▁▂▂
epoch,50
train_loss,0.01893
train_rmse,0.01895
val_loss,0.17587
val_rmse,0.17677


In [ ]:
model_checkpoint = ModelCheckpoint(
    '/content/drive/MyDrive/Time Series Project/Stock Price/Model/model_path/int_hyper8.keras',
    monitor='val_loss',
    save_best_only=True,
    mode='min',
    verbose=1
)

time_steps = wandb.config.time_steps
X_train, y_train = create_sequences(x_train_int_df, y_train_int_df, time_steps)
X_val, y_val = create_sequences(x_val_int_df, y_val_int_df, time_steps)

model_interp_hyper8 = tf.keras.Sequential([
    tf.keras.layers.Conv1D(64, kernel_size=9, activation='relu', input_shape=(time_steps, len(features))),
    tf.keras.layers.MaxPooling1D(2),

    tf.keras.layers.Conv1D(64, kernel_size=9, activation='relu'),
    tf.keras.layers.MaxPooling1D(2),

    tf.keras.layers.LSTM(wandb.config.lstm_units[0], activation='tanh', return_sequences=True),
    tf.keras.layers.LSTM(wandb.config.lstm_units[1], activation='tanh', return_sequences=True),
    tf.keras.layers.LSTM(wandb.config.lstm_units[2], activation='tanh'),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(1)
])

model_interp_hyper8.compile(optimizer=wandb.config.optimizer, loss=rmse_metric, metrics=[rmse_metric])

for epoch in range(wandb.config.epochs):
    print(f"Epoch {epoch + 1}/{wandb.config.epochs}")
    history = model_interp_hyper8.fit(
        X_train, y_train,
        epochs=1,
        batch_size=wandb.config.batch_size,
        validation_data=(X_val, y_val),
        callbacks=[early_stopping, model_checkpoint],
        verbose=1,
    )

    wandb.log({
        "epoch": epoch + 1,
        "train_loss": history.history['loss'][0],
        "val_loss": history.history['val_loss'][0],
        "train_rmse": history.history['rmse_metric'][0],
        "val_rmse": history.history['val_rmse_metric'][0],
    })

model_interp_hyper8.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.1903 - rmse_metric: 0.1902
Epoch 1: val_loss improved from inf to 0.23058, saving model to /content/drive/MyDrive/Time Series Project/Stock Price/Model/model_path/int_hyper8.keras
55/55 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - loss: 0.1889 - rmse_metric: 0.1889 - val_loss: 0.2306 - val_rmse_metric: 0.2357
Restoring model weights from the end of the best epoch: 1.
Epoch 2/50
54/55 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0556 - rmse_metric: 0.0556
Epoch 1: val_loss improved from 0.23058 to 0.17917, saving model to /content/drive/MyDrive/Time Series Project/Stock Price/Model/model_path/int_hyper8.keras
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.0554 - rmse_metric: 0.0554 - val_loss: 0.1792 - val_rmse_metric: 0.1826
Restoring model weights from the end of the best epoch: 1.
Epoch 3/50
53/55 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0407 - rmse_metric: 0.0407
Epoch 1: val_loss did not improve from 0.17917
55/55 ━━━━━━━━━━━━━

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv1d_10 (Conv1D)                   │ (None, 52, 64)              │          20,224 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_10 (MaxPooling1D)      │ (None, 26, 64)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_11 (Conv1D)                   │ (None, 18, 64)              │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_11 (MaxPooling1D)      │ (None, 9, 64)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_15 (LSTM)                       │ (None, 9, 64)               │          33,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_16 (LSTM)                       │ (None, 9, 64)               │          33,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_17 (LSTM)                       │ (None, 64)                  │          33,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_5 (Flatten)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_10 (Dense)                     │ (None, 128)                 │           8,320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_11 (Dense)                     │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 494,021 (1.88 MB)

 Trainable params: 164,673 (643.25 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 329,348 (1.26 MB)

##### 9

In [ ]:
wandb.finish()

wandb.init(project="Stock Price CNN-LSTM - Split, Same", name="hyper9", config={
    "epochs": 50,
    "batch_size": 64,
    "time_steps": 30,
    "lstm_units": [128,128,128],
    "optimizer": "adam",
    "loss": "rmse",
},reinit=True)

epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
train_loss,█▃▂▃▂▂▂▂▂▂▂▂▂▂▁▂▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_rmse,█▃▂▃▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,▇▃█▅▄▂▃▂▂▄▄▁▅▁▄▄▂▅▅▆▅▅▄▅▄▅▄▅▅▅▅▆▅▅▅▆▅▅▅█
val_rmse,▇▃█▅▄▃▂▃▂▂▄▄▁▆▅▄▅▄▂▅▆▄▅▅▅▄▅▅▅▅▅▄▆▅▅▆▆▅▅█
epoch,50
train_loss,0.02031
train_rmse,0.02028
val_loss,0.24103
val_rmse,0.24581


In [ ]:
model_checkpoint = ModelCheckpoint(
    '/content/drive/MyDrive/Time Series Project/Stock Price/Model/model_path/int_hyper9.keras',
    monitor='val_loss',
    save_best_only=True,
    mode='min',
    verbose=1
)

time_steps = wandb.config.time_steps
X_train, y_train = create_sequences(x_train_int_df, y_train_int_df, time_steps)
X_val, y_val = create_sequences(x_val_int_df, y_val_int_df, time_steps)

model_interp_hyper9 = tf.keras.Sequential([
    tf.keras.layers.Conv1D(64, kernel_size=9, activation='relu', input_shape=(time_steps, len(features))),
    tf.keras.layers.MaxPooling1D(2),

    tf.keras.layers.Conv1D(64, kernel_size=9, activation='relu'),
    tf.keras.layers.MaxPooling1D(2),

    tf.keras.layers.LSTM(wandb.config.lstm_units[0], activation='tanh', return_sequences=True),
    tf.keras.layers.LSTM(wandb.config.lstm_units[1], activation='tanh', return_sequences=True),
    tf.keras.layers.LSTM(wandb.config.lstm_units[2], activation='tanh'),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(1)
])

model_interp_hyper9.compile(optimizer=wandb.config.optimizer, loss=rmse_metric, metrics=[rmse_metric])

for epoch in range(wandb.config.epochs):
    print(f"Epoch {epoch + 1}/{wandb.config.epochs}")
    history = model_interp_hyper9.fit(
        X_train, y_train,
        epochs=1,
        batch_size=wandb.config.batch_size,
        validation_data=(X_val, y_val),
        callbacks=[early_stopping, model_checkpoint],
        verbose=1,
    )

    wandb.log({
        "epoch": epoch + 1,
        "train_loss": history.history['loss'][0],
        "val_loss": history.history['val_loss'][0],
        "train_rmse": history.history['rmse_metric'][0],
        "val_rmse": history.history['val_rmse_metric'][0],
    })

model_interp_hyper9.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
52/55 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.2688 - rmse_metric: 0.2688
Epoch 1: val_loss improved from inf to 0.27288, saving model to /content/drive/MyDrive/Time Series Project/Stock Price/Model/model_path/int_hyper9.keras
55/55 ━━━━━━━━━━━━━━━━━━━━ 5s 22ms/step - loss: 0.2620 - rmse_metric: 0.2619 - val_loss: 0.2729 - val_rmse_metric: 0.2743
Restoring model weights from the end of the best epoch: 1.
Epoch 2/50
54/55 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0663 - rmse_metric: 0.0663
Epoch 1: val_loss improved from 0.27288 to 0.19533, saving model to /content/drive/MyDrive/Time Series Project/Stock Price/Model/model_path/int_hyper9.keras
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - loss: 0.0661 - rmse_metric: 0.0661 - val_loss: 0.1953 - val_rmse_metric: 0.1964
Restoring model weights from the end of the best epoch: 1.
Epoch 3/50
52/55 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 0.0478 - rmse_metric: 0.0478
Epoch 1: val_loss did not improve from 0.19533
55/55 ━━━━━━━━━━━━

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv1d_12 (Conv1D)                   │ (None, 22, 64)              │          20,224 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_12 (MaxPooling1D)      │ (None, 11, 64)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_13 (Conv1D)                   │ (None, 3, 64)               │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_13 (MaxPooling1D)      │ (None, 1, 64)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_18 (LSTM)                       │ (None, 1, 128)              │          98,816 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_19 (LSTM)                       │ (None, 1, 128)              │         131,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_20 (LSTM)                       │ (None, 128)                 │         131,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_6 (Flatten)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_12 (Dense)                     │ (None, 128)                 │          16,512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_5 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_13 (Dense)                     │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,307,333 (4.99 MB)

 Trainable params: 435,777 (1.66 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 871,556 (3.32 MB)

##### 10

In [ ]:
wandb.finish()

wandb.init(project="Stock Price CNN-LSTM - Split, Same", name="hyper10", config={
    "epochs": 50,
    "batch_size": 64,
    "time_steps": 30,
    "lstm_units": [64,64,64],
    "optimizer": "adam",
    "loss": "rmse",
},reinit=True)

epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
train_loss,█▃▂▂▂▂▂▂▂▁▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_rmse,█▃▂▂▂▂▂▂▂▁▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▃▄▂▂▂▂▁▂▂▂▂▂▁▃▂▂▂▂▂▁▂▂▂▂▁▂▃▂▂▂▂▂▂▁▂▂▂▂▂
val_rmse,█▃▄▂▂▁▂▂▂▂▂▂▂▁▃▂▂▂▂▂▁▂▂▂▂▂▁▂▃▂▂▂▂▂▁▂▂▂▂▂
epoch,50
train_loss,0.01939
train_rmse,0.01937
val_loss,0.17706
val_rmse,0.17814


In [ ]:
model_checkpoint = ModelCheckpoint(
    '/content/drive/MyDrive/Time Series Project/Stock Price/Model/model_path/int_hyper10.keras',
    monitor='val_loss',
    save_best_only=True,
    mode='min',
    verbose=1
)

time_steps = wandb.config.time_steps
X_train, y_train = create_sequences(x_train_int_df, y_train_int_df, time_steps)
X_val, y_val = create_sequences(x_val_int_df, y_val_int_df, time_steps)

model_interp_hyper10 = tf.keras.Sequential([
    tf.keras.layers.Conv1D(64, kernel_size=9, activation='relu', input_shape=(time_steps, len(features))),
    tf.keras.layers.MaxPooling1D(2),

    tf.keras.layers.Conv1D(64, kernel_size=9, activation='relu'),
    tf.keras.layers.MaxPooling1D(2),

    tf.keras.layers.LSTM(wandb.config.lstm_units[0], activation='tanh', return_sequences=True),
    tf.keras.layers.LSTM(wandb.config.lstm_units[1], activation='tanh', return_sequences=True),
    tf.keras.layers.LSTM(wandb.config.lstm_units[2], activation='tanh'),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(1)
])

model_interp_hyper10.compile(optimizer=wandb.config.optimizer, loss=rmse_metric, metrics=[rmse_metric])

for epoch in range(wandb.config.epochs):
    print(f"Epoch {epoch + 1}/{wandb.config.epochs}")
    history = model_interp_hyper10.fit(
        X_train, y_train,
        epochs=1,
        batch_size=wandb.config.batch_size,
        validation_data=(X_val, y_val),
        callbacks=[early_stopping, model_checkpoint],
        verbose=1,
    )

    wandb.log({
        "epoch": epoch + 1,
        "train_loss": history.history['loss'][0],
        "val_loss": history.history['val_loss'][0],
        "train_rmse": history.history['rmse_metric'][0],
        "val_rmse": history.history['val_rmse_metric'][0],
    })

model_interp_hyper10.summary()

Epoch 1/50
53/55 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.2854 - rmse_metric: 0.2854
Epoch 1: val_loss improved from inf to 0.30455, saving model to /content/drive/MyDrive/Time Series Project/Stock Price/Model/model_path/int_hyper10.keras
55/55 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - loss: 0.2807 - rmse_metric: 0.2807 - val_loss: 0.3046 - val_rmse_metric: 0.3058
Restoring model weights from the end of the best epoch: 1.
Epoch 2/50
51/55 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0788 - rmse_metric: 0.0788
Epoch 1: val_loss improved from 0.30455 to 0.26064, saving model to /content/drive/MyDrive/Time Series Project/Stock Price/Model/model_path/int_hyper10.keras
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 0.0774 - rmse_metric: 0.0774 - val_loss: 0.2606 - val_rmse_metric: 0.2619
Restoring model weights from the end of the best epoch: 1.
Epoch 3/50
50/55 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0475 - rmse_metric: 0.0475
Epoch 1: val_loss improved from 0.26064 to 0.22691, saving model 

Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv1d_16 (Conv1D)                   │ (None, 22, 64)              │          20,224 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_16 (MaxPooling1D)      │ (None, 11, 64)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_17 (Conv1D)                   │ (None, 3, 64)               │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_17 (MaxPooling1D)      │ (None, 1, 64)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_24 (LSTM)                       │ (None, 1, 64)               │          33,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_25 (LSTM)                       │ (None, 1, 64)               │          33,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_26 (LSTM)                       │ (None, 64)                  │          33,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_8 (Flatten)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_16 (Dense)                     │ (None, 64)                  │           4,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_7 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_17 (Dense)                     │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 481,349 (1.84 MB)

 Trainable params: 160,449 (626.75 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 320,900 (1.22 MB)

##### 11

In [ ]:
wandb.finish()

wandb.init(project="Stock Price CNN-LSTM - Split, Same", name="hyper11", config={
    "epochs": 50,
    "batch_size": 64,
    "time_steps": 30,
    "lstm_units": [64,64,64],
    "optimizer": "adam",
    "loss": "rmse",
},reinit=True)

epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
train_loss,█▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_rmse,█▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▆▅▄▄▃▃▃▂▂▃▂▂▁▂▁▂▂▁▁▁▂▁▂▁▁▂▂▁▁▁▂▂▁▂▁▁▁▁▂
val_rmse,█▆▅▄▄▃▃▃▃▂▂▃▂▂▁▁▁▂▂▁▁▂▁▂▁▁▂▂▁▁▁▂▂▁▂▁▁▁▁▂
epoch,50
train_loss,0.02088
train_rmse,0.02089
val_loss,0.15099
val_rmse,0.15182


In [ ]:
model_checkpoint = ModelCheckpoint(
    '/content/drive/MyDrive/Time Series Project/Stock Price/Model/model_path/int_hyper11.keras',
    monitor='val_loss',
    save_best_only=True,
    mode='min',
    verbose=1
)

time_steps = wandb.config.time_steps
X_train, y_train = create_sequences(x_train_int_df, y_train_int_df, time_steps)
X_val, y_val = create_sequences(x_val_int_df, y_val_int_df, time_steps)

model_interp_hyper11 = tf.keras.Sequential([
    tf.keras.layers.Conv1D(64, kernel_size=9, activation='relu', input_shape=(time_steps, len(features))),
    tf.keras.layers.MaxPooling1D(2),

    tf.keras.layers.Conv1D(64, kernel_size=9, activation='relu'),
    tf.keras.layers.MaxPooling1D(2),

    Bidirectional(tf.keras.layers.LSTM(wandb.config.lstm_units[0], activation='tanh', return_sequences=True)),
    Bidirectional(tf.keras.layers.LSTM(wandb.config.lstm_units[1], activation='tanh', return_sequences=True)),
    Bidirectional(tf.keras.layers.LSTM(wandb.config.lstm_units[2], activation='tanh')),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(1)
])

model_interp_hyper11.compile(optimizer=wandb.config.optimizer, loss=rmse_metric, metrics=[rmse_metric])

for epoch in range(wandb.config.epochs):
    print(f"Epoch {epoch + 1}/{wandb.config.epochs}")
    history = model_interp_hyper11.fit(
        X_train, y_train,
        epochs=1,
        batch_size=wandb.config.batch_size,
        validation_data=(X_val, y_val),
        callbacks=[early_stopping, model_checkpoint],
        verbose=1,
    )

    wandb.log({
        "epoch": epoch + 1,
        "train_loss": history.history['loss'][0],
        "val_loss": history.history['val_loss'][0],
        "train_rmse": history.history['rmse_metric'][0],
        "val_rmse": history.history['val_rmse_metric'][0],
    })

model_interp_hyper11.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
51/55 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.2547 - rmse_metric: 0.2547
Epoch 1: val_loss improved from inf to 0.25389, saving model to /content/drive/MyDrive/Time Series Project/Stock Price/Model/model_path/int_hyper11.keras
55/55 ━━━━━━━━━━━━━━━━━━━━ 6s 26ms/step - loss: 0.2463 - rmse_metric: 0.2462 - val_loss: 0.2539 - val_rmse_metric: 0.2551
Restoring model weights from the end of the best epoch: 1.
Epoch 2/50
54/55 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.0586 - rmse_metric: 0.0586
Epoch 1: val_loss improved from 0.25389 to 0.19775, saving model to /content/drive/MyDrive/Time Series Project/Stock Price/Model/model_path/int_hyper11.keras
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - loss: 0.0586 - rmse_metric: 0.0586 - val_loss: 0.1977 - val_rmse_metric: 0.1988
Restoring model weights from the end of the best epoch: 1.
Epoch 3/50
52/55 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0441 - rmse_metric: 0.0441
Epoch 1: val_loss did not improve from 0.19775
55/55 ━━━━━━━━━

Model: "sequential_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv1d_18 (Conv1D)                   │ (None, 22, 64)              │          20,224 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_18 (MaxPooling1D)      │ (None, 11, 64)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_19 (Conv1D)                   │ (None, 3, 64)               │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_19 (MaxPooling1D)      │ (None, 1, 64)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional (Bidirectional)        │ (None, 1, 128)              │          66,048 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_1 (Bidirectional)      │ (None, 1, 128)              │          98,816 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_2 (Bidirectional)      │ (None, 128)                 │          98,816 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_9 (Flatten)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_18 (Dense)                     │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_8 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_19 (Dense)                     │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 987,461 (3.77 MB)

 Trainable params: 329,153 (1.26 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 658,308 (2.51 MB)

##### 12

In [ ]:
wandb.finish()

wandb.init(project="Stock Price CNN-LSTM - Split, Same", name="hyper12", config={
    "epochs": 50,
    "batch_size": 64,
    "time_steps": 30,
    "lstm_units": [64,64,64],
    "optimizer": "adam",
    "loss": "rmse",
},reinit=True)

epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
train_loss,█▃▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_rmse,█▃▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▅▆▄▃▃▂▂▂▂▁▁▂▂▁▂▂▂▂▃▁▁▁▁▁▂▂▂▁▁▂▂▂▂▂▂▁▂▂▂
val_rmse,█▅▆▄▃▂▂▂▂▂▁▁▂▂▁▂▂▂▂▃▁▁▁▁▁▂▂▂▂▁▂▂▂▂▂▂▁▂▂▂
epoch,50
train_loss,0.02217
train_rmse,0.02214
val_loss,0.1426
val_rmse,0.14327


In [ ]:
model_checkpoint = ModelCheckpoint(
    '/content/drive/MyDrive/Time Series Project/Stock Price/Model/model_path/int_hyper12.keras',
    monitor='val_loss',
    save_best_only=True,
    mode='min',
    verbose=1
)

time_steps = wandb.config.time_steps
X_train, y_train = create_sequences(x_train_int_df, y_train_int_df, time_steps)
X_val, y_val = create_sequences(x_val_int_df, y_val_int_df, time_steps)

model_interp_hyper12 = tf.keras.Sequential([
    tf.keras.layers.Conv1D(64, kernel_size=9, activation='relu', input_shape=(time_steps, len(features))),
    tf.keras.layers.MaxPooling1D(2),

    tf.keras.layers.Conv1D(64, kernel_size=9, activation='relu'),
    tf.keras.layers.MaxPooling1D(2),

    Bidirectional(tf.keras.layers.LSTM(wandb.config.lstm_units[0], activation='tanh', return_sequences=True)),
    Bidirectional(tf.keras.layers.LSTM(wandb.config.lstm_units[1], activation='tanh', return_sequences=True)),
    Bidirectional(tf.keras.layers.LSTM(wandb.config.lstm_units[2], activation='tanh')),
    Attention(),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(1)
])

model_interp_hyper12.compile(optimizer=wandb.config.optimizer, loss=rmse_metric, metrics=[rmse_metric])

for epoch in range(wandb.config.epochs):
    print(f"Epoch {epoch + 1}/{wandb.config.epochs}")
    history = model_interp_hyper12.fit(
        X_train, y_train,
        epochs=1,
        batch_size=wandb.config.batch_size,
        validation_data=(X_val, y_val),
        callbacks=[early_stopping, model_checkpoint],
        verbose=1,
    )

    wandb.log({
        "epoch": epoch + 1,
        "train_loss": history.history['loss'][0],
        "val_loss": history.history['val_loss'][0],
        "train_rmse": history.history['rmse_metric'][0],
        "val_rmse": history.history['val_rmse_metric'][0],
    })

model_interp_hyper12.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
52/55 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.3931 - rmse_metric: 0.3931
Epoch 1: val_loss improved from inf to 0.28346, saving model to /content/drive/MyDrive/Time Series Project/Stock Price/Model/model_path/int_hyper12.keras
55/55 ━━━━━━━━━━━━━━━━━━━━ 8s 29ms/step - loss: 0.3860 - rmse_metric: 0.3860 - val_loss: 0.2835 - val_rmse_metric: 0.2841
Restoring model weights from the end of the best epoch: 1.
Epoch 2/50
53/55 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.1617 - rmse_metric: 0.1617
Epoch 1: val_loss did not improve from 0.28346
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 0.1618 - rmse_metric: 0.1618 - val_loss: 0.3131 - val_rmse_metric: 0.3138
Restoring model weights from the end of the best epoch: 1.
Epoch 3/50
53/55 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 0.1640 - rmse_metric: 0.1640
Epoch 1: val_loss did not improve from 0.28346
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - loss: 0.1639 - rmse_metric: 0.1638 - val_loss: 0.3210 - val_rmse_metric: 0.3217
Rest

Model: "sequential_10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv1d_20 (Conv1D)                   │ (None, 22, 64)              │          20,224 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_20 (MaxPooling1D)      │ (None, 11, 64)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_21 (Conv1D)                   │ (None, 3, 64)               │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_21 (MaxPooling1D)      │ (None, 1, 64)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_3 (Bidirectional)      │ (None, 1, 128)              │          66,048 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_4 (Bidirectional)      │ (None, 1, 128)              │          98,816 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_5 (Bidirectional)      │ (None, 128)                 │          98,816 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ attention (Attention)                │ (None)                      │          16,512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_10 (Flatten)                 │ (None, 1)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_20 (Dense)                     │ (None, 64)                  │             128 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_9 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_21 (Dense)                     │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,012,613 (3.86 MB)

 Trainable params: 337,537 (1.29 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 675,076 (2.58 MB)

##### 13

In [ ]:
wandb.finish()

wandb.init(project="Stock Price CNN-LSTM - Split, Same", name="hyper13", config={
    "epochs": 50,
    "batch_size": 64,
    "time_steps": 30,
    "lstm_units": [64,64,64],
    "optimizer": "adam",
    "loss": "rmse",
},reinit=True)

epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
train_loss,██▆▃▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_rmse,█▅▅▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,▆██▄▅▃▂▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▃▂▁▂▁▂▂▂▁▂▁▂▁▂▁▂▁▁
val_rmse,▆██▄▅▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▃▁▂▁▂▂▂▂▁▂▁▂▁▂▁▂▁▁
epoch,50
train_loss,0.02097
train_rmse,0.02097
val_loss,0.1998
val_rmse,0.20092


In [ ]:
model_checkpoint = ModelCheckpoint(
    '/content/drive/MyDrive/Time Series Project/Stock Price/Model/model_path/int_hyper13.keras',
    monitor='val_loss',
    save_best_only=True,
    mode='min',
    verbose=1
)

time_steps = wandb.config.time_steps
X_train, y_train = create_sequences(x_train_int_df, y_train_int_df, time_steps)
X_val, y_val = create_sequences(x_val_int_df, y_val_int_df, time_steps)

model_interp_hyper13 = tf.keras.Sequential([
    tf.keras.layers.Conv1D(64, kernel_size=7, activation='relu', input_shape=(time_steps, len(features))),
    tf.keras.layers.MaxPooling1D(2),

    tf.keras.layers.Conv1D(64, kernel_size=7, activation='relu'),
    tf.keras.layers.MaxPooling1D(2),

    Bidirectional(tf.keras.layers.LSTM(wandb.config.lstm_units[0], activation='tanh', return_sequences=True)),
    Bidirectional(tf.keras.layers.LSTM(wandb.config.lstm_units[1], activation='tanh')),
    Attention(),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(1)
])

model_interp_hyper13.compile(optimizer=wandb.config.optimizer, loss=rmse_metric, metrics=[rmse_metric])

for epoch in range(wandb.config.epochs):
    print(f"Epoch {epoch + 1}/{wandb.config.epochs}")
    history = model_interp_hyper13.fit(
        X_train, y_train,
        epochs=1,
        batch_size=wandb.config.batch_size,
        validation_data=(X_val, y_val),
        callbacks=[early_stopping, model_checkpoint],
        verbose=1,
    )

    wandb.log({
        "epoch": epoch + 1,
        "train_loss": history.history['loss'][0],
        "val_loss": history.history['val_loss'][0],
        "train_rmse": history.history['rmse_metric'][0],
        "val_rmse": history.history['val_rmse_metric'][0],
    })

model_interp_hyper13.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
52/55 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.4001 - rmse_metric: 0.4001
Epoch 1: val_loss improved from inf to 0.33138, saving model to /content/drive/MyDrive/Time Series Project/Stock Price/Model/model_path/int_hyper13.keras
55/55 ━━━━━━━━━━━━━━━━━━━━ 5s 21ms/step - loss: 0.3940 - rmse_metric: 0.3939 - val_loss: 0.3314 - val_rmse_metric: 0.3321
Restoring model weights from the end of the best epoch: 1.
Epoch 2/50
52/55 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.1226 - rmse_metric: 0.1226
Epoch 1: val_loss improved from 0.33138 to 0.23168, saving model to /content/drive/MyDrive/Time Series Project/Stock Price/Model/model_path/int_hyper13.keras
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 0.1207 - rmse_metric: 0.1207 - val_loss: 0.2317 - val_rmse_metric: 0.2330
Restoring model weights from the end of the best epoch: 1.
Epoch 3/50
52/55 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0506 - rmse_metric: 0.0506
Epoch 1: val_loss did not improve from 0.23168
55/55 ━━━━━━━━━

Model: "sequential_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv1d_22 (Conv1D)                   │ (None, 24, 64)              │          15,744 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_22 (MaxPooling1D)      │ (None, 12, 64)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_23 (Conv1D)                   │ (None, 6, 64)               │          28,736 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_23 (MaxPooling1D)      │ (None, 3, 64)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_6 (Bidirectional)      │ (None, 3, 128)              │          66,048 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_7 (Bidirectional)      │ (None, 128)                 │          98,816 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ attention_1 (Attention)              │ (None)                      │          16,512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_11 (Flatten)                 │ (None, 1)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_22 (Dense)                     │ (None, 64)                  │             128 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_10 (Dropout)                 │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_23 (Dense)                     │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 678,149 (2.59 MB)

 Trainable params: 226,049 (883.00 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 452,100 (1.72 MB)

##### 14

In [ ]:
wandb.init(project="Stock Price CNN-LSTM - Split, Same", name="hyper14", config={
    "epochs": 50,
    "batch_size": 64,
    "time_steps": 30,
    "lstm_units": [64,64,64],
    "optimizer": "adam",
    "loss": "rmse",
},reinit=True)

wandb: Currently logged in as: yebin. Use `wandb login --relogin` to force relogin


In [ ]:
# Skip Connection Model with Multi-Head Attention
def cnn_lstm_attention_skip_model(time_steps, num_features, lstm_units, num_heads, key_dim):
    # Input Layer
    inputs = Input(shape=(time_steps, num_features), name='inputs')

    # CNN Feature Extraction
    x = Conv1D(64, kernel_size=9, activation='relu', padding='same')(inputs)
    x = MaxPooling1D(pool_size=2)(x)
    x = Conv1D(64, kernel_size=9, activation='relu', padding='same')(x)
    x = MaxPooling1D(pool_size=2)(x)

    # LSTM Feature Extraction
    lstm_out = Bidirectional(LSTM(lstm_units[0], return_sequences=True))(x)
    lstm_out = Bidirectional(LSTM(lstm_units[1], return_sequences=True))(lstm_out)
    lstm_out = Bidirectional(LSTM(lstm_units[2], return_sequences=True))(lstm_out)

    # Multi-Head Attention
    attention = MultiHeadAttentionLayer(num_heads=num_heads, key_dim=key_dim)(lstm_out, lstm_out)

    # Flatten LSTM and Attention Outputs
    lstm_flatten = Flatten()(attention)

    # Skip Connection (Flatten input)
    skip_flatten = Flatten()(inputs)

    # Concatenate LSTM-Attention Features and Raw Input
    concat = Concatenate(axis=-1)([lstm_flatten, skip_flatten])

    # Fully Connected Layers
    x = Dense(128, activation='relu')(concat)
    x = Dropout(0.3)(x)
    x = Dense(64, activation='relu')(x)
    x = Dropout(0.3)(x)

    # Output Layer
    output = Dense(1, name='output')(x)

    # Define Model
    model = Model(inputs=inputs, outputs=output, name='cnn_lstm_attention_skip')

    return model

In [ ]:
# Model Parameters
time_steps = wandb.config.time_steps
num_features = len(features)
lstm_units = wandb.config.lstm_units
num_heads = 8  # Number of attention heads
key_dim = 64   # Dimensionality of the attention layer

X_train, y_train = create_sequences(x_train_int_df, y_train_int_df, time_steps)
X_val, y_val = create_sequences(x_val_int_df, y_val_int_df, time_steps)

# Build Model
model_interp_hyper14 = cnn_lstm_attention_skip_model(time_steps, num_features, lstm_units, num_heads, key_dim)

# Compile Model
model_interp_hyper14.compile(
    optimizer=wandb.config.optimizer,
    loss=rmse_metric,
    metrics=[rmse_metric]
)

# Callbacks
model_checkpoint = ModelCheckpoint(
    '/content/drive/MyDrive/Time Series Project/Stock Price/Model/model_path/int_hyper14.keras',
    monitor='val_loss',
    save_best_only=True,
    mode='min',
    verbose=1
)

# Train Model
for epoch in range(wandb.config.epochs):
    print(f"Epoch {epoch + 1}/{wandb.config.epochs}")
    history = model_interp_hyper14.fit(
        X_train, y_train,
        epochs=1,
        batch_size=wandb.config.batch_size,
        validation_data=(X_val, y_val),
        callbacks=[early_stopping, model_checkpoint],
        verbose=1,
    )

    # Log Metrics to W&B
    wandb.log({
        "epoch": epoch + 1,
        "train_loss": history.history['loss'][0],
        "val_loss": history.history['val_loss'][0],
        "train_rmse": history.history['rmse_metric'][0],
        "val_rmse": history.history['rmse_metric'][0],
    })

# Model Summary
model_interp_hyper14.summary()

Epoch 1/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 0.5427 - rmse_metric: 0.5427
Epoch 1: val_loss improved from inf to 0.21404, saving model to /content/drive/MyDrive/Time Series Project/Stock Price/Model/model_path/int_hyper14.keras
55/55 ━━━━━━━━━━━━━━━━━━━━ 18s 60ms/step - loss: 0.5385 - rmse_metric: 0.5384 - val_loss: 0.2140 - val_rmse_metric: 0.2149
Restoring model weights from the end of the best epoch: 1.
Epoch 2/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.1297 - rmse_metric: 0.1297
Epoch 1: val_loss did not improve from 0.21404
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 0.1295 - rmse_metric: 0.1295 - val_loss: 0.2895 - val_rmse_metric: 0.2900
Restoring model weights from the end of the best epoch: 1.
Epoch 3/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 0.1088 - rmse_metric: 0.1088
Epoch 1: val_loss improved from 0.21404 to 0.20593, saving model to /content/drive/MyDrive/Time Series Project/Stock Price/Model/model_path/int_hyper14.keras
55/55 ━━━━━━━━

Model: "cnn_lstm_attention_skip"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ inputs (InputLayer)       │ (None, 30, 35)         │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_2 (Conv1D)         │ (None, 30, 64)         │         20,224 │ inputs[0][0]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling1d_2           │ (None, 15, 64)         │              0 │ conv1d_2[0][0]         │
│ (MaxPooling1D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_3 (Conv1D)         │ (None, 15, 64)         │         36,928 │ max_pooling1d_2[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling1d_3           │ (None, 7, 64)          │              0 │ conv1d_3[0][0]         │
│ (MaxPooling1D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bidirectional_3           │ (None, 7, 128)         │         66,048 │ max_pooling1d_3[0][0]  │
│ (Bidirectional)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bidirectional_4           │ (None, 7, 128)         │         98,816 │ bidirectional_3[0][0]  │
│ (Bidirectional)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bidirectional_5           │ (None, 7, 128)         │         98,816 │ bidirectional_4[0][0]  │
│ (Bidirectional)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ multi_head_attention_lay… │ (None, 7, 128)         │        263,808 │ bidirectional_5[0][0], │
│ (MultiHeadAttentionLayer) │                        │                │ bidirectional_5[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ flatten_2 (Flatten)       │ (None, 896)            │              0 │ multi_head_attention_… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ flatten_3 (Flatten)       │ (None, 1050)           │              0 │ inputs[0][0]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_1             │ (None, 1946)           │              0 │ flatten_2[0][0],       │
│ (Concatenate)             │                        │                │ flatten_3[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_2 (Dense)           │ (None, 128)            │        249,216 │ concatenate_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_4 (Dropout)       │ (None, 128)            │              0 │ dense_2[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_3 (Dense)           │ (None, 64)             │          8,256 │ dropout_4[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_5 (Dropout)       │ (None, 64)             │              0 │ dense_3[0][0]          │
├──────────────────────

 Total params: 2,526,533 (9.64 MB)

 Trainable params: 842,177 (3.21 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 1,684,356 (6.43 MB)

##### 15

In [ ]:
wandb.finish()

epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇███
train_loss,█▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_rmse,█▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,▃█▂▂▁▂▃▃▅▅▄▆▇▃▅▆▅▅▆▄▅▆▆▆▆▆▆▆▅▇▆▅▇▆▅▆▇▆▇▆
val_rmse,█▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,50
train_loss,0.06352
train_rmse,0.06338
val_loss,0.25934
val_rmse,0.06338


In [ ]:
wandb.init(project="Stock Price CNN-LSTM - Split, Same", name="hyper15", config={
    "epochs": 50,
    "batch_size": 64,
    "time_steps": 30,
    "lstm_units": [128,64,32],
    "optimizer": "adam",
    "loss": "rmse",
},reinit=True)

In [ ]:
# Skip Connection Model with Multi-Head Attention
def cnn_lstm_attention_skip_model(time_steps, num_features, lstm_units, num_heads, key_dim):
    # Input Layer
    inputs = Input(shape=(time_steps, num_features), name='inputs')

    # CNN Feature Extraction
    x = Conv1D(64, kernel_size=7, activation='relu', padding='same')(inputs)
    x = MaxPooling1D(pool_size=2)(x)
    x = Conv1D(64, kernel_size=7, activation='relu', padding='same')(x)
    x = MaxPooling1D(pool_size=2)(x)

    # LSTM Feature Extraction
    lstm_out = Bidirectional(LSTM(lstm_units[0], return_sequences=True))(x)
    lstm_out = Bidirectional(LSTM(lstm_units[1], return_sequences=True))(lstm_out)
    lstm_out = Bidirectional(LSTM(lstm_units[2], return_sequences=True))(lstm_out)

    # Multi-Head Attention
    attention = MultiHeadAttentionLayer(num_heads=num_heads, key_dim=key_dim)(lstm_out, lstm_out)

    # Flatten LSTM and Attention Outputs
    lstm_flatten = Flatten()(attention)

    # Skip Connection (Flatten input)
    skip_flatten = Flatten()(inputs)

    # Concatenate LSTM-Attention Features and Raw Input
    concat = Concatenate(axis=-1)([lstm_flatten, skip_flatten])

    # Fully Connected Layers
    x = Dense(64, activation='relu')(concat)
    x = Dropout(0.1)(x)
    x = Dense(32, activation='relu')(x)
    x = Dropout(0.1)(x)

    # Output Layer
    output = Dense(1, name='output')(x)

    # Define Model
    model = Model(inputs=inputs, outputs=output, name='cnn_lstm_attention_skip')

    return model

In [ ]:
# Model Parameters
time_steps = wandb.config.time_steps
num_features = len(features)
lstm_units = wandb.config.lstm_units
num_heads = 8  # Number of attention heads
key_dim = 64   # Dimensionality of the attention layer

X_train, y_train = create_sequences(x_train_int_df, y_train_int_df, time_steps)
X_val, y_val = create_sequences(x_val_int_df, y_val_int_df, time_steps)

# Build Model
model_interp_hyper15 = cnn_lstm_attention_skip_model(time_steps, num_features, lstm_units, num_heads, key_dim)

# Compile Model
model_interp_hyper15.compile(
    optimizer=wandb.config.optimizer,
    loss=rmse_metric,
    metrics=[rmse_metric]
)

# Callbacks
model_checkpoint = ModelCheckpoint(
    '/content/drive/MyDrive/Time Series Project/Stock Price/Model/model_path/int_hyper15.keras',
    monitor='val_loss',
    save_best_only=True,
    mode='min',
    verbose=1
)

# Train Model
for epoch in range(wandb.config.epochs):
    print(f"Epoch {epoch + 1}/{wandb.config.epochs}")
    history = model_interp_hyper15.fit(
        X_train, y_train,
        epochs=1,
        batch_size=wandb.config.batch_size,
        validation_data=(X_val, y_val),
        callbacks=[early_stopping, model_checkpoint],
        verbose=1,
    )

    # Log Metrics to W&B
    wandb.log({
        "epoch": epoch + 1,
        "train_loss": history.history['loss'][0],
        "val_loss": history.history['val_loss'][0],
        "train_rmse": history.history['rmse_metric'][0],
        "val_rmse": history.history['rmse_metric'][0],
    })

# Model Summary
model_interp_hyper15.summary()

Epoch 1/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 0.3427 - rmse_metric: 0.3427
Epoch 1: val_loss improved from inf to 0.20622, saving model to /content/drive/MyDrive/Time Series Project/Stock Price/Model/model_path/int_hyper15.keras
55/55 ━━━━━━━━━━━━━━━━━━━━ 13s 36ms/step - loss: 0.3402 - rmse_metric: 0.3402 - val_loss: 0.2062 - val_rmse_metric: 0.2077
Restoring model weights from the end of the best epoch: 1.
Epoch 2/50
52/55 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.1040 - rmse_metric: 0.1040
Epoch 1: val_loss did not improve from 0.20622
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 0.1033 - rmse_metric: 0.1033 - val_loss: 0.2928 - val_rmse_metric: 0.2938
Restoring model weights from the end of the best epoch: 1.
Epoch 3/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.0784 - rmse_metric: 0.0784
Epoch 1: val_loss did not improve from 0.20622
55/55 ━━━━━━━━━━━━━━━━━━━━ 2s 29ms/step - loss: 0.0783 - rmse_metric: 0.0783 - val_loss: 0.2353 - val_rmse_metric: 0.2362
Res

Model: "cnn_lstm_attention_skip"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ inputs (InputLayer)       │ (None, 30, 35)         │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_4 (Conv1D)         │ (None, 30, 64)         │         15,744 │ inputs[0][0]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling1d_4           │ (None, 15, 64)         │              0 │ conv1d_4[0][0]         │
│ (MaxPooling1D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_5 (Conv1D)         │ (None, 15, 64)         │         28,736 │ max_pooling1d_4[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling1d_5           │ (None, 7, 64)          │              0 │ conv1d_5[0][0]         │
│ (MaxPooling1D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bidirectional_6           │ (None, 7, 256)         │        197,632 │ max_pooling1d_5[0][0]  │
│ (Bidirectional)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bidirectional_7           │ (None, 7, 128)         │        164,352 │ bidirectional_6[0][0]  │
│ (Bidirectional)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bidirectional_8           │ (None, 7, 64)          │         41,216 │ bidirectional_7[0][0]  │
│ (Bidirectional)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ multi_head_attention_lay… │ (None, 7, 64)          │        132,672 │ bidirectional_8[0][0], │
│ (MultiHeadAttentionLayer) │                        │                │ bidirectional_8[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ flatten_4 (Flatten)       │ (None, 448)            │              0 │ multi_head_attention_… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ flatten_5 (Flatten)       │ (None, 1050)           │              0 │ inputs[0][0]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_2             │ (None, 1498)           │              0 │ flatten_4[0][0],       │
│ (Concatenate)             │                        │                │ flatten_5[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_4 (Dense)           │ (None, 64)             │         95,936 │ concatenate_2[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_7 (Dropout)       │ (None, 64)             │              0 │ dense_4[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_5 (Dense)           │ (None, 32)             │          2,080 │ dropout_7[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_8 (Dropout)       │ (None, 32)             │              0 │ dense_5[0][0]          │
├──────────────────────

 Total params: 2,035,205 (7.76 MB)

 Trainable params: 678,401 (2.59 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 1,356,804 (5.18 MB)

 ##### 16

In [ ]:
wandb.finish()

epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
train_loss,█▃▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_rmse,█▃▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,▃█▅█▅▅▅▄▄▄▄▄▄▄▃▅▄▃▃▃▅▃▃▄▃▃▄▄▂▃▃▁▂▂▃▃▃▃▂▁
val_rmse,█▃▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,50
train_loss,0.04134
train_rmse,0.04125
val_loss,0.16456
val_rmse,0.04125


In [ ]:
wandb.init(project="Stock Price CNN-LSTM - Split, Same", name="hyper16", config={
    "epochs": 50,
    "batch_size": 64,
    "time_steps": 30,
    "lstm_units": [128,64,32],
    "optimizer": "adam",
    "loss": "rmse",
},reinit=True)

In [ ]:
# Skip Connection Model with Multi-Head Attention
def cnn_lstm_attention_skip_model(time_steps, num_features, lstm_units, num_heads, key_dim):
    # Input Layer
    inputs = Input(shape=(time_steps, num_features), name='inputs')

    # CNN Feature Extraction
    x = Conv1D(64, kernel_size=5, activation='relu', padding='same')(inputs)
    x = MaxPooling1D(pool_size=2)(x)
    x = Conv1D(64, kernel_size=5, activation='relu', padding='same')(x)
    x = MaxPooling1D(pool_size=2)(x)

    # LSTM Feature Extraction
    lstm_out = Bidirectional(LSTM(lstm_units[0], return_sequences=True))(x)
    lstm_out = Bidirectional(LSTM(lstm_units[1], return_sequences=True))(lstm_out)
    lstm_out = Bidirectional(LSTM(lstm_units[2], return_sequences=True))(lstm_out)

    # Multi-Head Attention
    attention = MultiHeadAttentionLayer(num_heads=num_heads, key_dim=key_dim)(lstm_out, lstm_out)

    # Flatten LSTM and Attention Outputs
    lstm_flatten = Flatten()(attention)

    # Skip Connection (Flatten input)
    skip_flatten = Flatten()(inputs)

    # Concatenate LSTM-Attention Features and Raw Input
    concat = Concatenate(axis=-1)([lstm_flatten, skip_flatten])

    # Fully Connected Layers
    x = Dense(64, activation='relu')(concat)
    x = Dropout(0.1)(x)
    x = Dense(32, activation='relu')(x)
    x = Dropout(0.1)(x)

    # Output Layer
    output = Dense(1, name='output')(x)

    # Define Model
    model = Model(inputs=inputs, outputs=output, name='cnn_lstm_attention_skip')

    return model

In [ ]:
# Model Parameters
time_steps = wandb.config.time_steps
num_features = len(features)
lstm_units = wandb.config.lstm_units
num_heads = 8  # Number of attention heads
key_dim = 64   # Dimensionality of the attention layer

X_train, y_train = create_sequences(x_train_int_df, y_train_int_df, time_steps)
X_val, y_val = create_sequences(x_val_int_df, y_val_int_df, time_steps)

# Build Model
model_interp_hyper16 = cnn_lstm_attention_skip_model(time_steps, num_features, lstm_units, num_heads, key_dim)

# Compile Model
model_interp_hyper16.compile(
    optimizer=wandb.config.optimizer,
    loss=rmse_metric,
    metrics=[rmse_metric]
)

# Callbacks
model_checkpoint = ModelCheckpoint(
    '/content/drive/MyDrive/Time Series Project/Stock Price/Model/model_path/int_hyper16.keras',
    monitor='val_loss',
    save_best_only=True,
    mode='min',
    verbose=1
)

# Train Model
for epoch in range(wandb.config.epochs):
    print(f"Epoch {epoch + 1}/{wandb.config.epochs}")
    history = model_interp_hyper16.fit(
        X_train, y_train,
        epochs=1,
        batch_size=wandb.config.batch_size,
        validation_data=(X_val, y_val),
        callbacks=[early_stopping, model_checkpoint],
        verbose=1,
    )

    # Log Metrics to W&B
    wandb.log({
        "epoch": epoch + 1,
        "train_loss": history.history['loss'][0],
        "val_loss": history.history['val_loss'][0],
        "train_rmse": history.history['rmse_metric'][0],
        "val_rmse": history.history['rmse_metric'][0],
    })

# Model Summary
model_interp_hyper16.summary()

Epoch 1/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.2212 - rmse_metric: 0.2212
Epoch 1: val_loss improved from inf to 0.31980, saving model to /content/drive/MyDrive/Time Series Project/Stock Price/Model/model_path/int_hyper16.keras
55/55 ━━━━━━━━━━━━━━━━━━━━ 13s 35ms/step - loss: 0.2199 - rmse_metric: 0.2199 - val_loss: 0.3198 - val_rmse_metric: 0.3213
Restoring model weights from the end of the best epoch: 1.
Epoch 2/50
52/55 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.0836 - rmse_metric: 0.0836
Epoch 1: val_loss improved from 0.31980 to 0.23410, saving model to /content/drive/MyDrive/Time Series Project/Stock Price/Model/model_path/int_hyper16.keras
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - loss: 0.0834 - rmse_metric: 0.0834 - val_loss: 0.2341 - val_rmse_metric: 0.2353
Restoring model weights from the end of the best epoch: 1.
Epoch 3/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.0691 - rmse_metric: 0.0690
Epoch 1: val_loss improved from 0.23410 to 0.22567, saving mo

Model: "cnn_lstm_attention_skip"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ inputs (InputLayer)       │ (None, 30, 35)         │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_8 (Conv1D)         │ (None, 30, 64)         │         11,264 │ inputs[0][0]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling1d_8           │ (None, 15, 64)         │              0 │ conv1d_8[0][0]         │
│ (MaxPooling1D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_9 (Conv1D)         │ (None, 15, 64)         │         20,544 │ max_pooling1d_8[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling1d_9           │ (None, 7, 64)          │              0 │ conv1d_9[0][0]         │
│ (MaxPooling1D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bidirectional_12          │ (None, 7, 256)         │        197,632 │ max_pooling1d_9[0][0]  │
│ (Bidirectional)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bidirectional_13          │ (None, 7, 128)         │        164,352 │ bidirectional_12[0][0] │
│ (Bidirectional)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bidirectional_14          │ (None, 7, 64)          │         41,216 │ bidirectional_13[0][0] │
│ (Bidirectional)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ multi_head_attention_lay… │ (None, 7, 64)          │        132,672 │ bidirectional_14[0][0… │
│ (MultiHeadAttentionLayer) │                        │                │ bidirectional_14[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ flatten_8 (Flatten)       │ (None, 448)            │              0 │ multi_head_attention_… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ flatten_9 (Flatten)       │ (None, 1050)           │              0 │ inputs[0][0]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_4             │ (None, 1498)           │              0 │ flatten_8[0][0],       │
│ (Concatenate)             │                        │                │ flatten_9[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_8 (Dense)           │ (None, 64)             │         95,936 │ concatenate_4[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_13 (Dropout)      │ (None, 64)             │              0 │ dense_8[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_9 (Dense)           │ (None, 32)             │          2,080 │ dropout_13[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_14 (Dropout)      │ (None, 32)             │              0 │ dense_9[0][0]          │
├──────────────────────

 Total params: 1,997,189 (7.62 MB)

 Trainable params: 665,729 (2.54 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 1,331,460 (5.08 MB)

##### 17

In [ ]:
wandb.finish()

epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train_loss,█▄▃▃▃▂▂▂▂▂▂▂▂▂▁▂▂▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_rmse,█▄▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▄▄▄▂▃▂▁▁▂▃▁▂▂▂▁▂▂▂▂▂▂▂▂▃▂▁▂▂▂▂▂▂▂▁▂▂▂▂▂
val_rmse,█▄▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,50
train_loss,0.02986
train_rmse,0.02978
val_loss,0.19781
val_rmse,0.02978


In [ ]:
wandb.init(project="Stock Price CNN-LSTM - Split, Same", name="hyper17", config={
    "epochs": 50,
    "batch_size": 64,
    "time_steps": 30,
    "lstm_units": [128,64,32],
    "optimizer": "adam",
    "loss": "rmse",
},reinit=True)

In [ ]:
# Skip Connection Model with Multi-Head Attention
def cnn_lstm_attention_skip_model(time_steps, num_features, lstm_units):
    # Input Layer
    inputs = Input(shape=(time_steps, num_features), name='inputs')

    # CNN Feature Extraction
    x = Conv1D(64, kernel_size=5, activation='relu', padding='same')(inputs)
    x = MaxPooling1D(pool_size=2)(x)
    x = Conv1D(64, kernel_size=5, activation='relu', padding='same')(x)
    x = MaxPooling1D(pool_size=2)(x)

    # LSTM Feature Extraction
    lstm_out = Bidirectional(LSTM(lstm_units[0], return_sequences=True))(x)
    lstm_out = Bidirectional(LSTM(lstm_units[1], return_sequences=True))(lstm_out)

    # Multi-Head Attention
    attention = Attention()(lstm_out)

    # Flatten LSTM and Attention Outputs
    lstm_flatten = Flatten()(attention)

    # Skip Connection (Flatten input)
    skip_flatten = Flatten()(inputs)

    # Concatenate LSTM-Attention Features and Raw Input
    concat = Concatenate(axis=-1)([lstm_flatten, skip_flatten])

    # Fully Connected Layers
    x = Dense(64, activation='relu')(concat)
    x = Dropout(0.1)(x)
    x = Dense(32, activation='relu')(x)
    x = Dropout(0.1)(x)

    # Output Layer
    output = Dense(1, name='output')(x)

    # Define Model
    model = Model(inputs=inputs, outputs=output, name='cnn_lstm_attention_skip')

    return model

In [ ]:
# Model Parameters
time_steps = wandb.config.time_steps
num_features = len(features)
lstm_units = wandb.config.lstm_units
num_heads = 8  # Number of attention heads
key_dim = 64   # Dimensionality of the attention layer

X_train, y_train = create_sequences(x_train_int_df, y_train_int_df, time_steps)
X_val, y_val = create_sequences(x_val_int_df, y_val_int_df, time_steps)

# Build Model
model_interp_hyper17 = cnn_lstm_attention_skip_model(time_steps, num_features, lstm_units)

# Compile Model
model_interp_hyper17.compile(
    optimizer=wandb.config.optimizer,
    loss=rmse_metric,
    metrics=[rmse_metric]
)

# Callbacks
model_checkpoint = ModelCheckpoint(
    '/content/drive/MyDrive/Time Series Project/Stock Price/Model/model_path/int_hyper17.keras',
    monitor='val_loss',
    save_best_only=True,
    mode='min',
    verbose=1
)

# Train Model
for epoch in range(wandb.config.epochs):
    print(f"Epoch {epoch + 1}/{wandb.config.epochs}")
    history = model_interp_hyper17.fit(
        X_train, y_train,
        epochs=1,
        batch_size=wandb.config.batch_size,
        validation_data=(X_val, y_val),
        callbacks=[early_stopping, model_checkpoint],
        verbose=1,
    )

    # Log Metrics to W&B
    wandb.log({
        "epoch": epoch + 1,
        "train_loss": history.history['loss'][0],
        "val_loss": history.history['val_loss'][0],
        "train_rmse": history.history['rmse_metric'][0],
        "val_rmse": history.history['rmse_metric'][0],
    })

# Model Summary
model_interp_hyper17.summary()

Epoch 1/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 0.3988 - rmse_metric: 0.3988
Epoch 1: val_loss improved from inf to 0.28633, saving model to /content/drive/MyDrive/Time Series Project/Stock Price/Model/model_path/int_hyper17.keras
55/55 ━━━━━━━━━━━━━━━━━━━━ 9s 63ms/step - loss: 0.3957 - rmse_metric: 0.3957 - val_loss: 0.2863 - val_rmse_metric: 0.2878
Restoring model weights from the end of the best epoch: 1.
Epoch 2/50
54/55 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.0966 - rmse_metric: 0.0966
Epoch 1: val_loss improved from 0.28633 to 0.24309, saving model to /content/drive/MyDrive/Time Series Project/Stock Price/Model/model_path/int_hyper17.keras
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: 0.0965 - rmse_metric: 0.0965 - val_loss: 0.2431 - val_rmse_metric: 0.2434
Restoring model weights from the end of the best epoch: 1.
Epoch 3/50
54/55 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 0.0849 - rmse_metric: 0.0849
Epoch 1: val_loss improved from 0.24309 to 0.15217, saving mod

Model: "cnn_lstm_attention_skip"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ inputs (InputLayer)       │ (None, 30, 35)         │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_14 (Conv1D)        │ (None, 30, 64)         │         11,264 │ inputs[0][0]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling1d_14          │ (None, 15, 64)         │              0 │ conv1d_14[0][0]        │
│ (MaxPooling1D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_15 (Conv1D)        │ (None, 15, 64)         │         20,544 │ max_pooling1d_14[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling1d_15          │ (None, 7, 64)          │              0 │ conv1d_15[0][0]        │
│ (MaxPooling1D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bidirectional_19          │ (None, 7, 256)         │        197,632 │ max_pooling1d_15[0][0] │
│ (Bidirectional)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bidirectional_20          │ (None, 7, 128)         │        164,352 │ bidirectional_19[0][0] │
│ (Bidirectional)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ attention_2 (Attention)   │ (None, 128)            │         16,512 │ bidirectional_20[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ flatten_10 (Flatten)      │ (None, 128)            │              0 │ attention_2[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ flatten_11 (Flatten)      │ (None, 1050)           │              0 │ inputs[0][0]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_5             │ (None, 1178)           │              0 │ flatten_10[0][0],      │
│ (Concatenate)             │                        │                │ flatten_11[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_10 (Dense)          │ (None, 64)             │         75,456 │ concatenate_5[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_15 (Dropout)      │ (None, 64)             │              0 │ dense_10[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_11 (Dense)          │ (None, 32)             │          2,080 │ dropout_15[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_16 (Dropout)      │ (None, 32)             │              0 │ dense_11[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ output (Dense)            │ (None, 1)              │             33 │ dropout_16[0][0]       │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 1,463,621 (5.58 MB)

 Trainable params: 487,873 (1.86 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 975,748 (3.72 MB)

##### 18

In [ ]:
wandb.finish()

wandb.init(project="Stock Price CNN-LSTM - Split, Same", name="hyper18", config={
    "epochs": 50,
    "batch_size": 64,
    "time_steps": 30,
    "lstm_units": [128,64,32],
    "optimizer": "adam",
    "loss": "rmse",
},reinit=True)

epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
train_loss,█▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_rmse,█▇▅▅▄▃▃▃▄▃▃▃▂▂▂▂▂▂▂▂▂▁▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▆▁▂▂▁▁▂▁▂▁▁▁▂▂▂▂▂▃▂▂▃▁▃▂▃▃▂▃▂▃▂▃▃▄▃▄▃▂▄
val_rmse,█▇▅▅▄▃▃▄▃▃▃▂▂▂▂▂▂▂▂▂▂▁▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,50
train_loss,0.02942
train_rmse,0.02957
val_loss,0.1964
val_rmse,0.02957


In [ ]:
# Skip Connection Model with Multi-Head Attention
def cnn_lstm_attention_skip_model(time_steps, num_features, lstm_units):
    # Input Layer
    inputs = Input(shape=(time_steps, num_features), name='inputs')

    # CNN Feature Extraction
    x = Conv1D(64, kernel_size=5, activation='relu', padding='same')(inputs)
    x = MaxPooling1D(pool_size=2)(x)
    x = Conv1D(64, kernel_size=5, activation='relu', padding='same')(x)
    x = MaxPooling1D(pool_size=2)(x)

    # LSTM Feature Extraction
    lstm_out = Bidirectional(LSTM(lstm_units[0], return_sequences=True))(x)
    lstm_out = Bidirectional(LSTM(lstm_units[1], return_sequences=True))(lstm_out)
    lstm_out = Bidirectional(LSTM(lstm_units[1], return_sequences=True))(lstm_out)
    lstm_out = Bidirectional(LSTM(lstm_units[2], return_sequences=True))(lstm_out)

    # Multi-Head Attention
    attention = Attention()(lstm_out)

    # Flatten LSTM and Attention Outputs
    lstm_flatten = Flatten()(attention)

    # Skip Connection (Flatten input)
    skip_flatten = Flatten()(inputs)

    # Concatenate LSTM-Attention Features and Raw Input
    concat = Concatenate(axis=-1)([lstm_flatten, skip_flatten])

    # Fully Connected Layers
    x = Dense(64, activation='relu')(concat)
    x = Dropout(0.1)(x)
    x = Dense(32, activation='relu')(x)
    x = Dropout(0.1)(x)

    # Output Layer
    output = Dense(1, name='output')(x)

    # Define Model
    model = Model(inputs=inputs, outputs=output, name='cnn_lstm_attention_skip')

    return model

In [ ]:
# Model Parameters
time_steps = wandb.config.time_steps
num_features = len(features)
lstm_units = wandb.config.lstm_units
num_heads = 8  # Number of attention heads
key_dim = 64   # Dimensionality of the attention layer

X_train, y_train = create_sequences(x_train_int_df, y_train_int_df, time_steps)
X_val, y_val = create_sequences(x_val_int_df, y_val_int_df, time_steps)

# Build Model
model_interp_hyper18 = cnn_lstm_attention_skip_model(time_steps, num_features, lstm_units)

# Compile Model
model_interp_hyper18.compile(
    optimizer=wandb.config.optimizer,
    loss=rmse_metric,
    metrics=[rmse_metric]
)

# Callbacks
model_checkpoint = ModelCheckpoint(
    '/content/drive/MyDrive/Time Series Project/Stock Price/Model/model_path/int_hyper18.keras',
    monitor='val_loss',
    save_best_only=True,
    mode='min',
    verbose=1
)

# Train Model
for epoch in range(wandb.config.epochs):
    print(f"Epoch {epoch + 1}/{wandb.config.epochs}")
    history = model_interp_hyper18.fit(
        X_train, y_train,
        epochs=1,
        batch_size=wandb.config.batch_size,
        validation_data=(X_val, y_val),
        callbacks=[early_stopping, model_checkpoint],
        verbose=1,
    )

    # Log Metrics to W&B
    wandb.log({
        "epoch": epoch + 1,
        "train_loss": history.history['loss'][0],
        "val_loss": history.history['val_loss'][0],
        "train_rmse": history.history['rmse_metric'][0],
        "val_rmse": history.history['rmse_metric'][0],
    })

# Model Summary
model_interp_hyper18.summary()

Epoch 1/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 0.3541 - rmse_metric: 0.3541
Epoch 1: val_loss improved from inf to 0.26605, saving model to /content/drive/MyDrive/Time Series Project/Stock Price/Model/model_path/int_hyper18.keras
55/55 ━━━━━━━━━━━━━━━━━━━━ 10s 37ms/step - loss: 0.3520 - rmse_metric: 0.3519 - val_loss: 0.2660 - val_rmse_metric: 0.2676
Restoring model weights from the end of the best epoch: 1.
Epoch 2/50
54/55 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 0.1012 - rmse_metric: 0.1012
Epoch 1: val_loss did not improve from 0.26605
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - loss: 0.1011 - rmse_metric: 0.1011 - val_loss: 0.3133 - val_rmse_metric: 0.3146
Restoring model weights from the end of the best epoch: 1.
Epoch 3/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 0.0857 - rmse_metric: 0.0857
Epoch 1: val_loss improved from 0.26605 to 0.23202, saving model to /content/drive/MyDrive/Time Series Project/Stock Price/Model/model_path/int_hyper18.keras
55/55 ━━━━━━━━

Model: "cnn_lstm_attention_skip"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ inputs (InputLayer)       │ (None, 30, 35)         │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_16 (Conv1D)        │ (None, 30, 64)         │         11,264 │ inputs[0][0]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling1d_16          │ (None, 15, 64)         │              0 │ conv1d_16[0][0]        │
│ (MaxPooling1D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_17 (Conv1D)        │ (None, 15, 64)         │         20,544 │ max_pooling1d_16[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling1d_17          │ (None, 7, 64)          │              0 │ conv1d_17[0][0]        │
│ (MaxPooling1D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bidirectional_21          │ (None, 7, 256)         │        197,632 │ max_pooling1d_17[0][0] │
│ (Bidirectional)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bidirectional_22          │ (None, 7, 128)         │        164,352 │ bidirectional_21[0][0] │
│ (Bidirectional)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bidirectional_23          │ (None, 7, 128)         │         98,816 │ bidirectional_22[0][0] │
│ (Bidirectional)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bidirectional_24          │ (None, 7, 64)          │         41,216 │ bidirectional_23[0][0] │
│ (Bidirectional)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ attention_3 (Attention)   │ (None, 64)             │          4,160 │ bidirectional_24[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ flatten_12 (Flatten)      │ (None, 64)             │              0 │ attention_3[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ flatten_13 (Flatten)      │ (None, 1050)           │              0 │ inputs[0][0]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_6             │ (None, 1114)           │              0 │ flatten_12[0][0],      │
│ (Concatenate)             │                        │                │ flatten_13[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_12 (Dense)          │ (None, 64)             │         71,360 │ concatenate_6[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_17 (Dropout)      │ (None, 64)             │              0 │ dense_12[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_13 (Dense)          │ (None, 32)             │          2,080 │ dropout_17[0][0]       │
├──────────────────────

 Total params: 1,834,373 (7.00 MB)

 Trainable params: 611,457 (2.33 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 1,222,916 (4.67 MB)

##### 19

In [ ]:
wandb.finish()

wandb.init(project="Stock Price CNN-LSTM - Split, Same", name="hyper19", config={
    "epochs": 50,
    "batch_size": 64,
    "time_steps": 30,
    "lstm_units": [128,64,32],
    "optimizer": "adam",
    "loss": "rmse",
},reinit=True)

epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
train_loss,█▃▃▂▂▂▂▂▂▂▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_rmse,█▃▃▂▂▂▂▂▂▂▁▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,▆█▄▂▂▂▂▂▃▂▃▃▃▃▂▄▃▂▁▂▂▂▂▂▂▂▃▂▃▂▄▂▂▂▃▃▂▂▂▃
val_rmse,█▃▂▂▂▂▂▂▂▂▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,50
train_loss,0.03103
train_rmse,0.03107
val_loss,0.21659
val_rmse,0.03107


In [ ]:
# Skip Connection Model with Multi-Head Attention
def cnn_lstm_attention_skip_model(time_steps, num_features, lstm_units):
    # Input Layer
    inputs = Input(shape=(time_steps, num_features), name='inputs')

    # CNN Feature Extraction
    x = Conv1D(64, kernel_size=5, activation='relu', padding='same')(inputs)
    x = MaxPooling1D(pool_size=2)(x)
    x = Conv1D(64, kernel_size=5, activation='relu', padding='same')(x)
    x = MaxPooling1D(pool_size=2)(x)

    # LSTM Feature Extraction
    lstm_out = Bidirectional(LSTM(lstm_units[1], return_sequences=True))(x)
    lstm_out = Bidirectional(LSTM(lstm_units[1], return_sequences=True))(lstm_out)

    # Multi-Head Attention
    attention = Attention()(lstm_out)

    # Flatten LSTM and Attention Outputs
    lstm_flatten = Flatten()(attention)

    # Skip Connection (Flatten input)
    skip_flatten = Flatten()(inputs)

    # Concatenate LSTM-Attention Features and Raw Input
    concat = Concatenate(axis=-1)([lstm_flatten, skip_flatten])

    # Fully Connected Layers
    x = Dense(64, activation='relu')(concat)
    x = Dropout(0.1)(x)
    x = Dense(32, activation='relu')(x)
    x = Dropout(0.1)(x)

    # Output Layer
    output = Dense(1, name='output')(x)

    # Define Model
    model = Model(inputs=inputs, outputs=output, name='cnn_lstm_attention_skip')

    return model

In [ ]:
# Model Parameters
time_steps = wandb.config.time_steps
num_features = len(features)
lstm_units = wandb.config.lstm_units
num_heads = 8  # Number of attention heads
key_dim = 64   # Dimensionality of the attention layer

X_train, y_train = create_sequences(x_train_int_df, y_train_int_df, time_steps)
X_val, y_val = create_sequences(x_val_int_df, y_val_int_df, time_steps)

# Build Model
model_interp_hyper19 = cnn_lstm_attention_skip_model(time_steps, num_features, lstm_units)

# Compile Model
model_interp_hyper19.compile(
    optimizer=wandb.config.optimizer,
    loss=rmse_metric,
    metrics=[rmse_metric]
)

# Callbacks
model_checkpoint = ModelCheckpoint(
    '/content/drive/MyDrive/Time Series Project/Stock Price/Model/model_path/int_hyper19.keras',
    monitor='val_loss',
    save_best_only=True,
    mode='min',
    verbose=1
)

# Train Model
for epoch in range(wandb.config.epochs):
    print(f"Epoch {epoch + 1}/{wandb.config.epochs}")
    history = model_interp_hyper19.fit(
        X_train, y_train,
        epochs=1,
        batch_size=wandb.config.batch_size,
        validation_data=(X_val, y_val),
        callbacks=[early_stopping, model_checkpoint],
        verbose=1,
    )

    # Log Metrics to W&B
    wandb.log({
        "epoch": epoch + 1,
        "train_loss": history.history['loss'][0],
        "val_loss": history.history['val_loss'][0],
        "train_rmse": history.history['rmse_metric'][0],
        "val_rmse": history.history['rmse_metric'][0],
    })

# Model Summary
model_interp_hyper19.summary()

Epoch 1/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.3471 - rmse_metric: 0.3470
Epoch 1: val_loss improved from inf to 0.29654, saving model to /content/drive/MyDrive/Time Series Project/Stock Price/Model/model_path/int_hyper19.keras
55/55 ━━━━━━━━━━━━━━━━━━━━ 11s 26ms/step - loss: 0.3444 - rmse_metric: 0.3444 - val_loss: 0.2965 - val_rmse_metric: 0.2983
Restoring model weights from the end of the best epoch: 1.
Epoch 2/50
53/55 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.1013 - rmse_metric: 0.1013
Epoch 1: val_loss improved from 0.29654 to 0.25251, saving model to /content/drive/MyDrive/Time Series Project/Stock Price/Model/model_path/int_hyper19.keras
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 0.1011 - rmse_metric: 0.1010 - val_loss: 0.2525 - val_rmse_metric: 0.2540
Restoring model weights from the end of the best epoch: 1.
Epoch 3/50
53/55 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0809 - rmse_metric: 0.0809
Epoch 1: val_loss improved from 0.25251 to 0.24607, saving mo

Model: "cnn_lstm_attention_skip"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ inputs (InputLayer)       │ (None, 30, 35)         │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_18 (Conv1D)        │ (None, 30, 64)         │         11,264 │ inputs[0][0]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling1d_18          │ (None, 15, 64)         │              0 │ conv1d_18[0][0]        │
│ (MaxPooling1D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_19 (Conv1D)        │ (None, 15, 64)         │         20,544 │ max_pooling1d_18[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling1d_19          │ (None, 7, 64)          │              0 │ conv1d_19[0][0]        │
│ (MaxPooling1D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bidirectional_25          │ (None, 7, 128)         │         66,048 │ max_pooling1d_19[0][0] │
│ (Bidirectional)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bidirectional_26          │ (None, 7, 128)         │         98,816 │ bidirectional_25[0][0] │
│ (Bidirectional)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ attention_4 (Attention)   │ (None, 128)            │         16,512 │ bidirectional_26[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ flatten_14 (Flatten)      │ (None, 128)            │              0 │ attention_4[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ flatten_15 (Flatten)      │ (None, 1050)           │              0 │ inputs[0][0]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_7             │ (None, 1178)           │              0 │ flatten_14[0][0],      │
│ (Concatenate)             │                        │                │ flatten_15[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_14 (Dense)          │ (None, 64)             │         75,456 │ concatenate_7[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_19 (Dropout)      │ (None, 64)             │              0 │ dense_14[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_15 (Dense)          │ (None, 32)             │          2,080 │ dropout_19[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_20 (Dropout)      │ (None, 32)             │              0 │ dense_15[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ output (Dense)            │ (None, 1)              │             33 │ dropout_20[0][0]       │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 872,261 (3.33 MB)

 Trainable params: 290,753 (1.11 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 581,508 (2.22 MB)

## LSTM + CNN-LSTM Hyperparameter Tuning - Interp

##### 1

In [ ]:
wandb.finish()

wandb.init(project="Stock Price LSTM - Split, Interpolation", name="hyper1", config={
    "epochs": 30,
    "batch_size": 64,
    "time_steps": 30,
    "lstm_units": [64, 32],
    "optimizer": "adam",
    "loss": "rmse",
},reinit=True)

epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇██
train_loss,█▄▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_rmse,█▄▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▆▅▃▃▃▃▁▄▃▂▂▄▃▂▃▃▂▁▂▂▃▂▂▂▂▂▂▂▂▃▃▁▂▃▃▂▂▂▂
val_rmse,█▄▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,50
train_loss,0.03459
train_rmse,0.03452
val_loss,0.19159
val_rmse,0.03452


In [ ]:
model_checkpoint = ModelCheckpoint(
    '/content/drive/MyDrive/Time Series Project/Stock Price/Model/model_path/intep-hyper1.keras',
    monitor='val_loss',
    save_best_only=True,
    mode='min',
    verbose=1
)

time_steps = wandb.config.time_steps
X_train, y_train = create_sequences(x_train_int_df, y_train_int_df, time_steps)
X_val, y_val = create_sequences(x_val_int_df, y_val_int_df, time_steps)

model_interp_hyper1 = tf.keras.Sequential([
    tf.keras.layers.LSTM(wandb.config.lstm_units[0], activation='tanh', input_shape=(time_steps, len(features)), return_sequences=True),
    tf.keras.layers.LSTM(wandb.config.lstm_units[1], activation='tanh'),
    tf.keras.layers.Dense(1)
])

model_interp_hyper1.compile(optimizer=wandb.config.optimizer, loss=rmse_metric, metrics=[rmse_metric])

for epoch in range(wandb.config.epochs):
    print(f"Epoch {epoch + 1}/{wandb.config.epochs}")
    history = model_interp_hyper1.fit(
        X_train, y_train,
        epochs=1,
        batch_size=wandb.config.batch_size,
        validation_data=(X_val, y_val),
        callbacks=[early_stopping, model_checkpoint],
        verbose=1,
    )
    wandb.log({
        "epoch": epoch + 1,
        "train_loss": history.history['loss'][0],
        "val_loss": history.history['val_loss'][0],
        "train_rmse": history.history['rmse_metric'][0],
        "val_rmse": history.history['val_rmse_metric'][0],
    })

model_interp_hyper1.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/30
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.1879 - rmse_metric: 0.1879
Epoch 1: val_loss improved from inf to 0.22788, saving model to /content/drive/MyDrive/Time Series Project/Stock Price/Model/model_path/intep-hyper1.keras
55/55 ━━━━━━━━━━━━━━━━━━━━ 5s 24ms/step - loss: 0.1862 - rmse_metric: 0.1862 - val_loss: 0.2279 - val_rmse_metric: 0.2287
Restoring model weights from the end of the best epoch: 1.
Epoch 2/30
53/55 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0327 - rmse_metric: 0.0327
Epoch 1: val_loss improved from 0.22788 to 0.17839, saving model to /content/drive/MyDrive/Time Series Project/Stock Price/Model/model_path/intep-hyper1.keras
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.0327 - rmse_metric: 0.0327 - val_loss: 0.1784 - val_rmse_metric: 0.1789
Restoring model weights from the end of the best epoch: 1.
Epoch 3/30
52/55 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0297 - rmse_metric: 0.0297
Epoch 1: val_loss improved from 0.17839 to 0.17708, saving mo

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_27 (LSTM)                       │ (None, 30, 64)              │          25,600 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_28 (LSTM)                       │ (None, 32)                  │          12,416 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_16 (Dense)                     │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 114,149 (445.90 KB)

 Trainable params: 38,049 (148.63 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 76,100 (297.27 KB)

##### 2

In [ ]:
wandb.finish()

wandb.init(project="Stock Price LSTM - Split, Interpolation", name="hyper2", config={
    "epochs": 30,
    "batch_size": 64,
    "time_steps": 30,
    "lstm_units": [64, 64],
    "optimizer": "adam",
    "loss": "rmse",
},reinit=True)

model_checkpoint = ModelCheckpoint(
    '/content/drive/MyDrive/Time Series Project/Stock Price/Model/model_path/intep-hyper2.keras',
    monitor='val_loss',
    save_best_only=True,
    mode='min',
    verbose=1
)

time_steps = wandb.config.time_steps
X_train, y_train = create_sequences(x_train_int_df, y_train_int_df, time_steps)
X_val, y_val = create_sequences(x_val_int_df, y_val_int_df, time_steps)

model_interp_hyper2 = tf.keras.Sequential([
    tf.keras.layers.LSTM(wandb.config.lstm_units[0], activation='tanh', input_shape=(time_steps, len(features)), return_sequences=True),
    tf.keras.layers.LSTM(wandb.config.lstm_units[1], activation='tanh'),
    tf.keras.layers.Dense(1)
])

model_interp_hyper2.compile(optimizer=wandb.config.optimizer, loss=rmse_metric, metrics=[rmse_metric])

for epoch in range(wandb.config.epochs):
    print(f"Epoch {epoch + 1}/{wandb.config.epochs}")
    history = model_interp_hyper2.fit(
        X_train, y_train,
        epochs=1,
        batch_size=wandb.config.batch_size,
        validation_data=(X_val, y_val),
        callbacks=[early_stopping, model_checkpoint],
        verbose=1,
    )
    wandb.log({
        "epoch": epoch + 1,
        "train_loss": history.history['loss'][0],
        "val_loss": history.history['val_loss'][0],
        "train_rmse": history.history['rmse_metric'][0],
        "val_rmse": history.history['val_rmse_metric'][0],
    })

model_interp_hyper2.summary()

epoch,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
train_loss,█▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_rmse,█▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▃▂▁▂▂▃▁▂▂▂▂▃▂▃▂▃▂▂▂▂▃▃▂▃▃▄▄▃▃
val_rmse,█▃▂▁▂▂▃▁▂▂▂▂▃▂▃▂▃▂▂▂▂▃▃▂▃▃▄▄▃▃
epoch,30
train_loss,0.01074
train_rmse,0.01071
val_loss,0.18489
val_rmse,0.18583


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/30
52/55 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.1828 - rmse_metric: 0.1828
Epoch 1: val_loss improved from inf to 0.15239, saving model to /content/drive/MyDrive/Time Series Project/Stock Price/Model/model_path/intep-hyper2.keras
55/55 ━━━━━━━━━━━━━━━━━━━━ 4s 25ms/step - loss: 0.1765 - rmse_metric: 0.1765 - val_loss: 0.1524 - val_rmse_metric: 0.1531
Restoring model weights from the end of the best epoch: 1.
Epoch 2/30
53/55 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0326 - rmse_metric: 0.0326
Epoch 1: val_loss did not improve from 0.15239
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.0325 - rmse_metric: 0.0325 - val_loss: 0.1637 - val_rmse_metric: 0.1641
Restoring model weights from the end of the best epoch: 1.
Epoch 3/30
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0274 - rmse_metric: 0.0274
Epoch 1: val_loss improved from 0.15239 to 0.13664, saving model to /content/drive/MyDrive/Time Series Project/Stock Price/Model/model_path/intep-hyper2.keras
55/55 ━━━━━━━━━

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_29 (LSTM)                       │ (None, 30, 64)              │          25,600 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_30 (LSTM)                       │ (None, 64)                  │          33,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_17 (Dense)                     │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 176,069 (687.77 KB)

 Trainable params: 58,689 (229.25 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 117,380 (458.52 KB)

##### 3

In [ ]:
#wandb.finish()

wandb.init(project="Stock Price LSTM - Split, Interpolation", name="hyper3", config={
    "epochs": 50,
    "batch_size": 64,
    "time_steps": 30,
    "lstm_units": [64, 64],
    "optimizer": "adam",
    "loss": "rmse",
},reinit=True)

model_checkpoint = ModelCheckpoint(
    '/content/drive/MyDrive/Time Series Project/Stock Price/Model/model_path/intep-hyper3.keras',
    monitor='val_loss',
    save_best_only=True,
    mode='min',
    verbose=1
)

time_steps = wandb.config.time_steps
X_train, y_train = create_sequences(x_train_int_df, y_train_int_df, time_steps)
X_val, y_val = create_sequences(x_val_int_df, y_val_int_df, time_steps)

model_interp_hyper3 = tf.keras.Sequential([
    tf.keras.layers.LSTM(wandb.config.lstm_units[0], activation='tanh', input_shape=(time_steps, len(features)), return_sequences=True),
    tf.keras.layers.LSTM(wandb.config.lstm_units[1], activation='tanh'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(1)
])

model_interp_hyper3.compile(optimizer=wandb.config.optimizer, loss=rmse_metric, metrics=[rmse_metric])

for epoch in range(wandb.config.epochs):
    print(f"Epoch {epoch + 1}/{wandb.config.epochs}")
    history = model_interp_hyper3.fit(
        X_train, y_train,
        epochs=1,
        batch_size=wandb.config.batch_size,
        validation_data=(X_val, y_val),
        callbacks=[early_stopping, model_checkpoint],
        verbose=1,
    )
    wandb.log({
        "epoch": epoch + 1,
        "train_loss": history.history['loss'][0],
        "val_loss": history.history['val_loss'][0],
        "train_rmse": history.history['rmse_metric'][0],
        "val_rmse": history.history['val_rmse_metric'][0],
    })

model_interp_hyper3.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50
54/55 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - loss: 0.1464 - rmse_metric: 0.1464
Epoch 1: val_loss improved from inf to 0.23311, saving model to /content/drive/MyDrive/Time Series Project/Stock Price/Model/model_path/intep-hyper3.keras
55/55 ━━━━━━━━━━━━━━━━━━━━ 10s 66ms/step - loss: 0.1444 - rmse_metric: 0.1444 - val_loss: 0.2331 - val_rmse_metric: 0.2346
Restoring model weights from the end of the best epoch: 1.
Epoch 2/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - loss: 0.0477 - rmse_metric: 0.0477
Epoch 1: val_loss improved from 0.23311 to 0.21400, saving model to /content/drive/MyDrive/Time Series Project/Stock Price/Model/model_path/intep-hyper3.keras
55/55 ━━━━━━━━━━━━━━━━━━━━ 3s 58ms/step - loss: 0.0477 - rmse_metric: 0.0477 - val_loss: 0.2140 - val_rmse_metric: 0.2152
Restoring model weights from the end of the best epoch: 1.
Epoch 3/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - loss: 0.0420 - rmse_metric: 0.0420
Epoch 1: val_loss did not improve from 0.21400
55/55 ━━━━━━

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_4 (LSTM)                        │ (None, 30, 64)              │          25,600 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_5 (LSTM)                        │ (None, 64)                  │          33,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 64)                  │           4,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 188,549 (736.52 KB)

 Trainable params: 62,849 (245.50 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 125,700 (491.02 KB)

##### 4

In [ ]:
wandb.finish()

wandb.init(project="Stock Price LSTM - Split, Interpolation", name="hyper4", config={
    "epochs": 50,
    "batch_size": 64,
    "time_steps": 30,
    "lstm_units": [64, 64],
    "optimizer": "adam",
    "loss": "rmse",
},reinit=True)

model_checkpoint = ModelCheckpoint(
    '/content/drive/MyDrive/Time Series Project/Stock Price/Model/model_path/intep-hyper4.keras',
    monitor='val_loss',
    save_best_only=True,
    mode='min',
    verbose=1
)

time_steps = wandb.config.time_steps
X_train, y_train = create_sequences(x_train_int_df, y_train_int_df, time_steps)
X_val, y_val = create_sequences(x_val_int_df, y_val_int_df, time_steps)

model_interp_hyper4 = tf.keras.Sequential([
    tf.keras.layers.LSTM(wandb.config.lstm_units[0], activation='tanh', input_shape=(time_steps, len(features)), return_sequences=True),
    tf.keras.layers.LSTM(wandb.config.lstm_units[1], activation='tanh'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1)
])

model_interp_hyper4.compile(optimizer=wandb.config.optimizer, loss=rmse_metric, metrics=[rmse_metric])

for epoch in range(wandb.config.epochs):
    print(f"Epoch {epoch + 1}/{wandb.config.epochs}")
    history = model_interp_hyper4.fit(
        X_train, y_train,
        epochs=1,
        batch_size=wandb.config.batch_size,
        validation_data=(X_val, y_val),
        callbacks=[early_stopping, model_checkpoint],
        verbose=1,
    )
    wandb.log({
        "epoch": epoch + 1,
        "train_loss": history.history['loss'][0],
        "val_loss": history.history['val_loss'][0],
        "train_rmse": history.history['rmse_metric'][0],
        "val_rmse": history.history['val_rmse_metric'][0],
    })

model_interp_hyper4.summary()

Epoch 1/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - loss: 0.1525 - rmse_metric: 0.1525
Epoch 1: val_loss improved from inf to 0.25729, saving model to /content/drive/MyDrive/Time Series Project/Stock Price/Model/model_path/intep-hyper4.keras
55/55 ━━━━━━━━━━━━━━━━━━━━ 8s 99ms/step - loss: 0.1516 - rmse_metric: 0.1516 - val_loss: 0.2573 - val_rmse_metric: 0.2588
Restoring model weights from the end of the best epoch: 1.
Epoch 2/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - loss: 0.0619 - rmse_metric: 0.0619
Epoch 1: val_loss improved from 0.25729 to 0.22513, saving model to /content/drive/MyDrive/Time Series Project/Stock Price/Model/model_path/intep-hyper4.keras
55/55 ━━━━━━━━━━━━━━━━━━━━ 2s 44ms/step - loss: 0.0618 - rmse_metric: 0.0618 - val_loss: 0.2251 - val_rmse_metric: 0.2264
Restoring model weights from the end of the best epoch: 1.
Epoch 3/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - loss: 0.0559 - rmse_metric: 0.0559
Epoch 1: val_loss improved from 0.22513 to 0.19707, saving m

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_6 (LSTM)                        │ (None, 30, 64)              │          25,600 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_7 (LSTM)                        │ (None, 64)                  │          33,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_2 (Flatten)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 64)                  │           4,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 188,549 (736.52 KB)

 Trainable params: 62,849 (245.50 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 125,700 (491.02 KB)

##### 5

In [ ]:
wandb.finish()

epoch,▁▁▁▁▂▃▃▄▄▄▅▅▅▆▆▆▇▇▇▇▁▁▂▂▂▃▃▃▃▄▅▆▆▆▆▇▇▇▇█
train_loss,█▄▄▃▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁
train_rmse,█▄▄▃▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▇▄▃▃▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁
val_loss,█▃▃▄▄▁▂▂▂▂▂▂▂▃▃▂▁▃▃▂▃▄▃▅▅▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇
val_rmse,█▃▃▄▄▁▂▂▂▁▃▂▃▃▂▂▂▂▂▄▃▄▄▃▅▄▄▅▄▅▆▅▇▅▆▆▇▆▇▇
epoch,50
train_loss,0.02144
train_rmse,0.02148
val_loss,0.23186
val_rmse,0.23336


In [ ]:
wandb.init(project="Stock Price LSTM - Split, Interpolation", name="hyper5", config={
    "epochs": 50,
    "batch_size": 64,
    "time_steps": 30,
    "lstm_units": [64, 32],
    "optimizer": "adam",
    "loss": "rmse",
},reinit=True)

model_checkpoint = ModelCheckpoint(
    '/content/drive/MyDrive/Time Series Project/Stock Price/Model/model_path/intep-hyper5.keras',
    monitor='val_loss',
    save_best_only=True,
    mode='min',
    verbose=1
)

time_steps = wandb.config.time_steps
X_train, y_train = create_sequences(x_train_int_df, y_train_int_df, time_steps)
X_val, y_val = create_sequences(x_val_int_df, y_val_int_df, time_steps)

model_interp_hyper5 = tf.keras.Sequential([
    tf.keras.layers.LSTM(wandb.config.lstm_units[0], activation='tanh', input_shape=(time_steps, len(features)), return_sequences=True),
    tf.keras.layers.LSTM(wandb.config.lstm_units[0], activation='tanh', return_sequences=True),
    tf.keras.layers.LSTM(wandb.config.lstm_units[1], activation='tanh', return_sequences=True),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1)
])

model_interp_hyper5.compile(optimizer=wandb.config.optimizer, loss=rmse_metric, metrics=[rmse_metric])

for epoch in range(wandb.config.epochs):
    print(f"Epoch {epoch + 1}/{wandb.config.epochs}")
    history = model_interp_hyper5.fit(
        X_train, y_train,
        epochs=1,
        batch_size=wandb.config.batch_size,
        validation_data=(X_val, y_val),
        callbacks=[early_stopping, model_checkpoint],
        verbose=1,
    )
    wandb.log({
        "epoch": epoch + 1,
        "train_loss": history.history['loss'][0],
        "val_loss": history.history['val_loss'][0],
        "train_rmse": history.history['rmse_metric'][0],
        "val_rmse": history.history['val_rmse_metric'][0],
    })

model_interp_hyper5.summary()

Epoch 1/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step - loss: 0.1744 - rmse_metric: 0.1744
Epoch 1: val_loss improved from inf to 0.21043, saving model to /content/drive/MyDrive/Time Series Project/Stock Price/Model/model_path/intep-hyper5.keras
55/55 ━━━━━━━━━━━━━━━━━━━━ 58s 967ms/step - loss: 0.1733 - rmse_metric: 0.1732 - val_loss: 0.2104 - val_rmse_metric: 0.2115
Restoring model weights from the end of the best epoch: 1.
Epoch 2/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step - loss: 0.0666 - rmse_metric: 0.0666
Epoch 1: val_loss improved from 0.21043 to 0.18648, saving model to /content/drive/MyDrive/Time Series Project/Stock Price/Model/model_path/intep-hyper5.keras
55/55 ━━━━━━━━━━━━━━━━━━━━ 6s 103ms/step - loss: 0.0666 - rmse_metric: 0.0666 - val_loss: 0.1865 - val_rmse_metric: 0.1873
Restoring model weights from the end of the best epoch: 1.
Epoch 3/50
54/55 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - loss: 0.0642 - rmse_metric: 0.0642
Epoch 1: val_loss improved from 0.18648 to 0.18572, savin

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_15 (LSTM)                       │ (None, 30, 64)              │          25,600 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_16 (LSTM)                       │ (None, 30, 64)              │          33,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_17 (LSTM)                       │ (None, 30, 32)              │          12,416 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_4 (Flatten)                  │ (None, 960)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ (None, 64)                  │          61,504 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_7 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_10 (Dense)                     │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 397,829 (1.52 MB)

 Trainable params: 132,609 (518.00 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 265,220 (1.01 MB)

##### 6

In [ ]:
wandb.finish()

epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇██
train_loss,█▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_rmse,█▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▅▅▂▁▅▆▄▆▃▇▄▃▄▅▃▃▁▃▃▅▃▃▄▅▆▄▆▄▆▇▅▄▆█▇▅▅▇▄
val_rmse,█▅▅▂▂▅▆▄▆▁▁▇▄▄▆▄▃▂▄▃▅▄▃▅▅▅▆▅▆▄▇▅▅▆█▇▅▅▇▅
epoch,50
train_loss,0.0231
train_rmse,0.02306
val_loss,0.18465
val_rmse,0.18584


In [ ]:
wandb.init(project="Stock Price LSTM - Split, Interpolation", name="hyper6", config={
    "epochs": 50,
    "batch_size": 64,
    "time_steps": 30,
    "lstm_units": [64, 32],
    "optimizer": "adam",
    "loss": "rmse",
},reinit=True)

model_checkpoint = ModelCheckpoint(
    '/content/drive/MyDrive/Time Series Project/Stock Price/Model/model_path/intep-hyper6.keras',
    monitor='val_loss',
    save_best_only=True,
    mode='min',
    verbose=1
)

time_steps = wandb.config.time_steps
X_train, y_train = create_sequences(x_train_int_df, y_train_int_df, time_steps)
X_val, y_val = create_sequences(x_val_int_df, y_val_int_df, time_steps)

model_interp_hyper6 = tf.keras.Sequential([
    tf.keras.layers.LSTM(wandb.config.lstm_units[0], activation='tanh', input_shape=(time_steps, len(features)), return_sequences=True),
    tf.keras.layers.LSTM(wandb.config.lstm_units[0], activation='tanh', return_sequences=True),
    tf.keras.layers.Dropout(0.2),

    tf.keras.layers.LSTM(wandb.config.lstm_units[1], activation='tanh', return_sequences=True),
    tf.keras.layers.LSTM(wandb.config.lstm_units[1], activation='tanh'),

    tf.keras.layers.Flatten(),

    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.2),

    tf.keras.layers.Dense(1)
])

model_interp_hyper6.compile(optimizer=wandb.config.optimizer, loss=rmse_metric, metrics=[rmse_metric])

for epoch in range(wandb.config.epochs):
    print(f"Epoch {epoch + 1}/{wandb.config.epochs}")
    history = model_interp_hyper6.fit(
        X_train, y_train,
        epochs=1,
        batch_size=wandb.config.batch_size,
        validation_data=(X_val, y_val),
        callbacks=[early_stopping, model_checkpoint],
        verbose=1,
    )
    wandb.log({
        "epoch": epoch + 1,
        "train_loss": history.history['loss'][0],
        "val_loss": history.history['val_loss'][0],
        "train_rmse": history.history['rmse_metric'][0],
        "val_rmse": history.history['val_rmse_metric'][0],
    })

model_interp_hyper6.summary()

Epoch 1/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step - loss: 0.2017 - rmse_metric: 0.2016
Epoch 1: val_loss improved from inf to 0.22509, saving model to /content/drive/MyDrive/Time Series Project/Stock Price/Model/model_path/intep-hyper6.keras
55/55 ━━━━━━━━━━━━━━━━━━━━ 17s 132ms/step - loss: 0.2003 - rmse_metric: 0.2003 - val_loss: 0.2251 - val_rmse_metric: 0.2262
Restoring model weights from the end of the best epoch: 1.
Epoch 2/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step - loss: 0.0705 - rmse_metric: 0.0705
Epoch 1: val_loss improved from 0.22509 to 0.22480, saving model to /content/drive/MyDrive/Time Series Project/Stock Price/Model/model_path/intep-hyper6.keras
55/55 ━━━━━━━━━━━━━━━━━━━━ 5s 87ms/step - loss: 0.0705 - rmse_metric: 0.0705 - val_loss: 0.2248 - val_rmse_metric: 0.2258
Restoring model weights from the end of the best epoch: 1.
Epoch 3/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step - loss: 0.0644 - rmse_metric: 0.0644
Epoch 1: val_loss improved from 0.22480 to 0.19943, savin

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_18 (LSTM)                       │ (None, 30, 64)              │          25,600 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_19 (LSTM)                       │ (None, 30, 64)              │          33,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_8 (Dropout)                  │ (None, 30, 64)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_20 (LSTM)                       │ (None, 30, 32)              │          12,416 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_21 (LSTM)                       │ (None, 32)                  │           8,320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_5 (Flatten)                  │ (None, 32)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_11 (Dense)                     │ (None, 64)                  │           2,112 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_9 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_12 (Dense)                     │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 244,613 (955.52 KB)

 Trainable params: 81,537 (318.50 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 163,076 (637.02 KB)

##### 7

In [ ]:
wandb.finish()

epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
train_loss,█▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_rmse,█▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,██▄▇▄▃▄▆▃▃▄▃▄▂▅▂▃▃▃▄▄▅▄▄▃▄▅▃▃▃▄▄▃▄▃▃▃▁▂▁
val_rmse,██▄▇▅▄▆▃▃▄▃▄▂▅▄▃▃▃▄▃▄▅▄▄▃▄▄▅▃▃▃▄▄▃▄▃▃▁▂▁
epoch,50
train_loss,0.02149
train_rmse,0.02145
val_loss,0.17605
val_rmse,0.17705


In [ ]:
wandb.init(project="Stock Price LSTM - Split, Interpolation", name="hyper7", config={
    "epochs": 50,
    "batch_size": 64,
    "time_steps": 30,
    "lstm_units": [64, 32],
    "optimizer": "adam",
    "loss": "rmse",
},reinit=True)

model_checkpoint = ModelCheckpoint(
    '/content/drive/MyDrive/Time Series Project/Stock Price/Model/model_path/intep-hyper7.keras',
    monitor='val_loss',
    save_best_only=True,
    mode='min',
    verbose=1
)

time_steps = wandb.config.time_steps
X_train, y_train = create_sequences(x_train_int_df, y_train_int_df, time_steps)
X_val, y_val = create_sequences(x_val_int_df, y_val_int_df, time_steps)

model_interp_hyper7 = tf.keras.Sequential([
    tf.keras.layers.LSTM(wandb.config.lstm_units[0], activation='tanh', input_shape=(time_steps, len(features)), return_sequences=True),
    tf.keras.layers.LSTM(wandb.config.lstm_units[0], activation='tanh', return_sequences=True),
    tf.keras.layers.Dropout(0.2),

    tf.keras.layers.LSTM(wandb.config.lstm_units[1], activation='tanh', return_sequences=True),
    tf.keras.layers.LSTM(wandb.config.lstm_units[1], activation='tanh'),

    tf.keras.layers.Flatten(),

    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.2),

    tf.keras.layers.Dense(32, activation='relu'),

    tf.keras.layers.Dense(1)
])

model_interp_hyper7.compile(optimizer=wandb.config.optimizer, loss=rmse_metric, metrics=[rmse_metric])

for epoch in range(wandb.config.epochs):
    print(f"Epoch {epoch + 1}/{wandb.config.epochs}")
    history = model_interp_hyper7.fit(
        X_train, y_train,
        epochs=1,
        batch_size=wandb.config.batch_size,
        validation_data=(X_val, y_val),
        callbacks=[early_stopping, model_checkpoint],
        verbose=1,
    )
    wandb.log({
        "epoch": epoch + 1,
        "train_loss": history.history['loss'][0],
        "val_loss": history.history['val_loss'][0],
        "train_rmse": history.history['rmse_metric'][0],
        "val_rmse": history.history['val_rmse_metric'][0],
    })

model_interp_hyper7.summary()

Epoch 1/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step - loss: 0.2415 - rmse_metric: 0.2415
Epoch 1: val_loss improved from inf to 0.25851, saving model to /content/drive/MyDrive/Time Series Project/Stock Price/Model/model_path/intep-hyper7.keras
55/55 ━━━━━━━━━━━━━━━━━━━━ 14s 100ms/step - loss: 0.2398 - rmse_metric: 0.2398 - val_loss: 0.2585 - val_rmse_metric: 0.2594
Restoring model weights from the end of the best epoch: 1.
Epoch 2/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step - loss: 0.0515 - rmse_metric: 0.0515
Epoch 1: val_loss improved from 0.25851 to 0.21409, saving model to /content/drive/MyDrive/Time Series Project/Stock Price/Model/model_path/intep-hyper7.keras
55/55 ━━━━━━━━━━━━━━━━━━━━ 7s 130ms/step - loss: 0.0514 - rmse_metric: 0.0514 - val_loss: 0.2141 - val_rmse_metric: 0.2148
Restoring model weights from the end of the best epoch: 1.
Epoch 3/50
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step - loss: 0.0406 - rmse_metric: 0.0406
Epoch 1: val_loss did not improve from 0.21409
55/55 ━━━

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_22 (LSTM)                       │ (None, 30, 64)              │          25,600 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_23 (LSTM)                       │ (None, 30, 64)              │          33,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_10 (Dropout)                 │ (None, 30, 64)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_24 (LSTM)                       │ (None, 30, 32)              │          12,416 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_25 (LSTM)                       │ (None, 32)                  │           8,320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_6 (Flatten)                  │ (None, 32)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_13 (Dense)                     │ (None, 64)                  │           2,112 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_11 (Dropout)                 │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_14 (Dense)                     │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_15 (Dense)                     │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 250,757 (979.52 KB)

 Trainable params: 83,585 (326.50 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 167,172 (653.02 KB)

##### 8